# TPC-DI 

## Setup

In [1]:
import os
from datetime import datetime
from lxml import etree

import pandas as pd
from tqdm.notebook import tqdm, trange
import numpy as np

import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy import text 

In [2]:
# Database connection details
host = "localhost"
user = "root"
password = "password"
database = "tpcdi_sf5"

# Create the SQLAlchemy engine
engine = create_engine(f"mysql+mysqlconnector://{user}:{password}@{host}/{database}?allow_local_infile=true")

## Historical Load

In [3]:
start_time = datetime.now()

In [4]:
BATCH_ID = 1
DATA_DIR = f"..\\data\\sf5\\Batch{BATCH_ID}\\"

In [5]:
with open(DATA_DIR + "BatchDate.txt", "r") as f:
    BATCH_DATE = f.read().strip()
BATCH_DATE = pd.to_datetime(BATCH_DATE, format="%Y-%m-%d")

### dimDate

In [6]:
date_df = pd.read_csv(
    DATA_DIR + "Date.txt",
    sep="|",
    header=None,
    names=[
        "SK_DateID", "DateValue", "DateDesc", "CalendarYearID", "CalendarYearDesc", 
        "CalendarQtrID", "CalendarQtrDesc", "CalendarMonthID", "CalendarMonthDesc", 
        "CalendarWeekID", "CalendarWeekDesc", "DayOfWeekNum", "DayOfWeekDesc", 
        "FiscalYearID", "FiscalYearDesc", "FiscalQtrID", "FiscalQtrDesc", "HolidayFlag"
    ],
    parse_dates=["DateValue"],
    dtype={
        "SK_DateID": "uint32", "DateDesc": "str", "CalendarYearID": "uint16", "CalendarYearDesc": "str",
        "CalendarQtrID": "uint16", "CalendarQtrDesc": "str", "CalendarMonthID": "uint32", "CalendarMonthDesc": "str",
        "CalendarWeekID": "uint32", "CalendarWeekDesc": "str", "DayOfWeekNum": "uint8", "DayOfWeekDesc": "str",
        "FiscalYearID": "uint16", "FiscalYearDesc": "str", "FiscalQtrID": "uint16", "FiscalQtrDesc": "str",
        "HolidayFlag": "bool"
    }
)
date_df.head()

,SK_DateID,DateValue,DateDesc,CalendarYearID,CalendarYearDesc,CalendarQtrID,CalendarQtrDesc,CalendarMonthID,CalendarMonthDesc,CalendarWeekID,CalendarWeekDesc,DayOfWeekNum,DayOfWeekDesc,FiscalYearID,FiscalYearDesc,FiscalQtrID,FiscalQtrDesc,HolidayFlag
0,19500101,1950-01-01,"January 1, 1950",1950,1950,19501,1950 Q1,19501,1950 January,19501,1950-W1,7,Sunday,1950,1950,19503,1950 Q3,True
1,19500102,1950-01-02,"January 2, 1950",1950,1950,19501,1950 Q1,19501,1950 January,19501,1950-W1,1,Monday,1950,1950,19503,1950 Q3,False
2,19500103,1950-01-03,"January 3, 1950",1950,1950,19501,1950 Q1,19501,1950 January,19501,1950-W1,2,Tuesday,1950,1950,19503,1950 Q3,False
3,19500104,1950-01-04,"January 4, 1950",1950,1950,19501,1950 Q1,19501,1950 January,19501,1950-W1,3,Wednesday,1950,1950,19503,1950 Q3,False
4,19500105,1950-01-05,"January 5, 1950",1950,1950,19501,1950 Q1,19501,1950 January,19501,1950-W1,4,Thursday,1950,1950,19503,1950 Q3,False


In [7]:
dtypes = {
    "SK_DateID": sqlalchemy.types.BigInteger,
    "DateValue": sqlalchemy.types.Date,
    "DateDesc": sqlalchemy.types.CHAR(length=20),
    "CalendarYearID": sqlalchemy.types.Integer,
    "CalendarYearDesc": sqlalchemy.types.CHAR(length=20),
    "CalendarQtrID": sqlalchemy.types.Integer,
    "CalendarQtrDesc": sqlalchemy.types.CHAR(length=20),
    "CalendarMonthID": sqlalchemy.types.Integer,
    "CalendarMonthDesc": sqlalchemy.types.CHAR(length=20),
    "CalendarWeekID": sqlalchemy.types.Integer,
    "CalendarWeekDesc": sqlalchemy.types.CHAR(length=20),
    "DayOfWeekNum": sqlalchemy.types.SmallInteger,
    "DayOfWeekDesc": sqlalchemy.types.CHAR(length=10),
    "FiscalYearID": sqlalchemy.types.Integer,
    "FiscalYearDesc": sqlalchemy.types.CHAR(length=20),
    "FiscalQtrID": sqlalchemy.types.Integer,
    "FiscalQtrDesc": sqlalchemy.types.CHAR(length=20),
    "HolidayFlag": sqlalchemy.types.Boolean
}

In [8]:
create_table = """CREATE TABLE DimDate (
    SK_DateID INT UNSIGNED NOT NULL,
    DateValue DATE NOT NULL,
    DateDesc CHAR(20) NOT NULL,
    CalendarYearID SMALLINT UNSIGNED NOT NULL,
    CalendarYearDesc CHAR(20) NOT NULL,
    CalendarQtrID SMALLINT UNSIGNED NOT NULL,
    CalendarQtrDesc CHAR(20) NOT NULL,
    CalendarMonthID MEDIUMINT UNSIGNED NOT NULL,
    CalendarMonthDesc CHAR(20) NOT NULL,
    CalendarWeekID MEDIUMINT UNSIGNED NOT NULL,
    CalendarWeekDesc CHAR(20) NOT NULL,
    DayOfWeekNum TINYINT UNSIGNED NOT NULL,
    DayOfWeekDesc CHAR(10) NOT NULL,
    FiscalYearID SMALLINT UNSIGNED NOT NULL,
    FiscalYearDesc CHAR(20) NOT NULL,
    FiscalQtrID SMALLINT UNSIGNED NOT NULL,
    FiscalQtrDesc CHAR(20) NOT NULL,
    HolidayFlag BOOLEAN,
    PRIMARY KEY (SK_DateID)
);"""
with engine.connect() as conn:
    conn.execute(text(create_table))

In [9]:
date_df.to_sql(name='dimdate', con=engine, if_exists='append', index=False, dtype=dtypes)

25933

### dimTime

In [10]:
# Read the data from the file
file_path = DATA_DIR + "Time.txt"
dim_time_df = pd.read_csv(
    file_path,
    sep="|",
    header=None,
    names=[
        "SK_TimeID", "TimeValue", "HourID", "HourDesc", 
        "MinuteID", "MinuteDesc", "SecondID", "SecondDesc",
        "MarketHoursFlag", "OfficeHoursFlag"
    ],
    dtype={
        "SK_TimeID": "uint32", "HourID": "uint8", 
        "HourDesc": "str", "MinuteID": "uint8", "MinuteDesc": "str", 
        "SecondID": "uint8", "SecondDesc": "str", "MarketHoursFlag": "bool", 
        "OfficeHoursFlag": "bool"
    },
    parse_dates=["TimeValue"],
    date_format="%H:%M:%S"
)
dim_time_df['TimeValue'] = dim_time_df['TimeValue'].dt.time

dim_time_df.head()

,SK_TimeID,TimeValue,HourID,HourDesc,MinuteID,MinuteDesc,SecondID,SecondDesc,MarketHoursFlag,OfficeHoursFlag
0,0,00:00:00,0,00,0,00:00,0,00:00:00,False,False
1,1,00:00:01,0,00,0,00:00,1,00:00:01,False,False
2,2,00:00:02,0,00,0,00:00,2,00:00:02,False,False
3,3,00:00:03,0,00,0,00:00,3,00:00:03,False,False
4,4,00:00:04,0,00,0,00:00,4,00:00:04,False,False


In [11]:
dtypes = {
    "SK_TimeID": sqlalchemy.types.BigInteger,
    "TimeValue": sqlalchemy.types.Time,
    "HourID": sqlalchemy.types.SmallInteger,
    "HourDesc": sqlalchemy.types.CHAR(length=20),
    "MinuteID": sqlalchemy.types.SmallInteger,
    "MinuteDesc": sqlalchemy.types.CHAR(length=20),
    "SecondID": sqlalchemy.types.SmallInteger,
    "SecondDesc": sqlalchemy.types.CHAR(length=20),
    "MarketHoursFlag": sqlalchemy.types.Boolean,
    "OfficeHoursFlag": sqlalchemy.types.Boolean
}

In [12]:
create_table = """CREATE TABLE DimTime (
    SK_TimeID INT UNSIGNED NOT NULL,
    TimeValue TIME(3) NOT NULL,
    HourID TINYINT UNSIGNED NOT NULL,
    HourDesc CHAR(20) NOT NULL,
    MinuteID TINYINT UNSIGNED NOT NULL,
    MinuteDesc CHAR(20) NOT NULL,
    SecondID TINYINT UNSIGNED NOT NULL,
    SecondDesc CHAR(20) NOT NULL,
    MarketHoursFlag BOOLEAN,
    OfficeHoursFlag BOOLEAN,
    PRIMARY KEY (SK_TimeID)
);"""
with engine.connect() as conn:
    conn.execute(text(create_table))
dim_time_df.to_sql(name='dimtime', con=engine, if_exists='append', index=False, dtype=dtypes)

86400

### Industry

In [13]:
# Read the data from the file
file_path = r"..\data\sf5\Batch1\Industry.txt"
industry_df = pd.read_csv(
    file_path,
    sep="|",
    header=None,
    names=["IN_ID", "IN_NAME", "IN_SC_ID"],
    dtype={
        "IN_ID": "str",
        "IN_NAME": "str",
        "IN_SC_ID": "str"
    }
)

In [14]:
sql_dtypes = {
    "IN_ID": sqlalchemy.types.CHAR(length=2),
    "IN_NAME": sqlalchemy.types.CHAR(length=50),
    "IN_SC_ID": sqlalchemy.types.CHAR(length=4)
}

In [15]:
create_table = """CREATE TABLE Industry (
    IN_ID CHAR(2) NOT NULL,
    IN_NAME CHAR(50) NOT NULL,
    IN_SC_ID CHAR(4) NOT NULL,
    PRIMARY KEY (IN_ID)
);"""
with engine.connect() as conn:
    conn.execute(text(create_table))

In [16]:
industry_df.to_sql(name="industry", con=engine, if_exists='append', index=False, dtype=dtypes)

102

### StatusType

In [17]:
# Read StatusType data
filepath = r"..\data\sf5\Batch1\StatusType.txt"
status_type_df = pd.read_csv(
    filepath,
    sep="|",
    header=None,
    names=["ST_ID", "ST_NAME"],
    dtype={"ST_ID": "str", "ST_NAME": "str"}
)
status_type_df.head()


,ST_ID,ST_NAME
0,ACTV,Active
1,CMPT,Completed
2,CNCL,Canceled
3,PNDG,Pending
4,SBMT,Submitted


In [18]:
sql_dtypes = {
    "ST_ID": sqlalchemy.types.CHAR(length=4),
    "ST_NAME": sqlalchemy.types.CHAR(length=10)
}

In [19]:
with engine.connect() as conn:
    conn.execute(text("""CREATE TABLE StatusType (
    ST_ID CHAR(4) NOT NULL,
    ST_NAME CHAR(10) NOT NULL,
    PRIMARY KEY (ST_ID)
);"""))

In [20]:
status_type_df.to_sql(name="statustype", con=engine, if_exists='append', index=False, dtype=sql_dtypes)

6

### TradeType 

In [21]:
# Read TradeType data
filepath = r"..\data\sf5\Batch1\TradeType.txt"
trade_type_df = pd.read_csv(
    filepath,
    sep="|",
    header=None,
    names=["TT_ID", "TT_NAME", "TT_IS_SELL", "TT_IS_MRKT"],
    dtype={"TT_ID": "str", "TT_NAME": "str", "TT_IS_SELL": "uint8", "TT_IS_MRKT": "uint8"}
)

In [22]:
dtypes = {
    "TT_ID": sqlalchemy.types.CHAR(length=3),
    "TT_NAME": sqlalchemy.types.CHAR(length=12),
    "TT_IS_SELL": sqlalchemy.types.SmallInteger,
    "TT_IS_MRKT": sqlalchemy.types.SmallInteger
}
with engine.connect() as conn:
    conn.execute(text("""CREATE TABLE TradeType (
    TT_ID CHAR(3) NOT NULL,
    TT_NAME CHAR(12) NOT NULL,
    TT_IS_SELL TINYINT UNSIGNED NOT NULL CHECK (TT_IS_SELL IN (0, 1)),
    TT_IS_MRKT TINYINT UNSIGNED NOT NULL CHECK (TT_IS_MRKT IN (0, 1)),
    PRIMARY KEY (TT_ID)
);"""))
trade_type_df.to_sql(name="tradetype", con=engine, if_exists='append', index=False, dtype=dtypes)

5

### TaxRate 

In [23]:
# Read TaxRate data
filepath = r"..\data\sf5\Batch1\TaxRate.txt"
tax_rate_df = pd.read_csv(
    filepath,
    sep="|",
    header=None,
    names=["TX_ID", "TX_NAME", "TX_RATE"],
    dtype={"TX_ID": "str", "TX_NAME": "str", "TX_RATE": "float64"}
)

In [24]:
sql_dtypes = {
    "TX_ID": sqlalchemy.types.CHAR(length=4),
    "TX_NAME": sqlalchemy.types.CHAR(length=50),
    "TX_RATE": sqlalchemy.types.Numeric(precision=6, scale=5)
}

with engine.connect() as conn:
    conn.execute(text("""CREATE TABLE TaxRate (
    TX_ID CHAR(4) NOT NULL,
    TX_NAME CHAR(50) NOT NULL,
    TX_RATE DECIMAL(6, 5) NOT NULL,
    PRIMARY KEY (TX_ID)
);"""))

tax_rate_df.to_sql(name="taxrate", con=engine, if_exists='append', index=False, dtype=sql_dtypes)

320

### dimBroker

In [25]:
hr_df = pd.read_csv(
    r"..\data\sf5\Batch1\HR.csv",
    sep=",",
    header=None,
    names=[
        "EmployeeID", "ManagerID", "EmployeeFirstName", "EmployeeLastName",
        "EmployeeMI", "EmployeeJobCode", "EmployeeBranch",
        "EmployeeOffice", "EmployeePhone"
    ],
    dtype={
        "EmployeeID": "uint32",
        "ManagerID": "uint32",
        "EmployeeFirstName": "str",
        "EmployeeLastName": "str",
        "EmployeeMI": "str",
        "EmployeeJobCode": "UInt16",
        "EmployeeBranch": "str",
        "EmployeeOffice": "str",
        "EmployeePhone": "str"
    }
)
hr_df.head()

,EmployeeID,ManagerID,EmployeeFirstName,EmployeeLastName,EmployeeMI,EmployeeJobCode,EmployeeBranch,EmployeeOffice,EmployeePhone
0,0,351,Ozkan,Douglas,NaN,647,EGZKSobTeknHCbLuHczvWmhTmCSGXD,OFFICE7152,(726) 088-3331
1,1,689,Suer,Candice,NaN,314,OfOBVvpzNvHCebxyuxXFwsMju JRU,OFFICE8586,(344) 999-2652
2,2,410,Somisetty,Jami,P,534,rAHWYkktOXAyPAYHlncZPG,NaN,(984) 538-5366
3,3,412,Mazurek,Rosalinda,J,364,TJQqsUQQGqWG QleLheUoYlgRNVT,OFFICE8487,(860) 037-6897
4,4,2173,Aronovich,Delphine,M,314,IEMJHuQgCPDHCwwJkgQQeaqGvzMcVD,OFFICE9420,(604) 387-9350


In [26]:
# 1. Filter Records
filtered_df = hr_df[hr_df['EmployeeJobCode'] == 314]
filtered_df.drop(columns=['EmployeeJobCode'], inplace=True)
# 2. Map Columns
DimBroker = filtered_df.rename(columns={
    'EmployeeID': 'BrokerID',
    'ManagerID': 'ManagerID',
    'EmployeeFirstName': 'FirstName',
    'EmployeeLastName': 'LastName',
    'EmployeeMI': 'MiddleInitial',
    'EmployeeBranch': 'Branch',
    'EmployeeOffice': 'Office',
    'EmployeePhone': 'Phone'
})
# 3. Handle Surrogate Key (SK_BrokerID)
# Using cumcount to generate a unique ID for each row
DimBroker.loc[:, 'SK_BrokerID'] = range(1, len(DimBroker) + 1)

# 4. Set Default Values for New Fields
DimBroker.loc[:, 'IsCurrent'] = True
DimBroker.loc[:, 'BatchID'] = BATCH_ID
# EffectiveDate is set to the earliest date in the DimDate table and EndDate is set to 9999- 12-31
DimBroker.loc[:, 'EffectiveDate'] = pd.read_sql_query("SELECT MIN(datevalue) FROM dimdate", engine).iloc[0, 0]
DimBroker.loc[:, 'EndDate'] = pd.Timestamp('9999-12-31')

# Display the first few rows of the newly created DimBroker DataFrame
DimBroker.head()

C:\Users\strai\AppData\Local\Temp\ipykernel_15716\84927967.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.drop(columns=['EmployeeJobCode'], inplace=True)


,BrokerID,ManagerID,FirstName,LastName,MiddleInitial,Branch,Office,Phone,SK_BrokerID,IsCurrent,BatchID,EffectiveDate,EndDate
1,1,689,Suer,Candice,NaN,OfOBVvpzNvHCebxyuxXFwsMju JRU,OFFICE8586,(344) 999-2652,1,True,1,1950-01-01,9999-12-31
4,4,2173,Aronovich,Delphine,M,IEMJHuQgCPDHCwwJkgQQeaqGvzMcVD,OFFICE9420,(604) 387-9350,2,True,1,1950-01-01,9999-12-31
8,8,1073,Hansen,Montreal,T,sGIpORbLsRjTdhqBNBQppRRkp,OFFICE6343,(991) 491-4907,3,True,1,1950-01-01,9999-12-31
11,11,1130,Charchanko,Sheela,NaN,Cw QJMHPgpozCKsFZMmmbokwSRTPD,OFFICE7705,(977) 726-0106,4,True,1,1950-01-01,9999-12-31
14,14,1832,Knorp,Uday,NaN,QmCLAAAJibegHoPZcQJCdDfukiRQo,OFFICE6437,(254) 560-8156,5,True,1,1950-01-01,9999-12-31


In [27]:
dtypes = {
    "SK_BrokerID": sqlalchemy.types.BigInteger,
    "BrokerID": sqlalchemy.types.BigInteger,
    "ManagerID": sqlalchemy.types.BigInteger,
    "FirstName": sqlalchemy.types.CHAR(length=50),
    "LastName": sqlalchemy.types.CHAR(length=50),
    "MiddleInitial": sqlalchemy.types.CHAR(length=1),
    "Branch": sqlalchemy.types.CHAR(length=50),
    "Office": sqlalchemy.types.CHAR(length=50),
    "Phone": sqlalchemy.types.CHAR(length=14),
    "IsCurrent": sqlalchemy.types.Boolean,
    "BatchID": sqlalchemy.types.Integer,
    "EffectiveDate": sqlalchemy.types.Date,
    "EndDate": sqlalchemy.types.Date
}

In [28]:
create_table = """CREATE TABLE DimBroker (
    SK_BrokerID INT UNSIGNED NOT NULL,
    BrokerID INT UNSIGNED NOT NULL,
    ManagerID INT UNSIGNED,
    FirstName CHAR(50) NOT NULL,
    LastName CHAR(50) NOT NULL,
    MiddleInitial CHAR(1),
    Branch CHAR(50),
    Office CHAR(50),
    Phone CHAR(14),
    IsCurrent BOOLEAN NOT NULL,
    BatchID SMALLINT UNSIGNED NOT NULL,
    EffectiveDate DATE NOT NULL,
    EndDate DATE NOT NULL,
    PRIMARY KEY (SK_BrokerID)
);"""
with engine.connect() as conn:
    conn.execute(text(create_table))

In [29]:
DimBroker.to_sql(name='dimbroker', con=engine, if_exists='append', index=False, dtype=dtypes)

7144

### dimCompany

In [30]:
def read_finwire(file_path):
    # Define the column widths and names
    col_widths = [15, 3, 60, 10, 4, 2, 4, 8, 80, 80, 12, 25, 20, 24, 46, 150]
    col_names = [
        "PTS", "RecType", "CompanyName", "CIK", "Status", "IndustryID",
        "SPrating", "FoundingDate", "AddrLine1", "AddrLine2", "PostalCode",
        "City", "StateProvince", "Country", "CEOname", "Description"
    ]
    # Read the fixed-width file
    df = pd.read_fwf(file_path, widths=col_widths, header=None, names=col_names)

    # Filter the DataFrame for CMP records
    df_cmp = df[df['RecType'] == 'CMP']
    return df_cmp

In [31]:
# Query StatusType table and create a mapping dictionary
with engine.connect() as conn:
    statustype_df = pd.read_sql("SELECT * FROM statustype", conn)
status_mapping = dict(statustype_df[["ST_ID", "ST_NAME"]].values)

# Query Industry table and create a mapping dictionary
with engine.connect() as conn:
    industry_df = pd.read_sql("SELECT * FROM industry", conn)
industry_mapping = dict(industry_df[["IN_ID", "IN_NAME"]].values)

# Valid SPrating values
valid_spratings = [
    "AAA",
    "AA+",
    "AA",
    "AA-",
    "A+",
    "A",
    "A-",
    "BBB+",
    "BBB",
    "BBB-",
    "BB+",
    "BB",
    "BB-",
    "B+",
    "B",
    "B-",
    "CCC+",
    "CCC",
    "CCC-",
    "CC",
    "C",
    "D",
]

In [32]:
files = os.listdir(DATA_DIR)
finwire_files = [file for file in files if file.startswith("FINWIRE") and 'audit' not in file]

In [33]:
sql_dtypes = {
    "SK_CompanyID": sqlalchemy.types.BigInteger,
    "CompanyID": sqlalchemy.types.BigInteger,
    "Status": sqlalchemy.types.CHAR(length=10),
    "Name": sqlalchemy.types.CHAR(length=60),
    "Industry": sqlalchemy.types.CHAR(length=50),
    "SPrating": sqlalchemy.types.CHAR(length=4),
    "isLowGrade": sqlalchemy.types.Boolean,
    "CEO": sqlalchemy.types.CHAR(length=100),
    "AddressLine1": sqlalchemy.types.CHAR(length=80),
    "AddressLine2": sqlalchemy.types.CHAR(length=80),
    "PostalCode": sqlalchemy.types.CHAR(length=12),
    "City": sqlalchemy.types.CHAR(length=25),
    "StateProv": sqlalchemy.types.CHAR(length=20),
    "Country": sqlalchemy.types.CHAR(length=24),
    "Description": sqlalchemy.types.CHAR(length=150),
    "FoundingDate": sqlalchemy.types.Date,
    "IsCurrent": sqlalchemy.types.Boolean,
    "BatchID": sqlalchemy.types.Integer,
    "EffectiveDate": sqlalchemy.types.Date,
    "EndDate": sqlalchemy.types.Date
}

In [34]:
def load_dimcompany(filename, is_first_batch=False):
    file_path = DATA_DIR + filename
    df_cmp = read_finwire(file_path)

    if len(df_cmp) == 0:
        return
    
    # Define the column names and data types
    column_names = [
        "SK_CompanyID",
        "CompanyID",
        "Status",
        "Name",
        "Industry",
        "SPrating",
        "isLowGrade",
        "CEO",
        "AddressLine1",
        "AddressLine2",
        "PostalCode",
        "City",
        "StateProv",
        "Country",
        "Description",
        "FoundingDate",
        "IsCurrent",
        "BatchID",
        "EffectiveDate",
        "EndDate",
    ]
    dtypes = {
        "SK_CompanyID": "uint32",
        "CompanyID": "uint32",
        "Status": "str",
        "Name": "str",
        "Industry": "str",
        "SPrating": "str",
        "isLowGrade": "boolean",
        "CEO": "str",
        "AddressLine1": "str",
        "AddressLine2": "str",
        "PostalCode": "str",
        "City": "str",
        "StateProv": "str",
        "Country": "str",
        "Description": "str",
        "FoundingDate": "datetime64[ns]",
        "IsCurrent": "bool",
        "BatchID": "uint8",
        "EffectiveDate": "datetime64[ns]",
        "EndDate": "datetime64[ns]",
    }
    # Create an empty DataFrame with the specified schema
    dimCompany = pd.DataFrame(columns=column_names).astype(dtypes)

    # Copy and map relevant columns
    df_cmp["CIK"] = pd.to_numeric(df_cmp["CIK"], downcast="unsigned")
    dimCompany["CompanyID"] = pd.to_numeric(df_cmp["CIK"], downcast="unsigned")
    dimCompany["Name"] = df_cmp["CompanyName"].str.strip()
    dimCompany["SPrating"] = df_cmp["SPrating"].str.upper()
    dimCompany["CEO"] = df_cmp["CEOname"].str.strip()
    dimCompany["Description"] = df_cmp["Description"].str.strip()
    dimCompany["FoundingDate"] = pd.to_datetime(
        df_cmp["FoundingDate"], format="%Y%m%d", errors="coerce"
    )

    # For address fields
    dimCompany["AddressLine1"] = df_cmp["AddrLine1"].str.strip()
    dimCompany["AddressLine2"] = df_cmp["AddrLine2"].str.strip()
    dimCompany["PostalCode"] = df_cmp["PostalCode"].str.strip()
    dimCompany["City"] = df_cmp["City"].str.strip()
    dimCompany["StateProv"] = df_cmp["StateProvince"].str.strip()
    dimCompany["Country"] = df_cmp["Country"].str.strip()

    # Replace all-blank strings with None (NULL)
    for col in [
        "Name",
        "SPrating",
        "CEO",
        "Description",
        "AddressLine1",
        "AddressLine2",
        "PostalCode",
        "City",
        "StateProv",
        "Country",
    ]:
        dimCompany[col] = dimCompany[col].replace(r"^\s*$", None, regex=True)

    # Update Status in dimCompany
    dimCompany["Status"] = df_cmp["Status"].map(status_mapping)
    # Update Industry in dimCompany
    dimCompany["Industry"] = df_cmp["IndustryID"].map(industry_mapping)
    # isLowGrade is set to False if SPrating begins with ‘A’ or ‘BBB’ otherwise set to True
    dimCompany["isLowGrade"] = ~df_cmp["SPrating"].str.startswith(("A", "BBB"))

    # Identify invalid SPratings
    invalid_sprating_mask = ~dimCompany["SPrating"].isin(valid_spratings)
    # Filter dimCompany for invalid SPrating
    invalid_sprating_data = dimCompany[invalid_sprating_mask]
    if len(invalid_sprating_data) > 0:
        message_data = (
            "CO_ID = "
            + invalid_sprating_data["CompanyID"].astype(str)
            + ", CO_SP_RATE = "
            + invalid_sprating_data["SPrating"]
        )
        # Create DImessages DataFrame
        dimessages = pd.DataFrame(
            {
                "MessageDateAndTime": [datetime.now()] * len(message_data),
                "BatchID": [1] * len(message_data),
                "MessageSource": ["DimCompany"] * len(message_data),
                "MessageText": ["Invalid SPRating"] * len(message_data),
                "MessageType": ["Alert"] * len(message_data),
                "MessageData": message_data,
            }
        )
        # Update dimCompany for invalid SPrating
        dimCompany.loc[invalid_sprating_mask, ["SPrating", "isLowGrade"]] = pd.NA
        # Insert DImessages into MySQL
        dimessages.to_sql("dimessages", engine, if_exists="append", index=False)

    dimCompany.loc[:, "BatchID"] = 1
    dimCompany["EffectiveDate"] = pd.to_datetime(df_cmp["PTS"], format="%Y%m%d-%H%M%S")
    # Identify new and existing records based on CIK
    if is_first_batch:
        new_records = dimCompany
        existing_records = pd.DataFrame(columns=column_names).astype(dtypes)
        next_sk_id = 0
    else:
        existing_cik = pd.read_sql_query(
            "SELECT CompanyID FROM dimCompany WHERE IsCurrent = 1", engine
        )["CompanyID"]
        new_records = dimCompany[~df_cmp["CIK"].isin(existing_cik)]
        existing_records = dimCompany[df_cmp["CIK"].isin(existing_cik)]
        next_sk_id_query = "SELECT MAX(SK_CompanyID) FROM dimCompany"
        next_sk_id = pd.read_sql_query(next_sk_id_query, engine).iloc[0, 0] or 0

    new_records.loc[:, "SK_CompanyID"] = range(
        next_sk_id + 1, next_sk_id + 1 + len(new_records)
    )
    new_records.loc[:, "IsCurrent"] = True
    new_records.loc[:, "EndDate"] = pd.Timestamp("9999-12-31")
    new_records.to_sql(
        "dimcompany", engine, if_exists="append", index=False, dtype=sql_dtypes
    )
    next_sk_id = new_records["SK_CompanyID"].max()

    # Process existing records
    for _, row in existing_records.iterrows():
        effective_date = row["EffectiveDate"]
        company_id = row["CompanyID"]
        # get the current record
        select_query = f"""SELECT * FROM dimCompany
        WHERE CompanyID = '{company_id}' AND IsCurrent = 1"""
        existing_record = pd.read_sql(select_query, engine).iloc[0]
        compare_cols = [
            "Status",
            "Name",
            "Industry",
            "SPrating",
            "isLowGrade",
            "CEO",
            "AddressLine1",
            "AddressLine2",
            "PostalCode",
            "City",
            "StateProv",
            "Country",
            "Description",
            "FoundingDate",
        ]
        is_same = True
        for col in compare_cols:
            if row[col] != existing_record[col]:
                is_same = False
                break
        if not is_same:
            # Expire the current record in MySQL
            update_query = f"""UPDATE dimcompany 
            SET IsCurrent = 0, EndDate = '{effective_date}' 
            WHERE CompanyID = '{company_id}' AND IsCurrent = 1
            """
            with engine.connect() as conn:
                conn.execute(text(update_query))
                conn.commit()
            # Insert updated record
            row["SK_CompanyID"] = next_sk_id + 1
            row["IsCurrent"] = True
            row["EndDate"] = pd.Timestamp("9999-12-31")
            row_df = pd.DataFrame(row).T
            # insert records with existing SK_CompanyID
            row_df.to_sql(
                "dimcompany", engine, if_exists="append", index=False, dtype=sql_dtypes
            )        
            next_sk_id += 1

    return df_cmp, new_records, existing_records

In [35]:
create_table = """CREATE TABLE DimCompany (
    SK_CompanyID INT UNSIGNED NOT NULL,
    CompanyID INT UNSIGNED NOT NULL,
    Status CHAR(10) NOT NULL,
    Name CHAR(60) NOT NULL,
    Industry CHAR(50) NOT NULL,
    SPrating CHAR(4),
    isLowGrade BOOLEAN,
    CEO CHAR(100) NOT NULL,
    AddressLine1 CHAR(80),
    AddressLine2 CHAR(80),
    PostalCode CHAR(12) NOT NULL,
    City CHAR(25) NOT NULL,
    StateProv CHAR(20) NOT NULL,
    Country CHAR(24),
    Description CHAR(150) NOT NULL,
    FoundingDate DATE,
    IsCurrent BOOLEAN NOT NULL,
    BatchID SMALLINT UNSIGNED NOT NULL,
    EffectiveDate DATE NOT NULL,
    EndDate DATE NOT NULL,
    PRIMARY KEY (SK_CompanyID)
);"""
with engine.connect() as conn:
    conn.execute(text(create_table))

In [36]:
for i, file in enumerate(tqdm(finwire_files)):
    load_dimcompany(file, i == 0)

  0%|          | 0/203 [00:00<?, ?it/s]

C:\Users\strai\AppData\Local\Temp\ipykernel_15716\221389655.py:120: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  dimCompany.loc[invalid_sprating_mask, ["SPrating", "isLowGrade"]] = pd.NA
C:\Users\strai\AppData\Local\Temp\ipykernel_15716\221389655.py:120: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  dimCompany.loc[invalid_sprating_mask, ["SPrating", "isLowGrade"]] = pd.NA
C:\Users\strai\AppData\Local\Temp\ipykernel_15716\221389655.py:120: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'nan' has dtype incompatible with bool, please explicitly cast to a compatible dtype first.
  dimCompany.loc[

### Financial

In [37]:
def read_finwire_fin(file_path):
    # Define the column widths and names
    col_widths = [15, 3, 4, 1, 8, 8, 17, 17, 12, 12, 12, 17, 17, 17, 13, 13, 60]
    col_names = [
        "PTS", "RecType", "Year", "Quarter", "QtrStartDate", "PostingDate",
        "Revenue", "Earnings", "EPS", "DilutedEPS", "Margin", "Inventory",
        "Assets", "Liabilities", "ShOut", "DilutedShOut", "CoNameOrCIK"
    ]
    # Read the fixed-width file
    df_fin = pd.read_fwf(file_path, widths=col_widths, header=None, names=col_names)
    # Filter the DataFrame for CMP records
    df_fin = df_fin[df_fin['RecType'] == 'FIN']
    # Convert PTS to datetime
    df_fin['PTS'] = pd.to_datetime(df_fin['PTS'], format='%Y%m%d-%H%M%S')

    return df_fin

In [38]:
def load_financial():
    files = os.listdir(DATA_DIR)
    finwire_files = [file for file in files if file.startswith("FINWIRE") and 'audit' not in file]
    

    for filename in tqdm(finwire_files):
        file_path = DATA_DIR + filename
        df_fin = read_finwire_fin(file_path)
        if len(df_fin) == 0:
            continue
        # datatypes for the mysql table
        sql_dtypes = {
            "SK_CompanyID": sqlalchemy.types.BigInteger,
            "FI_YEAR": sqlalchemy.types.Integer,
            "FI_QTR": sqlalchemy.types.SmallInteger,
            "FI_QTR_START_DATE": sqlalchemy.types.Date,
            "FI_REVENUE": sqlalchemy.types.Numeric(precision=15, scale=2),
            "FI_NET_EARN": sqlalchemy.types.Numeric(precision=15, scale=2),
            "FI_BASIC_EPS": sqlalchemy.types.Numeric(precision=10, scale=2),
            "FI_DILUT_EPS": sqlalchemy.types.Numeric(precision=10, scale=2),
            "FI_MARGIN": sqlalchemy.types.Numeric(precision=10, scale=2),
            "FI_INVENTORY": sqlalchemy.types.Numeric(precision=15, scale=2),
            "FI_ASSETS": sqlalchemy.types.Numeric(precision=15, scale=2),
            "FI_LIABILITY": sqlalchemy.types.Numeric(precision=15, scale=2),
            "FI_OUT_BASIC": sqlalchemy.types.BigInteger,
            "FI_OUT_DILUT": sqlalchemy.types.BigInteger
        }

        # data types for the DataFrame
        dtypes = {
            'SK_CompanyID': 'uint32',
            'FI_YEAR': 'uint16',
            'FI_QTR': 'uint8',
            'FI_QTR_START_DATE': 'datetime64[ns]',
            'FI_REVENUE': 'float64',
            'FI_NET_EARN': 'float64',
            'FI_BASIC_EPS': 'float64',
            'FI_DILUT_EPS': 'float64',
            'FI_MARGIN': 'float64',
            'FI_INVENTORY': 'float64',
            'FI_ASSETS': 'float64',
            'FI_LIABILITY': 'float64',
            'FI_OUT_BASIC': 'uint64',
            'FI_OUT_DILUT': 'uint64'
        }

        # Create empty DataFrame
        financial_df = pd.DataFrame({col: pd.Series(dtype=typ) for col, typ in dtypes.items()})

        # copy directly
        financial_df['FI_YEAR'] = pd.to_numeric(df_fin['Year'].str.strip(), downcast='unsigned')
        financial_df['FI_QTR'] = pd.to_numeric(df_fin['Quarter'].str.strip(), downcast='unsigned')
        financial_df['FI_QTR_START_DATE'] = pd.to_datetime(df_fin['QtrStartDate'], format='%Y%m%d')
        financial_df['FI_REVENUE'] = pd.to_numeric(df_fin['Revenue'].str.strip(), downcast='float')
        financial_df['FI_NET_EARN'] = pd.to_numeric(df_fin['Earnings'].str.strip(), downcast='float')
        financial_df['FI_BASIC_EPS'] = pd.to_numeric(df_fin['EPS'].str.strip(), downcast='float')
        financial_df['FI_DILUT_EPS'] = pd.to_numeric(df_fin['DilutedEPS'].str.strip(), downcast='float')
        financial_df['FI_MARGIN'] = pd.to_numeric(df_fin['Margin'].str.strip(), downcast='float')
        financial_df['FI_INVENTORY'] = pd.to_numeric(df_fin['Inventory'].str.strip(), downcast='float')
        financial_df['FI_ASSETS'] = pd.to_numeric(df_fin['Assets'].str.strip(), downcast='float')
        financial_df['FI_LIABILITY'] = pd.to_numeric(df_fin['Liabilities'].str.strip(), downcast='float')
        financial_df['FI_OUT_BASIC'] = pd.to_numeric(df_fin['ShOut'].str.strip(), downcast='unsigned')
        financial_df['FI_OUT_DILUT'] = pd.to_numeric(df_fin['DilutedShOut'].str.strip(), downcast='unsigned')

        # Split df_fin based on the length of CoNameOrCIK
        df_fin_id = df_fin[df_fin['CoNameOrCIK'].str.len() == 10][['PTS', 'CoNameOrCIK']]
        df_fin_id['PTS'] = df_fin_id['PTS'].dt.strftime('%Y-%m-%d')
        df_fin_id['CoNameOrCIK'] = pd.to_numeric(df_fin_id['CoNameOrCIK'], downcast='unsigned')
        df_fin_name = df_fin[df_fin['CoNameOrCIK'].str.len() != 10][['PTS', 'CoNameOrCIK']]
        df_fin_name['PTS'] = df_fin_name['PTS'].dt.strftime('%Y-%m-%d')
        df_fin_name['CoNameOrCIK'] = df_fin_name['CoNameOrCIK'].str.strip()

        def build_query(df, id_or_name_col):
            '''Function to build SQL query for date range checks'''
            query_parts = []
            for _, row in df.iterrows():
                pts = row['PTS']
                if id_or_name_col == 'CompanyID':
                    company_id = row['CoNameOrCIK']
                    query_part = f"(CompanyID = {company_id} AND EffectiveDate <= '{pts}' AND '{pts}' < EndDate)"
                else:  # Name
                    company_name = row['CoNameOrCIK']
                    query_part = f"(Name = '{company_name}' AND EffectiveDate <= '{pts}' AND '{pts}' < EndDate)"
                query_parts.append(query_part)
            return ' OR '.join(query_parts)

        # Execute query and map results for ID-based records
        if not df_fin_id.empty:
            query_id = f"SELECT CompanyID, SK_CompanyID FROM dimcompany WHERE " + build_query(df_fin_id, 'CompanyID')
            sk_id_map = pd.read_sql_query(query_id, engine).set_index('CompanyID')['SK_CompanyID']            
            financial_df.loc[df_fin_id.index, 'SK_CompanyID'] = df_fin_id['CoNameOrCIK'].astype(int).map(sk_id_map)

        # Execute query and map results for Name-based records
        if not df_fin_name.empty:
            query_name = f"SELECT Name, SK_CompanyID FROM dimcompany WHERE " + build_query(df_fin_name, 'Name')
            sk_name_map = pd.read_sql_query(query_name, engine).set_index('Name')['SK_CompanyID']
            financial_df.loc[df_fin_name.index, 'SK_CompanyID'] = df_fin_name['CoNameOrCIK'].map(sk_name_map)

        financial_df['SK_CompanyID'] = financial_df['SK_CompanyID'].astype('uint32')

        # perform migration
        financial_df.to_sql('financial', engine, if_exists='append', index=False, dtype=sql_dtypes)

In [39]:
create_table = """CREATE TABLE Financial (
    SK_CompanyID INT UNSIGNED NOT NULL,
    FI_YEAR YEAR NOT NULL,
    FI_QTR TINYINT UNSIGNED NOT NULL CHECK (FI_QTR IN (1, 2, 3, 4)),
    FI_QTR_START_DATE DATE NOT NULL,
    FI_REVENUE DECIMAL(15, 2) NOT NULL,
    FI_NET_EARN DECIMAL(15, 2) NOT NULL,
    FI_BASIC_EPS DECIMAL(10, 2) NOT NULL,
    FI_DILUT_EPS DECIMAL(10, 2) NOT NULL,
    FI_MARGIN DECIMAL(10, 2) NOT NULL,
    FI_INVENTORY DECIMAL(15, 2) NOT NULL,
    FI_ASSETS DECIMAL(15, 2) NOT NULL,
    FI_LIABILITY DECIMAL(15, 2) NOT NULL,
    FI_OUT_BASIC BIGINT NOT NULL,
    FI_OUT_DILUT BIGINT NOT NULL,
    PRIMARY KEY (SK_CompanyID, FI_YEAR, FI_QTR)
);"""
with engine.connect() as conn:
    conn.execute(text(create_table))

In [40]:
load_financial()

  0%|          | 0/203 [00:00<?, ?it/s]

### dimSecurity

In [41]:
def read_finwire_sec(file_path):
    # Define the column widths and names
    col_widths = [15, 3, 15, 6, 4, 70, 6, 13, 8, 8, 12, 60]
    col_names = [
        "PTS", "RecType", "Symbol", "IssueType", "Status", "Name", "ExID",
        "ShOut", "FirstTradeDate", "FirstTradeExchg", "Dividend", "CoNameOrCIK"
    ]
    # Read the fixed-width file
    df_sec = pd.read_fwf(file_path, widths=col_widths, header=None, names=col_names)
    # Filter the DataFrame for CMP records
    df_sec = df_sec[df_sec['RecType'] == 'SEC']
    # Convert date cols to datetime
    df_sec['PTS'] = pd.to_datetime(df_sec['PTS'], format='%Y%m%d-%H%M%S')
    df_sec['FirstTradeDate'] = pd.to_datetime(df_sec['FirstTradeDate'], format='%Y%m%d')
    df_sec['FirstTradeExchg'] = pd.to_datetime(df_sec['FirstTradeExchg'], format='%Y%m%d')

    return df_sec

In [42]:
sql_dtypes = {
    "SK_SecurityID": sqlalchemy.types.Integer,
    "Symbol": sqlalchemy.types.String(15),
    "Issue": sqlalchemy.types.String(6),
    "Status": sqlalchemy.types.String(10),
    "Name": sqlalchemy.types.String(70),
    "ExchangeID": sqlalchemy.types.String(6),
    "SK_CompanyID": sqlalchemy.types.Integer,
    "SharesOutstanding": sqlalchemy.types.Integer,
    "FirstTrade": sqlalchemy.types.Date,
    "FirstTradeOnExchange": sqlalchemy.types.Date,
    "Dividend": sqlalchemy.types.Numeric(10, 2),
    "IsCurrent": sqlalchemy.types.Boolean,
    "BatchID": sqlalchemy.types.SmallInteger,
    "EffectiveDate": sqlalchemy.types.Date,
    "EndDate": sqlalchemy.types.Date,
}

dtypes = {
    "SK_SecurityID": "uint32",
    "Symbol": "str",
    "Issue": "str",
    "Status": "str",
    "Name": "str",
    "ExchangeID": "str",
    "SK_CompanyID": "uint32",
    "SharesOutstanding": "uint32",
    "FirstTrade": "datetime64[ns]",
    "FirstTradeOnExchange": "datetime64[ns]",
    "Dividend": "float64",
    "IsCurrent": "bool",
    "BatchID": "uint8",
    "EffectiveDate": "datetime64[ns]",
    "EndDate": "datetime64[ns]",
}

In [43]:
# Query StatusType table and create a mapping dictionary
with engine.connect() as conn:
    statustype_df = pd.read_sql("SELECT * FROM statustype", conn)
status_mapping = dict(statustype_df[['ST_ID', 'ST_NAME']].values)

In [44]:
def build_query(df, id_or_name_col):
    '''Function to build SQL query for date range checks'''
    query_parts = []
    for _, row in df.iterrows():
        pts = row['PTS']
        if id_or_name_col == 'CompanyID':
            company_id = row['CoNameOrCIK']
            query_part = f"(CompanyID = {company_id} AND EffectiveDate <= '{pts}' AND '{pts}' < EndDate)"
        else:  # Name
            company_name = row['CoNameOrCIK']
            query_part = f"(Name = '{company_name}' AND EffectiveDate <= '{pts}' AND '{pts}' < EndDate)"
        query_parts.append(query_part)
    return ' OR '.join(query_parts)

In [45]:
def load_dimsecurity():
    finwire_files = os.listdir(DATA_DIR)
    finwire_files = [
        DATA_DIR + file
        for file in finwire_files
        if file.startswith("FINWIRE") and "audit" not in file
    ]
    for i, file in enumerate(tqdm(finwire_files)):
        # raw data from file
        df_sec = read_finwire_sec(file)
        # dimension table in data warehouse
        security_df = pd.DataFrame(
            {col: pd.Series(dtype=typ) for col, typ in dtypes.items()}
        )
        # copy directly
        security_df["Symbol"] = df_sec["Symbol"]
        security_df["Issue"] = df_sec["IssueType"]
        security_df["Name"] = df_sec["Name"]
        security_df["ExchangeID"] = df_sec["ExID"]
        security_df["SharesOutstanding"] = pd.to_numeric(
            df_sec["ShOut"], downcast="unsigned"
        )
        security_df["FirstTrade"] = df_sec["FirstTradeDate"]
        security_df["FirstTradeOnExchange"] = df_sec["FirstTradeExchg"]
        security_df["Dividend"] = pd.to_numeric(df_sec["Dividend"], downcast="float")
        # Update Status in security_df
        security_df["Status"] = df_sec["Status"].map(status_mapping)
        # BatchID is set to 1
        security_df["BatchID"] = BATCH_ID
        # Split df_sec based on the length of CoNameOrCIK
        df_sec_id = df_sec[df_sec["CoNameOrCIK"].str.len() == 10][
            ["PTS", "CoNameOrCIK"]
        ]
        df_sec_id["PTS"] = df_sec_id["PTS"].dt.strftime("%Y-%m-%d")
        df_sec_id["CoNameOrCIK"] = pd.to_numeric(
            df_sec_id["CoNameOrCIK"], downcast="unsigned"
        )
        df_sec_name = df_sec[df_sec["CoNameOrCIK"].str.len() != 10][
            ["PTS", "CoNameOrCIK"]
        ]
        df_sec_name["PTS"] = df_sec_name["PTS"].dt.strftime("%Y-%m-%d")
        df_sec_name["CoNameOrCIK"] = df_sec_name["CoNameOrCIK"].str.strip()
        # Map results for ID-based records
        if not df_sec_id.empty:
            query_id = (
                f"SELECT CompanyID, SK_CompanyID FROM dimcompany WHERE "
                + build_query(df_sec_id, "CompanyID")
            )
            sk_id_map = pd.read_sql_query(query_id, engine).set_index("CompanyID")[
                "SK_CompanyID"
            ]
            # drop duplicates from the index
            sk_id_map = sk_id_map[~sk_id_map.index.duplicated(keep="last")]
            security_df.loc[df_sec_id.index, "SK_CompanyID"] = (
                df_sec_id["CoNameOrCIK"].astype(int).map(sk_id_map)
            )
        # Map results for Name-based records
        if not df_sec_name.empty:
            query_name = (
                f"SELECT Name, SK_CompanyID FROM dimcompany WHERE "
                + build_query(df_sec_name, "Name")
            )
            sk_name_map = pd.read_sql_query(query_name, engine).set_index("Name")[
                "SK_CompanyID"
            ]
            # drop duplicates from the index
            sk_name_map = sk_name_map[~sk_name_map.index.duplicated(keep="last")]
            security_df.loc[df_sec_name.index, "SK_CompanyID"] = df_sec_name[
                "CoNameOrCIK"
            ].map(sk_name_map)
        # change the type back to uint32
        security_df["SK_CompanyID"] = security_df["SK_CompanyID"].astype("uint32")
        # get effective date from posting date
        security_df["EffectiveDate"] = df_sec["PTS"].dt.strftime("%Y-%m-%d")
        # Identify new and existing records based on Symbol
        is_first_batch = i == 0
        if is_first_batch:
            new_records = security_df
            existing_records = pd.DataFrame(
                {col: pd.Series(dtype=typ) for col, typ in dtypes.items()}
            )
            next_sk_id = 0
        else:
            existing_symbol = pd.read_sql_query(
                "SELECT Symbol FROM dimsecurity WHERE IsCurrent = 1", engine
            )["Symbol"]
            new_records = security_df[~security_df["Symbol"].isin(existing_symbol)]
            existing_records = security_df[security_df["Symbol"].isin(existing_symbol)]
            next_sk_id_query = "SELECT MAX(SK_SecurityID) FROM dimsecurity"
            next_sk_id = pd.read_sql_query(next_sk_id_query, engine).iloc[0, 0] or 0
        # update SK_SecurityID, IsCurrent, EndDate
        new_records.loc[:, "SK_SecurityID"] = range(
            next_sk_id + 1, next_sk_id + 1 + len(new_records)
        )
        new_records.loc[:, "IsCurrent"] = True
        new_records.loc[:, "EndDate"] = pd.Timestamp("9999-12-31")
        # Insert records with new SK_CompanyID
        new_records.to_sql(
            "dimsecurity", engine, if_exists="append", index=False, dtype=sql_dtypes
        )
        next_sk_id = new_records["SK_SecurityID"].max()

        # Process existing records
        for _, row in existing_records.iterrows():
            effective_date = row["EffectiveDate"]
            symbol = row["Symbol"]
            # Expire the current record in MySQL
            update_query = f"""UPDATE dimsecurity 
            SET IsCurrent = 0, EndDate = '{effective_date}' 
            WHERE Symbol = '{symbol}' AND IsCurrent = 1
            """
            with engine.connect() as conn:
                conn.execute(text(update_query))
                conn.commit()
            row["SK_SecurityID"] = next_sk_id + 1
            row["IsCurrent"] = True
            row["EndDate"] = pd.Timestamp("9999-12-31")
            row_df = pd.DataFrame(row).T
            # insert records with existing SK_CompanyID
            row_df.to_sql(
                "dimsecurity", engine, if_exists="append", index=False, dtype=sql_dtypes
            )
            next_sk_id += 1

In [46]:
create_table = """CREATE TABLE DimSecurity (
    SK_SecurityID INT UNSIGNED NOT NULL,
    Symbol CHAR(15) NOT NULL,
    Issue CHAR(6) NOT NULL,
    Status CHAR(10) NOT NULL,
    Name CHAR(70) NOT NULL,
    ExchangeID CHAR(6) NOT NULL,
    SK_CompanyID INT UNSIGNED NOT NULL,
    SharesOutstanding BIGINT UNSIGNED NOT NULL,
    FirstTrade DATE NOT NULL,
    FirstTradeOnExchange DATE NOT NULL,
    Dividend DECIMAL(10, 2) NOT NULL,
    IsCurrent BOOLEAN NOT NULL,
    BatchID SMALLINT UNSIGNED NOT NULL,
    EffectiveDate DATE NOT NULL,
    EndDate DATE NOT NULL,
    PRIMARY KEY (SK_SecurityID)
);"""
with engine.connect() as conn:
    conn.execute(text(create_table))

In [47]:
load_dimsecurity()

  0%|          | 0/203 [00:00<?, ?it/s]

### Prospect

In [ ]:
prospect_start_time = datetime.now()

In [48]:
def read_prospect_file(filepath):
    # Define the column names and their data types
    columns = [
        'AgencyID', 'LastName', 'FirstName', 'MiddleInitial', 'Gender', 
        'AddressLine1', 'AddressLine2', 'PostalCode', 'City', 'State', 
        'Country', 'Phone', 'Income', 'NumberCars', 'NumberChildren', 
        'MaritalStatus', 'Age', 'CreditRating', 'OwnOrRentFlag', 
        'Employer', 'NumberCreditCards', 'NetWorth'
    ]

    # Define the data types for reading the file
    dtypes = {
        'AgencyID': 'str', 'LastName': 'str', 'FirstName': 'str', 
        'MiddleInitial': 'str', 'Gender': 'str', 'AddressLine1': 'str', 
        'AddressLine2': 'str', 'PostalCode': 'str', 'City': 'str', 
        'State': 'str', 'Country': 'str', 'Phone': 'str', 
        'Income': 'Int64', 'NumberCars': 'Int8', 'NumberChildren': 'Int8', 
        'MaritalStatus': 'str', 'Age': 'Int8', 'CreditRating': 'Int16', 
        'OwnOrRentFlag': 'str', 'Employer': 'str', 
        'NumberCreditCards': 'Int8', 'NetWorth': 'Int64'
    }

    # Read the CSV file
    raw_prospect_df = pd.read_csv(
        filepath, 
        header=None, 
        names=columns, 
        dtype=dtypes
    )

    return raw_prospect_df

In [49]:
raw_prospect_df = read_prospect_file(DATA_DIR + "Prospect.csv")
raw_prospect_df.info()
raw_prospect_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24970 entries, 0 to 24969
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   AgencyID           24970 non-null  object
 1   LastName           24970 non-null  object
 2   FirstName          24969 non-null  object
 3   MiddleInitial      10982 non-null  object
 4   Gender             11880 non-null  object
 5   AddressLine1       24097 non-null  object
 6   AddressLine2       17160 non-null  object
 7   PostalCode         24098 non-null  object
 8   City               24970 non-null  object
 9   State              24970 non-null  object
 10  Country            23929 non-null  object
 11  Phone              23674 non-null  object
 12  Income             23760 non-null  Int64 
 13  NumberCars         23708 non-null  Int8  
 14  NumberChildren     23741 non-null  Int8  
 15  MaritalStatus      23721 non-null  object
 16  Age                23685 non-null  Int8 

,AgencyID,LastName,FirstName,MiddleInitial,Gender,AddressLine1,AddressLine2,PostalCode,City,State,...,Income,NumberCars,NumberChildren,MaritalStatus,Age,CreditRating,OwnOrRentFlag,Employer,NumberCreditCards,NetWorth
0,CLY0,CLYSDALE,Leif,NaN,m,11538 kimberley boulevard,NaN,97217,Chicago,FL,...,368776,<NA>,3,W,20,760,O,Brink's,<NA>,1058868
1,NAN1,NANAMIYA,CHARANGIT,NaN,f,13347 norwalk road,NaN,89506,Miami,NH,...,177967,5,1,U,3,555,U,NaN,6,1988185
2,pea2,pearson,pier,NaN,F,2134 neff south,NaN,35207,Des Moines,NY,...,321772,2,1,S,<NA>,566,O,NaN,6,3673128
3,PIL3,PILOTE,Hazel,NaN,NaN,23923 potomac road,NaN,t6d 1h9,Fayetteville,MO,...,25449,2,1,W,77,<NA>,O,Air Products & Chemicals,3,2005895
4,DEN4,DENETTE,kattie,NaN,F,21376 kirby lower,NaN,63172,Greensboro,NC,...,166567,0,3,M,21,815,O,Oshkosh,4,624736


In [50]:
not_null_cols = ['LastName', 'FirstName', 'City', 'State']
for col in not_null_cols:
    raw_prospect_df.loc[raw_prospect_df[col].isna(), col] = ''
raw_prospect_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24970 entries, 0 to 24969
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   AgencyID           24970 non-null  object
 1   LastName           24970 non-null  object
 2   FirstName          24970 non-null  object
 3   MiddleInitial      10982 non-null  object
 4   Gender             11880 non-null  object
 5   AddressLine1       24097 non-null  object
 6   AddressLine2       17160 non-null  object
 7   PostalCode         24098 non-null  object
 8   City               24970 non-null  object
 9   State              24970 non-null  object
 10  Country            23929 non-null  object
 11  Phone              23674 non-null  object
 12  Income             23760 non-null  Int64 
 13  NumberCars         23708 non-null  Int8  
 14  NumberChildren     23741 non-null  Int8  
 15  MaritalStatus      23721 non-null  object
 16  Age                23685 non-null  Int8 

In [51]:
dtypes = {
    'AgencyID': 'str',
    'SK_RecordDateID': 'uint32',
    'SK_UpdateDateID': 'uint32',
    'BatchID': 'uint16',
    'IsCustomer': 'boolean',
    'LastName': 'str',
    'FirstName': 'str',
    'MiddleInitial': 'str',
    'Gender': 'str',
    'AddressLine1': 'str',
    'AddressLine2': 'str',
    'PostalCode': 'str',
    'City': 'str',
    'State': 'str',
    'Country': 'str',
    'Phone': 'str',
    'Income': 'uint32',
    'NumberCars': 'uint8',
    'NumberChildren': 'uint8',
    'MaritalStatus': 'str',
    'Age': 'uint8',
    'CreditRating': 'uint16',
    'OwnOrRentFlag': 'str',
    'Employer': 'str',
    'NumberCreditCards': 'uint8',
    'NetWorth': 'int64',
    'MarketingNameplate': 'str'
}

# Create an empty DataFrame with the specified schema
prospect_df = pd.DataFrame({col: pd.Series(dtype=typ) for col, typ in dtypes.items()})

In [52]:
prospect_df["AgencyID"] = raw_prospect_df["AgencyID"]
prospect_df["LastName"] = raw_prospect_df["LastName"]
prospect_df["FirstName"] = raw_prospect_df["FirstName"]
prospect_df["MiddleInitial"] = raw_prospect_df["MiddleInitial"]
prospect_df["Gender"] = raw_prospect_df["Gender"]
# fix data quality issues
prospect_df['Gender'] = prospect_df['Gender'].str.upper()
mask = ~prospect_df['Gender'].isin(["M", "F"])
prospect_df.loc[mask, "Gender"] = "U"
prospect_df["AddressLine1"] = raw_prospect_df["AddressLine1"]
prospect_df["AddressLine2"] = raw_prospect_df["AddressLine2"]
prospect_df["PostalCode"] = raw_prospect_df["PostalCode"]
prospect_df["City"] = raw_prospect_df["City"]
prospect_df["State"] = raw_prospect_df["State"]
prospect_df["Country"] = raw_prospect_df["Country"]
prospect_df["Phone"] = raw_prospect_df["Phone"]
prospect_df["Income"] = raw_prospect_df["Income"]
prospect_df["NumberCars"] = raw_prospect_df["NumberCars"]
prospect_df["NumberChildren"] = raw_prospect_df["NumberChildren"]
prospect_df["MaritalStatus"] = raw_prospect_df["MaritalStatus"]
prospect_df["MaritalStatus"] = prospect_df["MaritalStatus"].str.upper()
mask = ~prospect_df["MaritalStatus"].isin(["S", "M", "D", "W"])
prospect_df.loc[mask, "MaritalStatus"] = "U"
prospect_df["Age"] = raw_prospect_df["Age"]
prospect_df["CreditRating"] = raw_prospect_df["CreditRating"]
prospect_df["OwnOrRentFlag"] = raw_prospect_df["OwnOrRentFlag"]
prospect_df["OwnOrRentFlag"] = prospect_df["OwnOrRentFlag"].str.upper()
mask = ~prospect_df["OwnOrRentFlag"].isin(["O", "R"])
prospect_df.loc[mask, "OwnOrRentFlag"] = "U"
prospect_df["Employer"] = raw_prospect_df["Employer"]
prospect_df["NumberCreditCards"] = raw_prospect_df["NumberCreditCards"]
prospect_df["NetWorth"] = raw_prospect_df["NetWorth"]

In [53]:
sk_dateid = pd.read_sql_query(f"select SK_DateID from dimdate where DateValue = '{BATCH_DATE}'", engine).iloc[0, 0]
# SK_RecordDateID is set to the DimDate SK_DateID field that corresponds to the Batch Date.
prospect_df['SK_RecordDateID'] = sk_dateid
# SK_UpdateDateID is set to the DimDate SK_DateID field that corresponds to the Batch Date
prospect_df['SK_UpdateDateID'] = sk_dateid

In [54]:
# Define conditions for each tag with null checks
conditions = {
    "HighValue": (prospect_df["NetWorth"].notnull() & prospect_df["Income"].notnull())
    & ((prospect_df["NetWorth"] > 1_000_000) | (prospect_df["Income"] > 200_000)),
    "Expenses": (
        prospect_df["NumberChildren"].notnull()
        & prospect_df["NumberCreditCards"].notnull()
    )
    & ((prospect_df["NumberChildren"] > 3) | (prospect_df["NumberCreditCards"] > 5)),
    "Boomer": prospect_df["Age"].notnull() & (prospect_df["Age"] > 45),
    "MoneyAlert": (
        prospect_df["Income"].notnull()
        & prospect_df["CreditRating"].notnull()
        & prospect_df["NetWorth"].notnull()
    )
    & (
        (prospect_df["Income"] < 50_000)
        | (prospect_df["CreditRating"] < 600)
        | (prospect_df["NetWorth"] < 100_000)
    ),
    "Spender": (
        prospect_df["NumberCars"].notnull() & prospect_df["NumberCreditCards"].notnull()
    )
    & ((prospect_df["NumberCars"] > 3) | (prospect_df["NumberCreditCards"] > 7)),
    "Inherited": (prospect_df["Age"].notnull() & prospect_df["NetWorth"].notnull())
    & ((prospect_df["Age"] < 25) & (prospect_df["NetWorth"] > 1_000_000)),
}

# Apply conditions to assign tags
prospect_df["MarketingNameplate"] = ""
for tag, condition in conditions.items():
    prospect_df["MarketingNameplate"] += np.where(condition, tag + "+", "")

# Remove trailing '+' and replace empty strings with None
prospect_df["MarketingNameplate"] = (
    prospect_df["MarketingNameplate"].str.rstrip("+").replace("", None)
)

In [55]:
prospect_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24970 entries, 0 to 24969
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   AgencyID            24970 non-null  object 
 1   SK_RecordDateID     24970 non-null  int64  
 2   SK_UpdateDateID     24970 non-null  int64  
 3   BatchID             0 non-null      float64
 4   IsCustomer          0 non-null      boolean
 5   LastName            24970 non-null  object 
 6   FirstName           24970 non-null  object 
 7   MiddleInitial       10982 non-null  object 
 8   Gender              24970 non-null  object 
 9   AddressLine1        24097 non-null  object 
 10  AddressLine2        17160 non-null  object 
 11  PostalCode          24098 non-null  object 
 12  City                24970 non-null  object 
 13  State               24970 non-null  object 
 14  Country             23929 non-null  object 
 15  Phone               23674 non-null  object 
 16  Inco

IsCurrent and BatchID are set after processing dimCustomer.

### dimCustomer

In [56]:
data_file = DATA_DIR + "CustomerMgmt.xml"
tree = etree.parse(data_file)
namespace = {'tpcdi': 'http://www.tpc.org/tpc-di'}

In [57]:
dtypes = {
    'SK_CustomerID': 'int32',
    'CustomerID': 'int32',
    'TaxID': 'str',
    'Status': 'str',
    'LastName': 'str',
    'FirstName': 'str',
    'MiddleInitial': 'str',
    'Gender': 'str',
    'Tier': 'UInt8',
    'DOB': 'datetime64[ns]',
    'AddressLine1': 'str',
    'AddressLine2': 'str',
    'PostalCode': 'str',
    'City': 'str',
    'StateProv': 'str',
    'Country': 'str',
    'Phone1': 'str',
    'Phone2': 'str',
    'Phone3': 'str',
    'Email1': 'str',
    'Email2': 'str',
    'NationalTaxRateDesc': 'str',
    'NationalTaxRate': 'Float64',
    'LocalTaxRateDesc': 'str',
    'LocalTaxRate': 'Float64',
    'AgencyID': 'str',
    'CreditRating': 'UInt16',
    'NetWorth': 'Float64',
    'MarketingNameplate': 'str',
    'IsCurrent': 'boolean',
    'BatchID': 'uint8',
    'EffectiveDate': 'datetime64[ns]',
    'EndDate': 'datetime64[ns]'
}


In [58]:
def format_phone_number(phone_element):
    # Extract components of the phone number
    ctry_code = phone_element.findtext('C_CTRY_CODE', default=None, namespaces=namespace)
    area_code = phone_element.findtext('C_AREA_CODE', default=None, namespaces=namespace)
    local = phone_element.findtext('C_LOCAL', default=None, namespaces=namespace)
    ext = phone_element.findtext('C_EXT', default=None, namespaces=namespace)

    # Apply transformation rules
    if ctry_code and area_code and local:
        phone = f"+{ctry_code} ({area_code}) {local}"
    elif area_code and local:
        phone = f"({area_code}) {local}"
    elif local:
        phone = local
    else:
        return None

    # Add extension if present
    if ext:
        phone += ext

    return phone

In [59]:
def get_all_tax_info(tax_ids):
    tax_ids_str = "','".join(tax_ids)
    query = f"SELECT TX_ID, TX_NAME, TX_RATE FROM taxrate WHERE TX_ID IN ('{tax_ids_str}')"
    result = pd.read_sql_query(query, engine)
    return result.set_index('TX_ID').to_dict('index')

In [60]:
# Initialize a dictionary to store the latest index of 'UPDCUST' or 'INACT' for each CustomerID
latest_updates = {}

# Get all actions
all_actions = tree.xpath(".//tpcdi:Action", namespaces=namespace)
# NEW actions
new_actions = [action for action in all_actions if action.get('ActionType') == 'NEW']
# UPD actions
upd_actions = [action for action in all_actions if action.get('ActionType') == 'UPDCUST']
# INACT actions
inact_actions = [action for action in all_actions if action.get('ActionType') == 'INACT']

# Preprocess to fill the dictionary
for i, action in enumerate(all_actions):
    if action.get('ActionType') in ['UPDCUST', 'INACT']:
        customer = action.find('Customer', namespaces=namespace)
        customer_id = customer.get('C_ID', None)
        if customer_id:
            latest_updates[int(customer_id)] = i

# Modified has_later_update function
def has_later_update(customer_id, current_index):
    """Check for subsequent 'UPDCUST' or 'INACT' actions for a given CustomerID at current_index"""
    return latest_updates.get(customer_id, -1) > current_index

In [61]:
# Create an empty DataFrame with the specified schema
dimCustomer_df = pd.DataFrame(
    {col: pd.Series(dtype=typ) for col, typ in dtypes.items()}
)

# Initialize lists to store tax IDs for each record
national_tax_ids = []
local_tax_ids = []

# temporary prospect_df for matching
prospect_df_temp = prospect_df[
    [
        "AgencyID",
        "CreditRating",
        "NetWorth",
        "MarketingNameplate",
        "LastName",
        "FirstName",
        "AddressLine1",
        "AddressLine2",
        "PostalCode",
    ]
].copy()
prospect_df_temp["LastName"] = prospect_df_temp["LastName"].str.upper()
prospect_df_temp["FirstName"] = prospect_df_temp["FirstName"].str.upper()
prospect_df_temp["AddressLine1"] = prospect_df_temp["AddressLine1"].str.upper()
prospect_df_temp["AddressLine2"] = prospect_df_temp["AddressLine2"].str.upper()
prospect_df_temp["PostalCode"] = prospect_df_temp["PostalCode"].str.upper()

In [62]:
# Initialize lists to store data for NEW actions
data = {
    "CustomerID": [],
    "TaxID": [],
    "LastName": [],
    "FirstName": [],
    "MiddleInitial": [],
    "Tier": [],
    "DOB": [],
    "Gender": [],
    "Email1": [],
    "Email2": [],
    "AddressLine1": [],
    "AddressLine2": [],
    "PostalCode": [],
    "City": [],
    "StateProv": [],
    "Country": [],
    "Phone1": [],
    "Phone2": [],
    "Phone3": [],
    "NationalTaxRateDesc": [],
    "NationalTaxRate": [],
    "LocalTaxRateDesc": [],
    "LocalTaxRate": [],
    "AgencyID": [],
    "CreditRating": [],
    "NetWorth": [],
    "MarketingNameplate": [],
    "EffectiveDate": [],
}

# Iterate through each 'Action' element with ActionType="NEW"
for index, action in enumerate(tqdm(new_actions)):
    customer = action.find("Customer", namespaces=namespace)
    name = customer.find("Name", namespaces=namespace)
    contact_info = customer.find("ContactInfo", namespaces=namespace)
    address = customer.find("Address", namespaces=namespace)
    tax_info = customer.find("TaxInfo", namespaces=namespace)

    customer_id = customer.get("C_ID", None)
    customer_id = int(customer_id) if customer_id else None
    data["CustomerID"].append(customer_id)
    data["TaxID"].append(customer.get("C_TAX_ID", None))
    tier = customer.get("C_TIER", None)
    tier = int(tier) if tier else None
    if tier is not None and tier not in (1,2,3):
        """
        A record will be inserted in the DImessages table if a customer's Tier is not one of the valid
        values (1,2,3). The MessageSource is “DimCustomer”, the MessageType is “Alert” and the
        MessageText is “Invalid customer tier”. The MessageData field is “C_ID = ” followed by the
        natural key value of the record, then “, C_TIER = ” and the C_TIER value.
        """
        MessageDateAndTime = pd.Timestamp("now")
        sk_customer_id = len(data["CustomerID"])
        message = f"C_ID = {sk_customer_id}, C_TIER = {tier}"
        message_source = "DimCustomer"
        message_type = "Alert"
        message_text = "Invalid customer tier"
        query = f"""INSERT INTO dimessages (MessageDateAndTime, BatchID, MessageSource, MessageText, MessageType, MessageData)
        VALUES ('{MessageDateAndTime}', {BATCH_ID}, '{message_source}', '{message_text}', '{message_type}', '{message}')"""
        with engine.connect() as conn:
            conn.execute(text(query))
            conn.commit()

    data["Tier"].append(tier)
    dob = customer.get("C_DOB", None)
    dob = pd.to_datetime(dob, format="%Y-%m-%d") if dob else None
    """A record will be reported in the DImessages table if a customer's DOB is invalid. A customer's
    DOB is invalid if DOB < Batch Date - 100 years or DOB > Batch Date (customer is over 100
    years old or born in the future). The MessageSource is “DimCustomer”, the MessageType is
    “Alert” and the MessageText is “DOB out of range”. The MessageData field is “C_ID = ”
    followed by the natural key value of the record, then “, C_DOB = ” and the C_DOB value."""
    if dob and (dob < BATCH_DATE - pd.Timedelta(days=100*365) or dob > BATCH_DATE):
        MessageDateAndTime = pd.Timestamp("now")
        batch_id = 1
        sk_customer_id = len(data["CustomerID"])
        message = f"C_ID = {sk_customer_id}, C_DOB = {dob}"
        message_source = "DimCustomer"
        message_type = "Alert"
        message_text = "DOB out of range"
        query = f"""INSERT INTO dimessages (MessageDateAndTime, BatchID, MessageSource, MessageText, MessageType, MessageData)
        VALUES ('{MessageDateAndTime}', {batch_id}, '{message_source}', '{message_text}', '{message_type}', '{message}')"""
        with engine.connect() as conn:
            conn.execute(text(query))
            conn.commit()
    data["DOB"].append(dob)
    gender = customer.get("C_GNDR", "U")
    if gender is not None:
        gender = gender.upper()
    gender = "U" if gender not in ("M", "F") else gender
    data["Gender"].append(gender)
    
    first_name = name.findtext("C_F_NAME", default=None, namespaces=namespace)
    data["FirstName"].append(first_name if first_name else None)
    middle_initial = name.findtext("C_M_NAME", default=None, namespaces=namespace)
    data["MiddleInitial"].append(middle_initial if middle_initial else None)
    last_name = name.findtext("C_L_NAME", default=None, namespaces=namespace)
    data["LastName"].append(last_name if last_name else None)

    
    prim_email = contact_info.findtext(
        "C_PRIM_EMAIL", default=None, namespaces=namespace
    )
    data["Email1"].append(prim_email if prim_email else None)
    alt_email = contact_info.findtext(
        "C_ALT_EMAIL", default=None, namespaces=namespace
    )
    data["Email2"].append(alt_email if alt_email else None)
    data["Phone1"].append(
        format_phone_number(contact_info.find("C_PHONE_1", namespaces=namespace))
    )
    data["Phone2"].append(
        format_phone_number(contact_info.find("C_PHONE_2", namespaces=namespace))
    )
    data["Phone3"].append(
        format_phone_number(contact_info.find("C_PHONE_3", namespaces=namespace))
    )

    # Extracting address information
    address_line1 = address.findtext(
        "C_ADLINE1", default=None, namespaces=namespace
    )
    data["AddressLine1"].append(address_line1 if address_line1 else None)
    address_line2 = address.findtext(
        "C_ADLINE2", default=None, namespaces=namespace
    )
    data["AddressLine2"].append(address_line2 if address_line2 else None)
    postalcode = address.findtext("C_ZIPCODE", default=None, namespaces=namespace)
    data["PostalCode"].append(postalcode if postalcode else None)
    city = address.findtext("C_CITY", default=None, namespaces=namespace)
    data["City"].append(city if city else None)
    state_prov = address.findtext(
        "C_STATE_PROV", default=None, namespaces=namespace
    )
    data["StateProv"].append(state_prov if state_prov else None)
    country = address.findtext("C_CTRY", default=None, namespaces=namespace)
    data["Country"].append(country if country else None)
    
    # Store TX_ID as placeholders
    national_tax_id = tax_info.findtext(
        "C_NAT_TX_ID", default=None, namespaces=namespace
    )
    national_tax_id = national_tax_id if national_tax_id else None
    national_tax_ids.append(national_tax_id)
    local_tax_id = tax_info.findtext(
        "C_LCL_TX_ID", default=None, namespaces=namespace
    )
    local_tax_id = local_tax_id if local_tax_id else None
    local_tax_ids.append(local_tax_id)

    if not has_later_update(customer_id, index):
        # Find matching prospect record
        match = prospect_df_temp[
            (prospect_df_temp["LastName"] == last_name.upper())
            & (prospect_df_temp["FirstName"] == first_name.upper())
            & (prospect_df_temp["AddressLine1"] == address_line1.upper())
            & (prospect_df_temp["AddressLine2"] == address_line2.upper())
            & (prospect_df_temp["PostalCode"] == postalcode.upper())
        ]
        if not match.empty:
            # Set values from the matching prospect record
            data["AgencyID"].append(match["AgencyID"].iloc[-1])
            data["CreditRating"].append(match["CreditRating"].iloc[-1])
            data["NetWorth"].append(match["NetWorth"].iloc[-1])
            data["MarketingNameplate"].append(match["MarketingNameplate"].iloc[-1])
        else:
            # Set values to NULL
            data["AgencyID"].append(None)
            data["CreditRating"].append(None)
            data["NetWorth"].append(None)
            data["MarketingNameplate"].append(None)
    else:
        # Set values to NULL due to later 'UPDCUST' or 'INACT'
        data["AgencyID"].append(None)
        data["CreditRating"].append(None)
        data["NetWorth"].append(None)
        data["MarketingNameplate"].append(None)
    # history tracking
    data["EffectiveDate"].append(pd.to_datetime(action.get("ActionTS"), format="%Y-%m-%dT%H:%M:%S"))

# Get unique TX_IDs and remove None values 
unique_tax_ids = set(national_tax_ids + local_tax_ids) - {None}
all_tax_info = get_all_tax_info(unique_tax_ids)

# map each tax ID to its description and rate
for i in range(len(national_tax_ids)):
    national_info = all_tax_info.get(
        national_tax_ids[i], {"TX_NAME": None, "TX_RATE": None}
    )
    data["NationalTaxRateDesc"].append(national_info["TX_NAME"])
    data["NationalTaxRate"].append(national_info["TX_RATE"])

    local_info = all_tax_info.get(local_tax_ids[i], {"TX_NAME": None, "TX_RATE": None})
    data["LocalTaxRateDesc"].append(local_info["TX_NAME"])
    data["LocalTaxRate"].append(local_info["TX_RATE"])

# Creating DataFrame
dimCustomer_df = pd.concat([dimCustomer_df, pd.DataFrame(data)])
dimCustomer_df["Status"] = "ACTIVE"

  0%|          | 0/7800 [00:00<?, ?it/s]

C:\Users\strai\AppData\Local\Temp\ipykernel_15716\4200087088.py:199: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dimCustomer_df = pd.concat([dimCustomer_df, pd.DataFrame(data)])


In [63]:
def df2dict(df, exclude_columns):
    # Remove the specified columns from the DataFrame
    df_filtered = df.drop(columns=exclude_columns)
    # Convert the filtered DataFrame to a dictionary
    df_dict = df_filtered.to_dict(orient='index')
    # Create a new dictionary that maps CustomerID to a dictionary of column values
    customer_dict = {row['CustomerID']: {col: val for col, val in row.items() if col != 'CustomerID'} for _, row in df_dict.items()}
    return customer_dict

In [64]:
exclude_columns = ['SK_CustomerID', 'IsCurrent', 'BatchID', 'EffectiveDate', 'EndDate', 'Status']
# dictionary to track latest values for each customer
customer_data = df2dict(dimCustomer_df, exclude_columns)

In [65]:
# Initialize lists to store data for NEW actions
data = {col: [] for col in data.keys()}

# Iterate through each 'Action' element with ActionType="UPDCUST"
for index, action in enumerate(upd_actions):
    customer = action.find("Customer", namespaces=namespace)
    name = customer.find("Name", namespaces=namespace)
    contact_info = customer.find("ContactInfo", namespaces=namespace)
    address = customer.find("Address", namespaces=namespace)
    tax_info = customer.find("TaxInfo", namespaces=namespace)

    customer_id = int(customer.get("C_ID", None))
    data["CustomerID"].append(customer_id)

    # Update tax_id
    tax_id = customer.get("C_TAX_ID", None)
    if tax_id is None:
        tax_id = customer_data[customer_id]["TaxID"]
    else:
        customer_data[customer_id]["TaxID"] = tax_id
    data["TaxID"].append(tax_id)
    # Update tier
    tier = customer.get("C_TIER", None)
    tier = int(tier) if tier else None
    if tier is None:
        tier = customer_data[customer_id]["Tier"]
    else:
        customer_data[customer_id]["Tier"] = tier
        if tier is not None and tier not in (1,2,3):
            """
            A record will be inserted in the DImessages table if a customer's Tier is not one of the valid
            values (1,2,3). The MessageSource is “DimCustomer”, the MessageType is “Alert” and the
            MessageText is “Invalid customer tier”. The MessageData field is “C_ID = ” followed by the
            natural key value of the record, then “, C_TIER = ” and the C_TIER value.
            """
            MessageDateAndTime = pd.Timestamp("now")
            batch_id = 1
            sk_customer_id = len(data["CustomerID"]) + dimCustomer_df.shape[0]
            message = f"C_ID = {sk_customer_id}, C_TIER = {tier}"
            message_source = "DimCustomer"
            message_type = "Alert"
            message_text = "Invalid customer tier"
            query = f"""INSERT INTO dimessages (MessageDateAndTime, BatchID, MessageSource, MessageText, MessageType, MessageData)
            VALUES ('{MessageDateAndTime}', {batch_id}, '{message_source}', '{message_text}', '{message_type}', '{message}')"""
            with engine.connect() as conn:
                conn.execute(text(query))
                conn.commit()
    data["Tier"].append(tier)
    # Update DOB
    dob = customer.get("C_DOB", None)
    dob = pd.to_datetime(dob, format="%Y-%m-%d") if dob else None
    if dob is None:
        dob = customer_data[customer_id]["DOB"]
    else:
        customer_data[customer_id]["DOB"] = dob
        """A record will be reported in the DImessages table if a customer's DOB is invalid. A customer's
        DOB is invalid if DOB < Batch Date - 100 years or DOB > Batch Date (customer is over 100
        years old or born in the future). The MessageSource is “DimCustomer”, the MessageType is
        “Alert” and the MessageText is “DOB out of range”. The MessageData field is “C_ID = ”
        followed by the natural key value of the record, then “, C_DOB = ” and the C_DOB value."""
        if dob and (dob < BATCH_DATE - pd.Timedelta(days=100*365) or dob > BATCH_DATE):
            MessageDateAndTime = pd.Timestamp("now")
            batch_id = 1
            sk_customer_id = len(data["CustomerID"])
            message = f"C_ID = {sk_customer_id}, C_DOB = {dob}"
            message_source = "DimCustomer"
            message_type = "Alert"
            message_text = "DOB out of range"
            query = f"""INSERT INTO dimessages (MessageDateAndTime, BatchID, MessageSource, MessageText, MessageType, MessageData)
            VALUES ('{MessageDateAndTime}', {batch_id}, '{message_source}', '{message_text}', '{message_type}', '{message}')"""
            with engine.connect() as conn:
                conn.execute(text(query))
                conn.commit()
    data["DOB"].append(dob)
    # Update gender
    gender = customer.get("C_GNDR", None)
    if gender is None:
        gender = customer_data[customer_id]["Gender"]
    else:
        gender = gender.upper()
        gender = "U" if gender not in ("M", "F") else gender
        customer_data[customer_id]["Gender"] = gender
    data["Gender"].append(gender)

    # Update first name
    if name is None:
        first_name = customer_data[customer_id]["FirstName"]
        data["FirstName"].append(first_name)
        middle_initial = customer_data[customer_id]["MiddleInitial"]
        data["MiddleInitial"].append(middle_initial)
        last_name = customer_data[customer_id]["LastName"]
        data["LastName"].append(last_name)
    else:
        first_name = name.findtext("C_F_NAME", default=None, namespaces=namespace)
        if first_name is None:
            first_name = customer_data[customer_id]["FirstName"]
        else:
            customer_data[customer_id]["FirstName"] = first_name
        data["FirstName"].append(first_name)
        # Update middle initial
        middle_initial = name.findtext("C_M_NAME", default=None, namespaces=namespace)
        if middle_initial is None:
            middle_initial = customer_data[customer_id]["MiddleInitial"]
        else:
            customer_data[customer_id]["MiddleInitial"] = middle_initial
        data["MiddleInitial"].append(middle_initial)
        # Update last name
        last_name = name.findtext("C_L_NAME", default=None, namespaces=namespace)
        if last_name is None:
            last_name = customer_data[customer_id]["LastName"]
        else:
            customer_data[customer_id]["LastName"] = last_name
        data["LastName"].append(last_name if last_name else None)

    if contact_info is None:
        prim_email = customer_data[customer_id]["Email1"]
        data["Email1"].append(prim_email)
        alt_email = customer_data[customer_id]["Email2"]
        data["Email2"].append(alt_email)
        phone1 = customer_data[customer_id]["Phone1"]
        data["Phone1"].append(phone1)
        phone2 = customer_data[customer_id]["Phone2"]
        data["Phone2"].append(phone2)
        phone3 = customer_data[customer_id]["Phone3"]
        data["Phone3"].append(phone3)
    else:
        # update primary email
        prim_email = contact_info.findtext(
            "C_PRIM_EMAIL", default=None, namespaces=namespace
        )
        if prim_email is None:
            prim_email = customer_data[customer_id]["Email1"]
        else:
            customer_data[customer_id]["Email1"] = prim_email
        data["Email1"].append(prim_email if prim_email else None)
        # update alternate email
        alt_email = contact_info.findtext(
            "C_ALT_EMAIL", default=None, namespaces=namespace
        )
        if alt_email is None:
            alt_email = customer_data[customer_id]["Email2"]
        else:
            customer_data[customer_id]["Email2"] = alt_email
        data["Email2"].append(alt_email if alt_email else None)
        # update phone numbers
        phone1 = contact_info.find("C_PHONE_1", namespaces=namespace)
        if phone1 is None:
            phone1 = customer_data[customer_id]["Phone1"]
        else:
            phone1 = format_phone_number(phone1)
            customer_data[customer_id]["Phone1"] = phone1
        data["Phone1"].append(phone1)
        phone2 = contact_info.find("C_PHONE_2", namespaces=namespace)
        if phone2 is None:
            phone2 = customer_data[customer_id]["Phone2"]
        else:
            phone2 = format_phone_number(phone2)
            customer_data[customer_id]["Phone2"] = phone2
        data["Phone2"].append(phone2)
        phone3 = contact_info.find("C_PHONE_3", namespaces=namespace)
        if phone3 is None:
            phone3 = customer_data[customer_id]["Phone3"]
        else:
            phone3 = format_phone_number(phone3)
            customer_data[customer_id]["Phone3"] = phone3
        data["Phone3"].append(phone3)

    if address is None:
        address_line1 = customer_data[customer_id]["AddressLine1"]
        data["AddressLine1"].append(address_line1)
        address_line2 = customer_data[customer_id]["AddressLine2"]
        data["AddressLine2"].append(address_line2)
        postalcode = customer_data[customer_id]["PostalCode"]
        data["PostalCode"].append(postalcode)
        city = customer_data[customer_id]["City"]
        data["City"].append(city)
        state_prov = customer_data[customer_id]["StateProv"]
        data["StateProv"].append(state_prov)
        country = customer_data[customer_id]["Country"]
        data["Country"].append(country)
    else:
        # Extracting address information
        address_line1 = address.findtext(
            "C_ADLINE1", default=None, namespaces=namespace
        )
        if address_line1 is None:
            address_line1 = customer_data[customer_id]["AddressLine1"]
        else:
            customer_data[customer_id]["AddressLine1"] = address_line1
        data["AddressLine1"].append(address_line1 if address_line1 else None)
        address_line2 = address.findtext(
            "C_ADLINE2", default=None, namespaces=namespace
        )
        if address_line2 is None:
            address_line2 = customer_data[customer_id]["AddressLine2"]
        else:
            customer_data[customer_id]["AddressLine2"] = address_line2
        data["AddressLine2"].append(address_line2 if address_line2 else None)
        postalcode = address.findtext("C_ZIPCODE", default=None, namespaces=namespace)
        if postalcode is None:
            postalcode = customer_data[customer_id]["PostalCode"]
        else:
            customer_data[customer_id]["PostalCode"] = postalcode
        data["PostalCode"].append(postalcode if postalcode else None)
        city = address.findtext("C_CITY", default=None, namespaces=namespace)
        if city is None:
            city = customer_data[customer_id]["City"]
        else:
            customer_data[customer_id]["City"] = city
        data["City"].append(city if city else None)
        state_prov = address.findtext(
            "C_STATE_PROV", default=None, namespaces=namespace
        )
        if state_prov is None:
            state_prov = customer_data[customer_id]["StateProv"]
        else:
            customer_data[customer_id]["StateProv"] = state_prov
        data["StateProv"].append(state_prov if state_prov else None)
        country = address.findtext("C_CTRY", default=None, namespaces=namespace)
        if country is None:
            country = customer_data[customer_id]["Country"]
        else:
            customer_data[customer_id]["Country"] = country
        data["Country"].append(country if country else None)
    
    # Store TX_ID as placeholders
    if tax_info is None:
        national_tax_rate_desc = customer_data[customer_id]["NationalTaxRateDesc"]
        data["NationalTaxRateDesc"].append(national_tax_rate_desc)
        national_tax_rate = customer_data[customer_id]["NationalTaxRate"]
        data["NationalTaxRate"].append(national_tax_rate)
        local_tax_rate_desc = customer_data[customer_id]["LocalTaxRateDesc"]
        data["LocalTaxRateDesc"].append(local_tax_rate_desc)
        local_tax_rate = customer_data[customer_id]["LocalTaxRate"]
        data["LocalTaxRate"].append(local_tax_rate)
        national_tax_ids.append(None)
        local_tax_ids.append(None)        
    else:
        national_tax_id = tax_info.findtext(
            "C_NAT_TX_ID", default=None, namespaces=namespace
        )
        if national_tax_id is None:
            national_tax_rate_desc = customer_data[customer_id]["NationalTaxRateDesc"]
            data["NationalTaxRateDesc"].append(national_tax_rate_desc)
            national_tax_rate = customer_data[customer_id]["NationalTaxRate"]
            data["NationalTaxRate"].append(national_tax_rate)
            national_tax_ids.append(None)
        else:
            result = pd.read_sql(f"SELECT TX_NAME, TX_RATE FROM taxrate WHERE TX_ID = '{national_tax_id}'", engine)
            national_tax_rate_desc = result.iloc[0, 0]
            national_tax_rate = result.iloc[0, 1]
            customer_data[customer_id]["NationalTaxRateDesc"] = national_tax_rate_desc
            customer_data[customer_id]["NationalTaxRate"] = national_tax_rate
            data["NationalTaxRateDesc"].append(national_tax_rate_desc)
            data["NationalTaxRate"].append(national_tax_rate)
        local_tax_id = tax_info.findtext(
            "C_LCL_TX_ID", default=None, namespaces=namespace
        )
        if local_tax_id is None:
            local_tax_rate_desc = customer_data[customer_id]["LocalTaxRateDesc"]
            data["LocalTaxRateDesc"].append(local_tax_rate_desc)
            local_tax_rate = customer_data[customer_id]["LocalTaxRate"]
            data["LocalTaxRate"].append(local_tax_rate)
        else:
            result = pd.read_sql(f"SELECT TX_NAME, TX_RATE FROM taxrate WHERE TX_ID = '{local_tax_id}'", engine)
            local_tax_rate_desc = result.iloc[0, 0]
            local_tax_rate = result.iloc[0, 1]
            customer_data[customer_id]["LocalTaxRateDesc"] = local_tax_rate_desc
            customer_data[customer_id]["LocalTaxRate"] = local_tax_rate
            data["LocalTaxRateDesc"].append(local_tax_rate_desc)
            data["LocalTaxRate"].append(local_tax_rate)


    if not has_later_update(customer_id, index):
        # Find matching prospect record
        match = prospect_df[
            (prospect_df_temp["LastName"] == last_name.upper())
            & (prospect_df_temp["FirstName"] == first_name.upper())
            & (prospect_df_temp["AddressLine1"] == address_line1.upper())
            & (prospect_df_temp["AddressLine2"] == address_line2.upper())
            & (prospect_df_temp["PostalCode"] == postalcode.upper())
        ]
        if not match.empty:
            # Set values from the matching prospect record
            data["AgencyID"].append(match["AgencyID"].iloc[-1])
            data["CreditRating"].append(match["CreditRating"].iloc[-1])
            data["NetWorth"].append(match["NetWorth"].iloc[-1])
            data["MarketingNameplate"].append(match["MarketingNameplate"].iloc[-1])
        else:
            # Set values to those in customer_data
            data["AgencyID"].append(customer_data[customer_id]["AgencyID"])
            data["CreditRating"].append(customer_data[customer_id]["CreditRating"])
            data["NetWorth"].append(customer_data[customer_id]["NetWorth"])
            data["MarketingNameplate"].append(customer_data[customer_id]["MarketingNameplate"])
    else:
        # Set values to those in customer_data due to later 'UPDCUST' or 'INACT'
        data["AgencyID"].append(customer_data[customer_id]["AgencyID"])
        data["CreditRating"].append(customer_data[customer_id]["CreditRating"])
        data["NetWorth"].append(customer_data[customer_id]["NetWorth"])
        data["MarketingNameplate"].append(customer_data[customer_id]["MarketingNameplate"])
    # history tracking
    data["EffectiveDate"].append(pd.to_datetime(action.get("ActionTS"), format="%Y-%m-%dT%H:%M:%S"))

# Creating DataFrame
dimCustomer_df = pd.concat([dimCustomer_df, pd.DataFrame(data)])
dimCustomer_df["Status"] = "ACTIVE"

In [66]:
# Initialize lists to store data for NEW actions
data = {col: [] for col in data.keys()}

# Iterate through each 'Action' element with ActionType="INACT"
for index, action in enumerate(inact_actions):
    customer = action.find("Customer", namespaces=namespace)
    customer_id = int(customer.get("C_ID", None))
    data["CustomerID"].append(customer_id)
    # Copy all fields from customer_data
    for col in data.keys():
        if col in ("CustomerID", "EffectiveDate"):
            continue
        else:
            data[col].append(customer_data[customer_id][col])
    # history tracking
    data["EffectiveDate"].append(pd.to_datetime(action.get("ActionTS"), format="%Y-%m-%dT%H:%M:%S"))

# Creating DataFrame
data_df = pd.DataFrame(data)
data_df["Status"] = "INACTIVE"
dimCustomer_df = pd.concat([dimCustomer_df, data_df])
dimCustomer_df["BatchID"] = 1

In [67]:
dimCustomer_df['SK_CustomerID'] = range(1, len(dimCustomer_df) + 1)
dimCustomer_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10810 entries, 0 to 859
Data columns (total 33 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   SK_CustomerID        10810 non-null  int64         
 1   CustomerID           10810 non-null  int64         
 2   TaxID                10810 non-null  object        
 3   Status               10810 non-null  object        
 4   LastName             10810 non-null  object        
 5   FirstName            10810 non-null  object        
 6   MiddleInitial        4036 non-null   object        
 7   Gender               10810 non-null  object        
 8   Tier                 10382 non-null  float64       
 9   DOB                  10810 non-null  datetime64[ns]
 10  AddressLine1         10810 non-null  object        
 11  AddressLine2         1230 non-null   object        
 12  PostalCode           10810 non-null  object        
 13  City                 10810 non-null  o

In [68]:
# Sort the DataFrame by CustomerID and EffectiveDate
dimCustomer_df.sort_values(by=['CustomerID', 'EffectiveDate'], inplace=True)
# Create a shifted DataFrame
shifted_df = dimCustomer_df.shift(-1)
# Update EndDate: If next row has same CustomerID, use its EffectiveDate; otherwise, use default date
dimCustomer_df['EndDate'] = pd.Timestamp('9999-12-31')
mask = dimCustomer_df['CustomerID'] == shifted_df['CustomerID']
dimCustomer_df.loc[mask, 'EndDate'] = shifted_df.loc[mask, 'EffectiveDate']

# Update IsCurrent: True if next row has different CustomerID or is the last row
dimCustomer_df['IsCurrent'] = ~mask
dimCustomer_df.sort_values(by=['SK_CustomerID'], inplace=True)

In [69]:
sql_dtypes = {
    'SK_CustomerID': sqlalchemy.types.Integer,
    'CustomerID': sqlalchemy.types.Integer,
    'TaxID': sqlalchemy.types.String(20),
    'Status': sqlalchemy.types.String(10),
    'LastName': sqlalchemy.types.String(30),
    'FirstName': sqlalchemy.types.String(30),
    'MiddleInitial': sqlalchemy.types.String(1),
    'Gender': sqlalchemy.types.String(1),
    'Tier': sqlalchemy.types.SmallInteger,
    'DOB': sqlalchemy.types.Date,
    'AddressLine1': sqlalchemy.types.String(80),
    'AddressLine2': sqlalchemy.types.String(80),
    'PostalCode': sqlalchemy.types.String(12),
    'City': sqlalchemy.types.String(25),
    'StateProv': sqlalchemy.types.String(20),
    'Country': sqlalchemy.types.String(24),
    'Phone1': sqlalchemy.types.String(30),
    'Phone2': sqlalchemy.types.String(30),
    'Phone3': sqlalchemy.types.String(30),
    'Email1': sqlalchemy.types.String(50),
    'Email2': sqlalchemy.types.String(50),
    'NationalTaxRateDesc': sqlalchemy.types.String(50),
    'NationalTaxRate': sqlalchemy.types.Numeric(6, 5),
    'LocalTaxRateDesc': sqlalchemy.types.String(50),
    'LocalTaxRate': sqlalchemy.types.Numeric(6, 5),
    'AgencyID': sqlalchemy.types.String(30),
    'CreditRating': sqlalchemy.types.SmallInteger,
    'NetWorth': sqlalchemy.types.Numeric(10),
    'MarketingNameplate': sqlalchemy.types.String(100),
    'IsCurrent': sqlalchemy.types.Boolean,
    'BatchID': sqlalchemy.types.SmallInteger,
    'EffectiveDate': sqlalchemy.types.Date,
    'EndDate': sqlalchemy.types.Date
}

In [70]:
# cast to int 32
cols = ['SK_CustomerID', 'CustomerID', 'BatchID']
for col in cols:
    dimCustomer_df[col] = dimCustomer_df[col].astype('int')
dimCustomer_df['Tier'] = dimCustomer_df['Tier'].astype('UInt8')
dimCustomer_df['NationalTaxRate'] = dimCustomer_df['NationalTaxRate'].astype('float32')
dimCustomer_df['LocalTaxRate'] = dimCustomer_df['LocalTaxRate'].astype('float32')

In [71]:
# Convert DataFrame columns to the appropriate types
dimCustomer_df['SK_CustomerID'] = dimCustomer_df['SK_CustomerID'].astype(np.int32)
dimCustomer_df['CustomerID'] = dimCustomer_df['CustomerID'].astype(np.int32)
dimCustomer_df['Tier'] = dimCustomer_df['Tier'].astype(pd.Int8Dtype())
dimCustomer_df['NationalTaxRate'] = dimCustomer_df['NationalTaxRate'].astype(np.float64)
dimCustomer_df['LocalTaxRate'] = dimCustomer_df['LocalTaxRate'].astype(np.float64)
dimCustomer_df['CreditRating'] = dimCustomer_df['CreditRating'].astype(pd.Int16Dtype())
dimCustomer_df['NetWorth'] = dimCustomer_df['NetWorth'].astype(pd.Float64Dtype())
dimCustomer_df['BatchID'] = dimCustomer_df['BatchID'].astype(np.int16)

# Convert date columns to datetime.date
dimCustomer_df['DOB'] = pd.to_datetime(dimCustomer_df['DOB']).dt.date
dimCustomer_df['EffectiveDate'] = pd.to_datetime(dimCustomer_df['EffectiveDate']).dt.date
dimCustomer_df['EndDate'] = pd.to_datetime(dimCustomer_df['EndDate']).dt.date

In [72]:
create_table = """CREATE TABLE DimCustomer (
    SK_CustomerID INT UNSIGNED NOT NULL,
    CustomerID INT UNSIGNED NOT NULL,
    TaxID CHAR(20) NOT NULL,
    Status CHAR(10) NOT NULL,
    LastName CHAR(30) NOT NULL,
    FirstName CHAR(30) NOT NULL,
    MiddleInitial CHAR(1),
    Gender CHAR(1),
    Tier TINYINT UNSIGNED,
    DOB DATE NOT NULL,
    AddressLine1 CHAR(80) NOT NULL,
    AddressLine2 CHAR(80),
    PostalCode CHAR(12) NOT NULL,
    City CHAR(25) NOT NULL,
    StateProv CHAR(20) NOT NULL,
    Country CHAR(24),
    Phone1 CHAR(30),
    Phone2 CHAR(30),
    Phone3 CHAR(30),
    Email1 CHAR(50),
    Email2 CHAR(50),
    NationalTaxRateDesc CHAR(50),
    NationalTaxRate DECIMAL(6, 5),
    LocalTaxRateDesc CHAR(50),
    LocalTaxRate DECIMAL(6, 5),
    AgencyID CHAR(30),
    CreditRating SMALLINT UNSIGNED,
    NetWorth DECIMAL(10),
    MarketingNameplate CHAR(100),
    IsCurrent BOOLEAN NOT NULL,
    BatchID SMALLINT UNSIGNED NOT NULL,
    EffectiveDate DATE NOT NULL,
    EndDate DATE NOT NULL,
    PRIMARY KEY (SK_CustomerID)
);"""

In [73]:
with engine.connect() as conn:
    conn.execute(text(create_table))

In [74]:
dimCustomer_df.to_sql('dimcustomer', engine, if_exists='append', index=False, dtype=sql_dtypes)

10810

In [75]:
# Create temporary uppercase columns for merging in both DataFrames
merge_fields = ["FirstName", "LastName", "AddressLine1", "AddressLine2", "PostalCode"]
for field in merge_fields:
    prospect_df[f"temp_{field}"] = prospect_df[field].str.upper()
    dimCustomer_df[f"temp_{field}"] = dimCustomer_df[field].str.upper()

# Filter dimCustomer_df for active and current customers
active_customers = dimCustomer_df[(dimCustomer_df['IsCurrent'] == True) & (dimCustomer_df['Status'] == 'ACTIVE')]

# Perform an outer merge on the temporary uppercase fields
temp_merge_fields = [f"temp_{field}" for field in merge_fields]
merged_df = prospect_df.merge(active_customers, how='left', 
                              left_on=temp_merge_fields, right_on=temp_merge_fields,
                              indicator=True)

# Update IsCustomer based on whether a match was found
prospect_df['IsCustomer'] = merged_df['_merge'] == 'both'

# Clean up by dropping the temporary columns
prospect_df.drop(columns=temp_merge_fields, inplace=True)
dimCustomer_df.drop(columns=temp_merge_fields, inplace=True)

In [76]:
prospect_df['BatchID'] = 1

In [77]:
sql_dtypes = {
    'AgencyID': sqlalchemy.types.CHAR(30),
    'SK_RecordDateID': sqlalchemy.types.Integer,
    'SK_UpdateDateID': sqlalchemy.types.Integer,
    'BatchID': sqlalchemy.types.SmallInteger,
    'IsCustomer': sqlalchemy.types.Boolean,
    'LastName': sqlalchemy.types.CHAR(30),
    'FirstName': sqlalchemy.types.CHAR(30),
    'MiddleInitial': sqlalchemy.types.CHAR(1),
    'Gender': sqlalchemy.types.CHAR(1),
    'AddressLine1': sqlalchemy.types.CHAR(80),
    'AddressLine2': sqlalchemy.types.CHAR(80),
    'PostalCode': sqlalchemy.types.CHAR(12),
    'City': sqlalchemy.types.CHAR(25),
    'State': sqlalchemy.types.CHAR(20),
    'Country': sqlalchemy.types.CHAR(24),
    'Phone': sqlalchemy.types.CHAR(30),
    'Income': sqlalchemy.types.Integer,
    'NumberCars': sqlalchemy.types.SmallInteger,
    'NumberChildren': sqlalchemy.types.SmallInteger,
    'MaritalStatus': sqlalchemy.types.CHAR(1),
    'Age': sqlalchemy.types.SmallInteger,
    'CreditRating': sqlalchemy.types.SmallInteger,
    'OwnOrRentFlag': sqlalchemy.types.CHAR(1),
    'Employer': sqlalchemy.types.CHAR(30),
    'NumberCreditCards': sqlalchemy.types.SmallInteger,
    'NetWorth': sqlalchemy.types.BigInteger,
    'MarketingNameplate': sqlalchemy.types.CHAR(100)
}

In [78]:
create_table = """CREATE TABLE Prospect (
    AgencyID CHAR(30) NOT NULL,
    SK_RecordDateID INT UNSIGNED NOT NULL,
    SK_UpdateDateID INT UNSIGNED NOT NULL,
    BatchID SMALLINT UNSIGNED NOT NULL,
    IsCustomer BOOLEAN NOT NULL,
    LastName CHAR(30) NOT NULL,
    FirstName CHAR(30) NOT NULL,
    MiddleInitial CHAR(1),
    Gender CHAR(1) CHECK (Gender IN ('M', 'F', 'U')),
    AddressLine1 CHAR(80),
    AddressLine2 CHAR(80),
    PostalCode CHAR(12),
    City CHAR(25) NOT NULL,
    State CHAR(20) NOT NULL,
    Country CHAR(24),
    Phone CHAR(30),
    Income INT UNSIGNED,
    NumberCars TINYINT UNSIGNED,
    NumberChildren TINYINT UNSIGNED,
    MaritalStatus CHAR(1) CHECK (MaritalStatus IN ('S', 'M', 'D', 'W', 'U')),
    Age TINYINT UNSIGNED,
    CreditRating SMALLINT UNSIGNED,
    OwnOrRentFlag CHAR(1) CHECK (OwnOrRentFlag IN ('O', 'R', 'U')),
    Employer CHAR(30),
    NumberCreditCards TINYINT UNSIGNED,
    NetWorth BIGINT,
    MarketingNameplate CHAR(100),
    PRIMARY KEY (AgencyID, SK_RecordDateID)
);"""
with engine.connect() as conn:
    conn.execute(text(create_table))

In [79]:
prospect_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24970 entries, 0 to 24969
Data columns (total 27 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   AgencyID            24970 non-null  object
 1   SK_RecordDateID     24970 non-null  int64 
 2   SK_UpdateDateID     24970 non-null  int64 
 3   BatchID             24970 non-null  int64 
 4   IsCustomer          24970 non-null  bool  
 5   LastName            24970 non-null  object
 6   FirstName           24970 non-null  object
 7   MiddleInitial       10982 non-null  object
 8   Gender              24970 non-null  object
 9   AddressLine1        24097 non-null  object
 10  AddressLine2        17160 non-null  object
 11  PostalCode          24098 non-null  object
 12  City                24970 non-null  object
 13  State               24970 non-null  object
 14  Country             23929 non-null  object
 15  Phone               23674 non-null  object
 16  Income              23

In [80]:
prospect_df.to_sql('prospect', engine, if_exists='append', index=False, dtype=sql_dtypes)

24970

In [ ]:
prospect_end_time = datetime.now()
print(f"Prospect took {(prospect_end_time - prospect_start_time).total_seconds()}")

As the Prospect file is processed, the number of source rows is counted. After the last
row, a “Status” message is written to the DImessages table, with the MessageSource
“Prospect”, MessageText “Source rows” and the MessageData field containing the
number of rows.

In [81]:
num_rows = prospect_df.shape[0]
message_type = "Status"
message_source = "Prospect"
message_text = f"Inserted rows"
MessageDateAndTime = pd.Timestamp("now")
batch_id = 1

query = f"""INSERT INTO dimessages (MessageDateAndTime, BatchID, MessageSource, MessageText, MessageType, MessageData)
            VALUES ('{MessageDateAndTime}', {batch_id}, '{message_source}', '{message_text}', '{message_type}', '{num_rows}')"""
with engine.connect() as conn:
    conn.execute(text(query))
    conn.commit()

### dimAccount

In [82]:
# Define the schema as a dictionary
schema = {
    'SK_AccountID': 'uint32',
    'AccountID': 'uint32',
    'SK_BrokerID': 'uint32',
    'SK_CustomerID': 'uint32',
    'Status': 'str',
    'AccountDesc': 'str',
    'TaxStatus': 'UInt8',
    'IsCurrent': 'bool',
    'BatchID': 'uint8',
    'EffectiveDate': 'datetime64[ns]',
    'EndDate': 'datetime64[ns]'
}

In [83]:
data_file = DATA_DIR + "CustomerMgmt.xml"
tree = etree.parse(data_file)
namespace = {'tpcdi': 'http://www.tpc.org/tpc-di'}

# Get all actions
all_actions = tree.xpath(".//tpcdi:Action", namespaces=namespace)

In [84]:
# Create an empty DataFrame with the specified schema
dimAccount_df = pd.DataFrame({col: pd.Series(dtype=typ) for col, typ in schema.items()})

# initialize lists to store data
relevant_cols = ['AccountID', 'SK_BrokerID', 'SK_CustomerID', 'Status', 'AccountDesc', 'TaxStatus', 'EffectiveDate']
data = {col: [] for col in relevant_cols}

# initialize dict to store most recent values for each account of a customer
customer_accounts = dict()

for index, action in enumerate(tqdm(all_actions)):
    customer = action.find("Customer", namespaces=namespace)
    customer_id = int(customer.get("C_ID", None))
    if customer_id not in customer_accounts:
        customer_accounts[customer_id] = dict()
    if action.get("ActionType") in ("NEW", "ADDACCT"):
        accounts = action.findall('Customer/Account', namespaces=namespace)
        for account in accounts:
            # set effective date for this account
            action_ts = pd.to_datetime(action.get("ActionTS"), format="%Y-%m-%dT%H:%M:%S")
            data["EffectiveDate"].append(action_ts)
            # Customer/Account/@CA_ID
            account_id = account.get("CA_ID", None)
            account_id = int(account_id) if account_id else None
            data["AccountID"].append(account_id)
            # Customer/Account/CA_NAME
            account_desc = account.findtext("CA_NAME", default=None, namespaces=namespace)
            data["AccountDesc"].append(account_desc)
            # Customer/Account/@CA_TAX_ST
            tax_status = account.get("CA_TAX_ST", None)
            tax_status = int(tax_status) if tax_status else None
            data["TaxStatus"].append(tax_status)
            #  Customer/Account/CA_B_ID
            broker_id = account.findtext("CA_B_ID", default=None, namespaces=namespace)
            broker_id = int(broker_id) if broker_id else None
            data["SK_BrokerID"].append((broker_id, action_ts))
            data["SK_CustomerID"].append((customer_id, action_ts))
            status = "ACTIVE"
            data["Status"].append(status)
            # update customer_accounts
            customer_accounts[customer_id][account_id] = {
                "AccountDesc": account_desc,
                "TaxStatus": tax_status,
                "SK_BrokerID": (broker_id, action_ts),
                "SK_CustomerID": (customer_id, action_ts),
                "Status": status,
            }
    elif action.get("ActionType") == "UPDACCT":
        accounts = action.findall('Customer/Account', namespaces=namespace)
        for account in accounts:
            # set effective date for this account
            action_ts = pd.to_datetime(action.get("ActionTS"), format="%Y-%m-%dT%H:%M:%S")
            data["EffectiveDate"].append(action_ts)
            # Customer/Account/@CA_ID
            account_id = account.get("CA_ID", None)
            account_id = int(account_id) if account_id else None
            data["AccountID"].append(account_id)
            # Customer/Account/CA_NAME
            account_desc = account.findtext("CA_NAME", default=None, namespaces=namespace)
            if account_desc is None:
                account_desc = customer_accounts[customer_id][account_id]["AccountDesc"]
            else:
                customer_accounts[customer_id][account_id]["AccountDesc"] = account_desc
            data["AccountDesc"].append(account_desc)
            # Customer/Account/@CA_TAX_ST
            tax_status = account.get("CA_TAX_ST", None)
            tax_status = int(tax_status) if tax_status else None
            if tax_status is None:
                tax_status = customer_accounts[customer_id][account_id]["TaxStatus"]
            else:
                customer_accounts[customer_id][account_id]["TaxStatus"] = tax_status
            data["TaxStatus"].append(tax_status)
            #  Customer/Account/CA_B_ID
            broker_id = account.findtext("CA_B_ID", default=None, namespaces=namespace)
            broker_id = int(broker_id) if broker_id else None
            if broker_id is None:
                broker_id = customer_accounts[customer_id][account_id]["SK_BrokerID"][0]
            else:
                customer_accounts[customer_id][account_id]["SK_BrokerID"] = (broker_id, action_ts)
            sk_brokerid = (broker_id, action_ts)
            data["SK_BrokerID"].append(sk_brokerid)
            sk_customer_id = (customer_id, action_ts)
            customer_accounts[customer_id][account_id]["SK_CustomerID"] = sk_customer_id
            data["SK_CustomerID"].append(sk_customer_id)
            status = "ACTIVE"
            data["Status"].append(status)
    elif action.get("ActionType") == "UPDCUST":
        accounts = action.findall('Customer/Account', namespaces=namespace)
        for account in accounts:
            # set effective date for this account
            action_ts = pd.to_datetime(action.get("ActionTS"), format="%Y-%m-%dT%H:%M:%S")
            data["EffectiveDate"].append(action_ts)
            # Customer/Account/@CA_ID
            account_id = account.get("CA_ID", None)
            account_id = int(account_id) if account_id else None
            data["AccountID"].append(account_id)
            # set all other fields as is
            for col in customer_accounts[customer_id][account_id]:
                if not col.startswith("SK_"):
                    data[col].append(customer_accounts[customer_id][account_id][col])
            broker_id = customer_accounts[customer_id][account_id]["SK_BrokerID"][0]
            customer_accounts[customer_id][account_id]["SK_BrokerID"] = (broker_id, action_ts)
            sk_brokerid = (broker_id, action_ts)
            data["SK_BrokerID"].append(sk_brokerid)
            sk_customer_id = (customer_id, action_ts)
            customer_accounts[customer_id][account_id]["SK_CustomerID"] = sk_customer_id
            data["SK_CustomerID"].append(sk_customer_id)
    elif action.get("ActionType") in ("INACT", "CLOSEACCT"):
        accounts = action.findall('Customer/Account', namespaces=namespace)
        for account in accounts:
            # set effective date for this account
            action_ts = pd.to_datetime(action.get("ActionTS"), format="%Y-%m-%dT%H:%M:%S")
            data["EffectiveDate"].append(action_ts)
            # Customer/Account/@CA_ID
            account_id = account.get("CA_ID", None)
            account_id = int(account_id) if account_id else None
            data["AccountID"].append(account_id)
            # set all other fields as is
            for col in customer_accounts[customer_id][account_id]:
                if col.startswith("SK_"):
                    continue
                elif col != "Status":
                    data[col].append(customer_accounts[customer_id][account_id][col])
                else:
                    data[col].append("INACTIVE")
                    customer_accounts[customer_id][account_id][col] = "INACTIVE"
            broker_id = customer_accounts[customer_id][account_id]["SK_BrokerID"][0]
            customer_accounts[customer_id][account_id]["SK_BrokerID"] = (broker_id, action_ts)
            sk_brokerid = (broker_id, action_ts)
            data["SK_BrokerID"].append(sk_brokerid)
            sk_customer_id = (customer_id, action_ts)
            customer_accounts[customer_id][account_id]["SK_CustomerID"] = sk_customer_id
            data["SK_CustomerID"].append(sk_customer_id)

  0%|          | 0/24570 [00:00<?, ?it/s]

In [85]:
# query the database to get all SK_BrokerID
query_parts = [
    f"(BrokerID = {broker_id} AND EffectiveDate <= '{action_ts}' <= EndDate)"
    for broker_id, action_ts in data["SK_BrokerID"]
]
# Joining all conditions with 'OR'
conditions = " OR ".join(query_parts)
query = f"""SELECT BrokerID, EffectiveDate, EndDate, SK_BrokerID 
FROM dimbroker 
WHERE {conditions}"""
result = pd.read_sql_query(query, engine)
for index, pair in enumerate(data["SK_BrokerID"]):
    broker_id, action_ts = pair    
    sk_brokerid = result[
        (result["BrokerID"] == broker_id)
        & (result["EffectiveDate"] <= action_ts.date())
        & (action_ts.date() <= result["EndDate"])
    ].iloc[0, 3]
    data["SK_BrokerID"][index] = sk_brokerid

# query the database to get all SK_CustomerID
query_parts = [
    f"(CustomerID = {customer_id} AND EffectiveDate <= '{action_ts}' <= EndDate)"
    for customer_id, action_ts in data["SK_CustomerID"]
]
# Joining all conditions with 'OR'
conditions = " OR ".join(query_parts)
query = f"""SELECT CustomerID, EffectiveDate, EndDate, SK_CustomerID 
FROM dimcustomer 
WHERE {conditions}"""
result = pd.read_sql_query(query, engine)
for index, pair in enumerate(data["SK_CustomerID"]):
    customer_id, action_ts = pair
    sk_brokerid = result[
        (result["CustomerID"] == customer_id)
        & (result["EffectiveDate"] <= action_ts.date())
        & (action_ts.date() <= result["EndDate"])
    ].iloc[0, 3]
    data["SK_CustomerID"][index] = sk_brokerid

In [86]:
for col in data:
    dimAccount_df[col] = data[col]
dimAccount_df['SK_AccountID'] = range(1, len(dimAccount_df) + 1)
dimAccount_df['BatchID'] = 1

# Sort the DataFrame by CustomerID and EffectiveDate
dimAccount_df.sort_values(by=['AccountID', 'EffectiveDate'], inplace=True)
# Create a shifted DataFrame
shifted_df = dimAccount_df.shift(-1)
# Update EndDate: If next row has same CustomerID, use its EffectiveDate; otherwise, use default date
dimAccount_df['EndDate'] = pd.Timestamp('9999-12-31')
mask = dimAccount_df['AccountID'] == shifted_df['AccountID']
dimAccount_df.loc[mask, 'EndDate'] = shifted_df.loc[mask, 'EffectiveDate']

# Update IsCurrent: True if next row has different CustomerID or is the last row
dimAccount_df['IsCurrent'] = ~mask
dimAccount_df.sort_values(by=['SK_AccountID'], inplace=True)

In [87]:
sql_dtypes = {
    'SK_AccountID': sqlalchemy.types.Integer,
    'AccountID': sqlalchemy.types.Integer,
    'SK_BrokerID': sqlalchemy.types.Integer,
    'SK_CustomerID': sqlalchemy.types.Integer,
    'Status': sqlalchemy.types.String(10),
    'AccountDesc': sqlalchemy.types.String(50),
    'TaxStatus': sqlalchemy.types.SmallInteger,
    'IsCurrent': sqlalchemy.types.Boolean,
    'BatchID': sqlalchemy.types.SmallInteger,
    'EffectiveDate': sqlalchemy.types.Date,
    'EndDate': sqlalchemy.types.Date
}

In [88]:
create_table = """CREATE TABLE DimAccount (
    SK_AccountID INT UNSIGNED NOT NULL,
    AccountID INT UNSIGNED NOT NULL,
    SK_BrokerID INT UNSIGNED NOT NULL,
    SK_CustomerID INT UNSIGNED NOT NULL,
    Status CHAR(10) NOT NULL,
    AccountDesc CHAR(50),
    TaxStatus TINYINT UNSIGNED,
    IsCurrent BOOLEAN NOT NULL,
    BatchID SMALLINT UNSIGNED NOT NULL,
    EffectiveDate DATE NOT NULL,
    EndDate DATE NOT NULL,
    PRIMARY KEY (SK_AccountID)
);"""
with engine.connect() as conn:
    conn.execute(text(create_table))

In [89]:
dimAccount_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21560 entries, 0 to 21559
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   SK_AccountID   21560 non-null  int64         
 1   AccountID      21560 non-null  int64         
 2   SK_BrokerID    21560 non-null  int64         
 3   SK_CustomerID  21560 non-null  int64         
 4   Status         21560 non-null  object        
 5   AccountDesc    21560 non-null  object        
 6   TaxStatus      21560 non-null  int64         
 7   IsCurrent      21560 non-null  bool          
 8   BatchID        21560 non-null  int64         
 9   EffectiveDate  21560 non-null  datetime64[ns]
 10  EndDate        21560 non-null  datetime64[s] 
dtypes: bool(1), datetime64[ns](1), datetime64[s](1), int64(6), object(2)
memory usage: 1.8+ MB


In [90]:
dimAccount_df.to_sql('dimaccount', engine, if_exists='append', index=False, dtype=sql_dtypes)

21560

### dimTrade

In [91]:
columns = [
    'T_ID', 'T_DTS', 'T_ST_ID', 'T_TT_ID', 'T_IS_CASH', 
    'T_S_SYMB', 'T_QTY', 'T_BID_PRICE', 'T_CA_ID', 'T_EXEC_NAME', 
    'T_TRADE_PRICE', 'T_CHRG', 'T_COMM', 'T_TAX'
]
dtypes = {
    'T_ID': 'uint64',
    'T_DTS': 'str',
    'T_ST_ID': 'str',
    'T_TT_ID': 'str',
    'T_IS_CASH': 'bool',
    'T_S_SYMB': 'str',
    'T_QTY': 'uint32',
    'T_BID_PRICE': 'float64',
    'T_CA_ID': 'uint32',
    'T_EXEC_NAME': 'str',
    'T_TRADE_PRICE': 'float64',
    'T_CHRG': 'float64',
    'T_COMM': 'float64',
    'T_TAX': 'float64'
}

# Read the file into a DataFrame
trade_df = pd.read_csv(
    DATA_DIR + "Trade.txt", 
    sep='|', 
    header=None, 
    names=columns, 
    dtype=dtypes,
    parse_dates=['T_DTS']
)
trade_df['T_DTS'] = pd.to_datetime(trade_df['T_DTS'])

In [92]:
columns = ["TH_T_ID", "TH_DTS", "TH_ST_ID"]
dtypes = {
    "TH_T_ID": "uint64",
    "TH_DTS": "str",
    "TH_ST_ID": "str"
}
tradehistory_df = pd.read_csv(DATA_DIR + "TradeHistory.txt", sep="|", header=None, 
                              names=columns, dtype=dtypes, parse_dates=['TH_DTS'])
tradehistory_df['TH_DTS'] = pd.to_datetime(tradehistory_df['TH_DTS'])

In [93]:
trade_merged = tradehistory_df.merge(trade_df, left_on='TH_T_ID', right_on='T_ID')
del tradehistory_df
del trade_df

In [94]:
# Pre-fetch data from related tables
date_mapping = pd.read_sql("SELECT DateValue, SK_DateID FROM dimdate", engine).set_index("DateValue")["SK_DateID"].to_dict()
time_mapping = pd.read_sql("SELECT TimeValue, SK_TimeID FROM dimtime", engine).set_index("TimeValue")["SK_TimeID"].to_dict()
status_mapping = pd.read_sql("SELECT ST_ID, ST_NAME FROM statustype", engine).set_index("ST_ID")["ST_NAME"].to_dict()
trade_type_mapping = pd.read_sql("SELECT TT_ID, TT_NAME FROM tradetype", engine).set_index("TT_ID")["TT_NAME"].to_dict()

# Fetching security and account info in one go
security_info = pd.read_sql("SELECT Symbol, SK_SecurityID, SK_CompanyID, EffectiveDate, EndDate FROM dimsecurity", engine)
security_info['EffectiveDate'] = pd.to_datetime(security_info['EffectiveDate'])
account_info = pd.read_sql("SELECT AccountID, SK_AccountID, SK_CustomerID, SK_BrokerID, EffectiveDate, EndDate FROM dimaccount", engine)
account_info['EffectiveDate'] = pd.to_datetime(account_info['EffectiveDate'])

In [95]:
# direct copy
trade_merged["TradeID"] = trade_merged["T_ID"]
trade_merged["CashFlag"] = trade_merged["T_IS_CASH"]
trade_merged["Quantity"] = trade_merged["T_QTY"]
trade_merged["BidPrice"] = trade_merged["T_BID_PRICE"]
trade_merged["ExecutedBy"] = trade_merged["T_EXEC_NAME"]
trade_merged["TradePrice"] = trade_merged["T_TRADE_PRICE"]
trade_merged["Fee"] = trade_merged["T_CHRG"]
trade_merged["Commission"] = trade_merged["T_COMM"]
trade_merged["Tax"] = trade_merged["T_TAX"]
trade_merged["Status"] = trade_merged["T_ST_ID"].map(status_mapping)
trade_merged["Type"] = trade_merged["T_TT_ID"].map(trade_type_mapping)

In [96]:
# initially set null
trade_merged["SK_CreateDateID"] = None
trade_merged["SK_CreateTimeID"] = None
trade_merged["SK_CloseDateID"] = None
trade_merged["SK_CloseTimeID"] = None

# now populate
create_mask = (trade_merged["TH_ST_ID"] == "PNDG") | (trade_merged["TH_ST_ID"] == "SBMT")
trade_merged.loc[create_mask, "SK_CreateDateID"] = trade_merged.loc[create_mask, "TH_DTS"].dt.date.map(date_mapping)
trade_merged.loc[create_mask, "SK_CreateTimeID"] = pd.to_timedelta(trade_merged.loc[create_mask, "TH_DTS"].dt.time.astype(str)).map(time_mapping)
close_mask = (trade_merged["TH_ST_ID"] == "CMPT") | (trade_merged["TH_ST_ID"] == "CNCL")
trade_merged.loc[close_mask, "SK_CloseDateID"] = trade_merged.loc[close_mask, "TH_DTS"].dt.date.map(date_mapping)
trade_merged.loc[close_mask, "SK_CloseTimeID"] = pd.to_timedelta(trade_merged.loc[close_mask, "TH_DTS"].dt.time.astype(str)).map(time_mapping)

In [97]:
trade_merged.rename({"T_S_SYMB": "Symbol"}, axis=1, inplace=True)

In [98]:
trade_merged = pd.merge(
    trade_merged,
    security_info,
    how="left",
    on="Symbol",
)

In [99]:
# drop the rows where TH_DTS < EffectiveDate  or TH_DTS > EndDate
trade_merged = trade_merged[
    (trade_merged["TH_DTS"] >= trade_merged["EffectiveDate"])
    & (trade_merged["TH_DTS"].dt.date < trade_merged["EndDate"])
]

In [100]:
trade_merged = pd.merge(
    trade_merged,
    account_info,
    how="left",
    left_on="T_CA_ID",
    right_on="AccountID",
)
trade_merged = trade_merged[
    (trade_merged["TH_DTS"] >= trade_merged["EffectiveDate_y"])
    & (trade_merged["TH_DTS"].dt.date < trade_merged["EndDate_y"])
]

In [101]:
use_cols = [
    "TradeID",
    "SK_BrokerID",
    "SK_CreateDateID",
    "SK_CreateTimeID",
    "SK_CloseDateID",
    "SK_CloseTimeID",
    "Status",
    "Type",
    "CashFlag",
    "SK_SecurityID",
    "SK_CompanyID",
    "Quantity",
    "BidPrice",
    "SK_CustomerID",
    "SK_AccountID",
    "ExecutedBy",
    "TradePrice",
    "Fee",
    "Commission",
    "Tax",
]
trade_merged = trade_merged[use_cols]

In [102]:
trade_merged = trade_merged.groupby("TradeID").last().reset_index()
trade_merged['BatchID'] = 1

In [103]:
trade_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 650412 entries, 0 to 650411
Data columns (total 21 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   TradeID          650412 non-null  uint64 
 1   SK_BrokerID      650412 non-null  int64  
 2   SK_CreateDateID  650412 non-null  object 
 3   SK_CreateTimeID  650412 non-null  object 
 4   SK_CloseDateID   640835 non-null  object 
 5   SK_CloseTimeID   640835 non-null  object 
 6   Status           650412 non-null  object 
 7   Type             650412 non-null  object 
 8   CashFlag         650412 non-null  bool   
 9   SK_SecurityID    650412 non-null  int64  
 10  SK_CompanyID     650412 non-null  int64  
 11  Quantity         650412 non-null  uint32 
 12  BidPrice         650412 non-null  float64
 13  SK_CustomerID    650412 non-null  int64  
 14  SK_AccountID     650412 non-null  int64  
 15  ExecutedBy       650412 non-null  object 
 16  TradePrice       602956 non-null  floa

In [104]:
dtypes = {
    'TradeID': 'uint32',
    'SK_BrokerID': 'UInt32',
    'SK_CreateDateID': 'uint32',
    'SK_CreateTimeID': 'uint32',
    'SK_CloseDateID': 'UInt32',
    'SK_CloseTimeID': 'UInt32',
    'Status': 'str',
    'Type': 'str',
    'CashFlag': 'bool',
    'SK_SecurityID': 'uint32',
    'SK_CompanyID': 'uint32',
    'Quantity': 'uint32',
    'BidPrice': 'float64',
    'SK_CustomerID': 'uint32',
    'SK_AccountID': 'uint32',
    'ExecutedBy': 'str',
    'TradePrice': 'float64',
    'Fee': 'float64',
    'Commission': 'float64',
    'Tax': 'float64',
    'BatchID': 'uint8'
}
trade_merged = trade_merged.astype(dtypes)

In [105]:
sql_dtypes = {
    'TradeID': sqlalchemy.types.Integer,
    'SK_BrokerID': sqlalchemy.types.Integer,
    'SK_CreateDateID': sqlalchemy.types.Integer,
    'SK_CreateTimeID': sqlalchemy.types.Integer,
    'SK_CloseDateID': sqlalchemy.types.Integer,
    'SK_CloseTimeID': sqlalchemy.types.Integer,
    'Status': sqlalchemy.types.CHAR(10),
    'Type': sqlalchemy.types.CHAR(12),
    'CashFlag': sqlalchemy.types.Boolean,
    'SK_SecurityID': sqlalchemy.types.Integer,
    'SK_CompanyID': sqlalchemy.types.Integer,
    'Quantity': sqlalchemy.types.Integer,
    'BidPrice': sqlalchemy.types.Numeric(8, 2),
    'SK_CustomerID': sqlalchemy.types.Integer,
    'SK_AccountID': sqlalchemy.types.Integer,
    'ExecutedBy': sqlalchemy.types.CHAR(64),
    'TradePrice': sqlalchemy.types.Numeric(8, 2),
    'Fee': sqlalchemy.types.Numeric(10, 2),
    'Commission': sqlalchemy.types.Numeric(10, 2),
    'Tax': sqlalchemy.types.Numeric(10, 2),
    'BatchID': sqlalchemy.types.SmallInteger
}


In [106]:
create_table = """CREATE TABLE DimTrade (
    TradeID INT UNSIGNED NOT NULL,
    SK_BrokerID INT UNSIGNED,
    SK_CreateDateID INT UNSIGNED NOT NULL,
    SK_CreateTimeID INT UNSIGNED NOT NULL,
    SK_CloseDateID INT UNSIGNED,
    SK_CloseTimeID INT UNSIGNED,
    Status CHAR(10) NOT NULL,
    Type CHAR(12) NOT NULL,
    CashFlag BOOLEAN NOT NULL,
    SK_SecurityID INT UNSIGNED NOT NULL,
    SK_CompanyID INT UNSIGNED NOT NULL,
    Quantity MEDIUMINT UNSIGNED NOT NULL,
    BidPrice DECIMAL(8, 2) NOT NULL,
    SK_CustomerID INT UNSIGNED NOT NULL,
    SK_AccountID INT UNSIGNED NOT NULL,
    ExecutedBy CHAR(64) NOT NULL,
    TradePrice DECIMAL(8, 2),
    Fee DECIMAL(10, 2),
    Commission DECIMAL(10, 2),
    Tax DECIMAL(10, 2),
    BatchID SMALLINT UNSIGNED NOT NULL,
    PRIMARY KEY (TradeID)
);"""
with engine.connect() as conn:
    conn.execute(text(create_table))

In [107]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()
try:
    for i in trange(0, trade_merged.shape[0], 100000):
        trade_merged.iloc[i:i+100000].to_sql('dimtrade', engine, if_exists='append', index=False, dtype=sql_dtypes)
    session.commit()
except:
    session.rollback()
    raise
finally:
    session.close()

  0%|          | 0/7 [00:00<?, ?it/s]

In [108]:
# Filter the DataFrame
invalid_trades = trade_merged[
    (trade_merged["Commission"].notnull())
    & (trade_merged["Commission"] > (trade_merged["TradePrice"] * trade_merged["Quantity"]))
]
print(invalid_trades.shape)

(81, 21)


In [109]:
# Create lists without using iterrows
MessageSource = ["DimTrade"] * len(invalid_trades)
MessageType = ["Alert"] * len(invalid_trades)
MessageText = ["Invalid trade commission"] * len(invalid_trades)
MessageData = [
    "T_ID = "
    + invalid_trades["TradeID"].astype(str)
    + ", T_COMM = "
    + invalid_trades["Commission"].astype(str)
]
# Convert MessageData from a list of Series to a list of strings
MessageData = MessageData[0].tolist()

print(len(MessageSource))
print(len(MessageData))

81
81


In [110]:
query = f"""INSERT INTO Dimessages (MessageDateAndTime, BatchID, MessageSource, MessageText, MessageType, MessageData)
VALUES """
for i in range(len(MessageSource)):
    query += f"""('{pd.Timestamp("now")}', 1, '{MessageSource[i]}', '{MessageText[i]}', '{MessageType[i]}', '{MessageData[i]}'),"""
with engine.connect() as conn:
    conn.execute(text(query[:-1]))
    conn.commit()

In [111]:
# Filter the DataFrame for invalid trade fees
invalid_fee_trades = trade_merged[
    (trade_merged["Fee"].notnull())
    & (trade_merged["Fee"] > (trade_merged["TradePrice"] * trade_merged["Quantity"]))
]

# Create the required lists
MessageSource = ["DimTrade"] * len(invalid_fee_trades)
MessageType = ["Alert"] * len(invalid_fee_trades)
MessageText = ["Invalid trade fee"] * len(invalid_fee_trades)

# Vectorized operation for MessageData
MessageData = (
    "T_ID = "
    + invalid_fee_trades["TradeID"].astype(str)
    + ", T_CHRG = "
    + invalid_fee_trades["Fee"].astype(str)
)
MessageData = MessageData.tolist()

query = """INSERT INTO Dimessages (MessageDateAndTime, BatchID, MessageSource, MessageText, MessageType, MessageData) VALUES """
for i in range(len(MessageSource)):
    query += f"""('{pd.Timestamp("now")}', 1, '{MessageSource[i]}', '{MessageText[i]}', '{MessageType[i]}', '{MessageData[i]}'),"""
with engine.connect() as conn:
    conn.execute(text(query[:-1]))
    conn.commit()

### FactCashBalances

In [ ]:
fcb_start_time = datetime.now()

In [112]:
cash_txn_df = pd.read_csv(
    DATA_DIR + "CashTransaction.txt",
    sep="|",
    header=None,
    names=[
        "CT_CA_ID",
        "CT_DTS",
        "CT_AMT",
        "CT_NAME"
    ],
    dtype={
        "CT_CA_ID": "uint32",
        "CT_DTS": "str",
        "CT_AMT": "float64",
        "CT_NAME": "str"
    },
    parse_dates=["CT_DTS"],
)
cash_txn_df["CT_DTS"] = pd.to_datetime(cash_txn_df["CT_DTS"])
cash_txn_df.info()
cash_txn_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 602115 entries, 0 to 602114
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   CT_CA_ID  602115 non-null  uint32        
 1   CT_DTS    602115 non-null  datetime64[ns]
 2   CT_AMT    602115 non-null  float64       
 3   CT_NAME   602115 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(1), uint32(1)
memory usage: 16.1+ MB


,CT_CA_ID,CT_DTS,CT_AMT,CT_NAME
0,2,2012-07-11 08:10:15,-37215.14,TGDRsaHPherhApDuHfXUPdexIUoEzKdgRAGECsgXJIRZui...
1,34,2012-07-07 17:11:38,-3178.67,PGwhaPC igAVOmHLJppGbXaDEgHSurSvrCvKtSyJRfHyRV...
2,36,2012-07-12 17:37:44,-3172.19,uQOUlrpDGHQpeeBGxaLTrxUwMwNMFowWAjqENSawehdTQd...
3,40,2012-09-20 03:12:34,-16621.00,VRIGhrJYHmbmNyXtIutswBfyfSLRjEJdGhuHOGWHscOlcC...
4,19,2012-07-09 07:32:41,-1315.70,gySbOpZLevgVdfrrwPiqBrFJFQGWehUzTSmkIhCJUBLLmV...


SK_CustomerID and SK_AccountID are obtained from DimAccount by matching CT_CA_ID
with AccountID, where CT_DTS is in the range given by EffectiveDate and EndDate


In [113]:
account_info = pd.read_sql(
    "SELECT AccountID, SK_AccountID, SK_CustomerID, EffectiveDate, EndDate FROM dimaccount",
    engine,
)
account_info["EffectiveDate"] = pd.to_datetime(account_info["EffectiveDate"])
account_info.head()

,AccountID,SK_AccountID,SK_CustomerID,EffectiveDate,EndDate
0,0,1,1,2007-07-07,2007-10-15
1,1,2,2,2007-07-07,2007-07-21
2,2,3,3,2007-07-07,2007-07-23
3,3,4,4,2007-07-07,2007-09-16
4,4,5,5,2007-07-07,2007-07-27


In [114]:
cash_txn_df = cash_txn_df.merge(
    account_info,
    how="left",
    left_on="CT_CA_ID",
    right_on="AccountID",
)
cash_txn_df.info()
cash_txn_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197606 entries, 0 to 1197605
Data columns (total 9 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   CT_CA_ID       1197606 non-null  uint32        
 1   CT_DTS         1197606 non-null  datetime64[ns]
 2   CT_AMT         1197606 non-null  float64       
 3   CT_NAME        1197606 non-null  object        
 4   AccountID      1197606 non-null  int64         
 5   SK_AccountID   1197606 non-null  int64         
 6   SK_CustomerID  1197606 non-null  int64         
 7   EffectiveDate  1197606 non-null  datetime64[ns]
 8   EndDate        1197606 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(3), object(2), uint32(1)
memory usage: 77.7+ MB


,CT_CA_ID,CT_DTS,CT_AMT,CT_NAME,AccountID,SK_AccountID,SK_CustomerID,EffectiveDate,EndDate
0,2,2012-07-11 08:10:15,-37215.14,TGDRsaHPherhApDuHfXUPdexIUoEzKdgRAGECsgXJIRZui...,2,3,3,2007-07-07,2007-07-23
1,2,2012-07-11 08:10:15,-37215.14,TGDRsaHPherhApDuHfXUPdexIUoEzKdgRAGECsgXJIRZui...,2,109,9952,2007-07-23,2007-07-28
2,2,2012-07-11 08:10:15,-37215.14,TGDRsaHPherhApDuHfXUPdexIUoEzKdgRAGECsgXJIRZui...,2,133,9952,2007-07-28,9999-12-31
3,34,2012-07-07 17:11:38,-3178.67,PGwhaPC igAVOmHLJppGbXaDEgHSurSvrCvKtSyJRfHyRV...,34,35,35,2007-07-11,2007-08-11
4,34,2012-07-07 17:11:38,-3178.67,PGwhaPC igAVOmHLJppGbXaDEgHSurSvrCvKtSyJRfHyRV...,34,220,35,2007-08-11,9999-12-31


In [115]:
cash_txn_df = cash_txn_df[
    (cash_txn_df["CT_DTS"] >= cash_txn_df["EffectiveDate"])
    & (cash_txn_df["CT_DTS"].dt.date < cash_txn_df["EndDate"])
]
cash_txn_df.info()
cash_txn_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 602115 entries, 2 to 1197605
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   CT_CA_ID       602115 non-null  uint32        
 1   CT_DTS         602115 non-null  datetime64[ns]
 2   CT_AMT         602115 non-null  float64       
 3   CT_NAME        602115 non-null  object        
 4   AccountID      602115 non-null  int64         
 5   SK_AccountID   602115 non-null  int64         
 6   SK_CustomerID  602115 non-null  int64         
 7   EffectiveDate  602115 non-null  datetime64[ns]
 8   EndDate        602115 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(3), object(2), uint32(1)
memory usage: 43.6+ MB


,CT_CA_ID,CT_DTS,CT_AMT,CT_NAME,AccountID,SK_AccountID,SK_CustomerID,EffectiveDate,EndDate
2,2,2012-07-11 08:10:15,-37215.14,TGDRsaHPherhApDuHfXUPdexIUoEzKdgRAGECsgXJIRZui...,2,133,9952,2007-07-28,9999-12-31
4,34,2012-07-07 17:11:38,-3178.67,PGwhaPC igAVOmHLJppGbXaDEgHSurSvrCvKtSyJRfHyRV...,34,220,35,2007-08-11,9999-12-31
8,36,2012-07-12 17:37:44,-3172.19,uQOUlrpDGHQpeeBGxaLTrxUwMwNMFowWAjqENSawehdTQd...,36,5062,10007,2009-11-09,9999-12-31
10,40,2012-09-20 03:12:34,-16621.00,VRIGhrJYHmbmNyXtIutswBfyfSLRjEJdGhuHOGWHscOlcC...,40,1050,9957,2007-12-30,9999-12-31
14,19,2012-07-09 07:32:41,-1315.70,gySbOpZLevgVdfrrwPiqBrFJFQGWehUzTSmkIhCJUBLLmV...,19,6282,9960,2010-06-04,9999-12-31


SK_DateID is obtained from DimDate by matching just the date portion of CT_DTS with
DateValue to return the SK_DateID.

In [116]:
date_info = pd.read_sql("SELECT DateValue, SK_DateID FROM dimdate", engine)
date_info["DateValue"] = pd.to_datetime(date_info["DateValue"])
date_info.info()
date_info.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25933 entries, 0 to 25932
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   DateValue  25933 non-null  datetime64[ns]
 1   SK_DateID  25933 non-null  int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 405.3 KB


,DateValue,SK_DateID
0,1950-01-01,19500101
1,1950-01-02,19500102
2,1950-01-03,19500103
3,1950-01-04,19500104
4,1950-01-05,19500105


In [117]:
cash_txn_df['SK_DateID'] = cash_txn_df['CT_DTS'].dt.date.map(date_info.set_index('DateValue')['SK_DateID'])
cash_txn_df.info()
cash_txn_df.head()

<class 'pandas.core.frame.DataFrame'>
Index: 602115 entries, 2 to 1197605
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   CT_CA_ID       602115 non-null  uint32        
 1   CT_DTS         602115 non-null  datetime64[ns]
 2   CT_AMT         602115 non-null  float64       
 3   CT_NAME        602115 non-null  object        
 4   AccountID      602115 non-null  int64         
 5   SK_AccountID   602115 non-null  int64         
 6   SK_CustomerID  602115 non-null  int64         
 7   EffectiveDate  602115 non-null  datetime64[ns]
 8   EndDate        602115 non-null  object        
 9   SK_DateID      602115 non-null  int64         
dtypes: datetime64[ns](2), float64(1), int64(4), object(2), uint32(1)
memory usage: 48.2+ MB


,CT_CA_ID,CT_DTS,CT_AMT,CT_NAME,AccountID,SK_AccountID,SK_CustomerID,EffectiveDate,EndDate,SK_DateID
2,2,2012-07-11 08:10:15,-37215.14,TGDRsaHPherhApDuHfXUPdexIUoEzKdgRAGECsgXJIRZui...,2,133,9952,2007-07-28,9999-12-31,20120711
4,34,2012-07-07 17:11:38,-3178.67,PGwhaPC igAVOmHLJppGbXaDEgHSurSvrCvKtSyJRfHyRV...,34,220,35,2007-08-11,9999-12-31,20120707
8,36,2012-07-12 17:37:44,-3172.19,uQOUlrpDGHQpeeBGxaLTrxUwMwNMFowWAjqENSawehdTQd...,36,5062,10007,2009-11-09,9999-12-31,20120712
10,40,2012-09-20 03:12:34,-16621.00,VRIGhrJYHmbmNyXtIutswBfyfSLRjEJdGhuHOGWHscOlcC...,40,1050,9957,2007-12-30,9999-12-31,20120920
14,19,2012-07-09 07:32:41,-1315.70,gySbOpZLevgVdfrrwPiqBrFJFQGWehUzTSmkIhCJUBLLmV...,19,6282,9960,2010-06-04,9999-12-31,20120709


Cash is calculated as the sum of the prior Cash amount for this account plus the sum of all
CT_AMT values from all transactions in this account on this day. If there is no previous
FactCashBalances record for the associated account, zero is used. Remember that the net effect of all cash transactions for a given account on a given day is totaled, and only a single record is generated per account that had changes per day.

The procedure used to determine the new Cash total must account for the possibility that a
new surrogate key is created in DimAccount since the last cash transaction.


In [118]:
# Sort the DataFrame by account ID and transaction date
cash_txn_df.sort_values(by=['AccountID', 'CT_DTS'], inplace=True)

# Create a new column to store the prior cash amount
cash_txn_df['PriorCash'] = cash_txn_df.groupby('AccountID')['CT_AMT'].cumsum() - cash_txn_df['CT_AMT']
cash_txn_df['PriorCash'].fillna(0, inplace=True)

# Calculate the cash balance
cash_txn_df['Cash'] = cash_txn_df['PriorCash'] + cash_txn_df['CT_AMT']

# Keep only the last record for each account on each day
cash_txn_df = cash_txn_df.groupby(['AccountID', 'SK_DateID']).last().reset_index()

cash_txn_df.info()
cash_txn_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549361 entries, 0 to 549360
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   AccountID      549361 non-null  int64         
 1   SK_DateID      549361 non-null  int64         
 2   CT_CA_ID       549361 non-null  uint32        
 3   CT_DTS         549361 non-null  datetime64[ns]
 4   CT_AMT         549361 non-null  float64       
 5   CT_NAME        549361 non-null  object        
 6   SK_AccountID   549361 non-null  int64         
 7   SK_CustomerID  549361 non-null  int64         
 8   EffectiveDate  549361 non-null  datetime64[ns]
 9   EndDate        549361 non-null  object        
 10  PriorCash      549361 non-null  float64       
 11  Cash           549361 non-null  float64       
dtypes: datetime64[ns](2), float64(3), int64(4), object(2), uint32(1)
memory usage: 48.2+ MB


,AccountID,SK_DateID,CT_CA_ID,CT_DTS,CT_AMT,CT_NAME,SK_AccountID,SK_CustomerID,EffectiveDate,EndDate,PriorCash,Cash
0,0,20120708,0,2012-07-08 17:58:21,-71563.60,hyaGSBCOXigNPHVAQxiPjJFgKBcBBQGDBQDpBxISwKkJ,866,7802,2007-11-29,9999-12-31,0.00,-71563.60
1,0,20120710,0,2012-07-10 22:12:48,-89884.06,gSTsTBZrTEm PMYdHvhKxaJTrNxtosRUsJDwGBqwTbbjDO...,866,7802,2007-11-29,9999-12-31,3249.91,-86634.15
2,0,20120711,0,2012-07-11 04:19:10,8110.34,JGEQbvMIqUzPAbNiKtveoxbSUHRxlHkxdGZKeTdXWqhWGN...,866,7802,2007-11-29,9999-12-31,-95309.80,-87199.46
3,0,20120712,0,2012-07-12 21:36:34,1641.86,YAROnGbtCACCSuAbFQJxIZxwibmZPuvKjoENySzzhmajQe...,866,7802,2007-11-29,9999-12-31,-113718.54,-112076.68
4,0,20120713,0,2012-07-13 16:23:08,-52115.15,xFePvUHkWgBKpuTjsCBcnRCYTUjEtIKYNtgeBVLTsQZfHj...,866,7802,2007-11-29,9999-12-31,-604254.35,-656369.50


In [119]:
keep_cols = [
    "SK_CustomerID",
    "SK_AccountID",
    "SK_DateID",
    "Cash",
]
cash_txn_df = cash_txn_df[keep_cols]
cash_txn_df['BatchID'] = 1

In [120]:
cash_txn_df.info()
cash_txn_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 549361 entries, 0 to 549360
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   SK_CustomerID  549361 non-null  int64  
 1   SK_AccountID   549361 non-null  int64  
 2   SK_DateID      549361 non-null  int64  
 3   Cash           549361 non-null  float64
 4   BatchID        549361 non-null  int64  
dtypes: float64(1), int64(4)
memory usage: 21.0 MB


,SK_CustomerID,SK_AccountID,SK_DateID,Cash,BatchID
0,7802,866,20120708,-71563.60,1
1,7802,866,20120710,-86634.15,1
2,7802,866,20120711,-87199.46,1
3,7802,866,20120712,-112076.68,1
4,7802,866,20120713,-656369.50,1


In [121]:
create_table = """CREATE TABLE FactCashBalances (
    SK_CustomerID INT UNSIGNED NOT NULL,
    SK_AccountID INT UNSIGNED NOT NULL,
    SK_DateID INT UNSIGNED NOT NULL,
    Cash DECIMAL(15, 2) NOT NULL,
    BatchID SMALLINT UNSIGNED NOT NULL
);"""

with engine.connect() as connection:
    connection.execute(text(create_table))

In [122]:
sql_dtypes = {
    "SK_CustomerID": sqlalchemy.types.Integer,
    "SK_AccountID": sqlalchemy.types.Integer,
    "SK_DateID": sqlalchemy.types.Integer,
    "Cash": sqlalchemy.types.DECIMAL(precision=15, scale=2),
    "BatchID": sqlalchemy.types.SmallInteger
}

In [123]:
for i in trange(0, cash_txn_df.shape[0], 100000):
    cash_txn_df.iloc[i:i+100000].to_sql('factcashbalances', engine, if_exists='append', index=False, dtype=sql_dtypes)

  0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
fcb_end_time = datetime.now()
print(f"fcb took {(fcb_end_time - fcb_start_time).total_seconds()}")

### FactHoldings

In [ ]:
fh_start_time = datetime.now()

In [124]:
# SQL queries
sql_commands = [
    "DROP TABLE IF EXISTS TempHoldingHistory",
    """
    CREATE TEMPORARY TABLE TempHoldingHistory (
        HH_H_T_ID INT UNSIGNED NOT NULL,
        HH_T_ID INT UNSIGNED NOT NULL,
        HH_BEFORE_QTY INT NOT NULL,
        HH_AFTER_QTY INT NOT NULL
    )
    """,
    """
    LOAD DATA LOCAL INFILE 'E:\\\\Documents\\\\BDMA\\\\ULB\\\\Data Warehouses\\\\tpc-di\\\\TPC-DI\\\\data\\\\sf5\\\\Batch1\\\\HoldingHistory.txt'
    INTO TABLE TempHoldingHistory
    FIELDS TERMINATED BY '|'
    LINES TERMINATED BY '\n'
    (HH_H_T_ID, HH_T_ID, HH_BEFORE_QTY, HH_AFTER_QTY)
    """,
    "DROP TABLE IF EXISTS FactHoldings",
    """
    CREATE TABLE FactHoldings (
        TradeID INT UNSIGNED NOT NULL,
        CurrentTradeID INT UNSIGNED NOT NULL,
        SK_CustomerID INT UNSIGNED NOT NULL,
        SK_AccountID INT UNSIGNED NOT NULL,
        SK_SecurityID INT UNSIGNED NOT NULL,
        SK_CompanyID INT UNSIGNED NOT NULL,
        SK_DateID INT UNSIGNED NOT NULL,
        SK_TimeID INT UNSIGNED NOT NULL,
        CurrentPrice DECIMAL(8, 2) NOT NULL CHECK (CurrentPrice > 0),
        CurrentHolding INT NOT NULL,
        BatchID SMALLINT UNSIGNED NOT NULL
    )
    """,
    """
    INSERT INTO FactHoldings (TradeID, CurrentTradeID, SK_CustomerID, SK_AccountID, SK_SecurityID, SK_CompanyID, SK_DateID, SK_TimeID, CurrentPrice, CurrentHolding, BatchID)
    SELECT 
        thh.HH_H_T_ID AS TradeID,
        thh.HH_T_ID AS CurrentTradeID,
        dt.SK_CustomerID,
        dt.SK_AccountID,
        dt.SK_SecurityID,
        dt.SK_CompanyID,
        dt.SK_CloseDateID AS SK_DateID,
        dt.SK_CloseTimeID AS SK_TimeID,
        dt.TradePrice AS CurrentPrice,
        thh.HH_AFTER_QTY AS CurrentHolding,
        1 AS BatchID
    FROM 
        TempHoldingHistory thh
    JOIN 
        DimTrade dt ON thh.HH_T_ID = dt.TradeID
    """,
    "DROP TABLE IF EXISTS TempHoldingHistory"
]

In [125]:
# Executing the queries
with engine.connect() as connection:
    # connection.execute(text("SET GLOBAL local_infile = 1;"))
    for sql in sql_commands:
        connection.execute(text(sql))
    connection.commit()

In [ ]:
fh_end_time = datetime.now()
print(f"fh took {(fh_end_time - fh_start_time).total_seconds()}")

### FactMarketHistory

In [ ]:
fmh_start_time = datetime.now()

In [126]:
dailymarket_df = pd.read_csv(
    DATA_DIR + "DailyMarket.txt",
    sep="|",
    header=None,
    names=[
        "DM_DATE",
        "DM_S_SYMB",
        "DM_CLOSE",
        "DM_HIGH",
        "DM_LOW",
        "DM_VOL",
    ],
    dtype={
        "DM_DATE": "str",
        "DM_S_SYMB": "str",
        "DM_CLOSE": "float32",
        "DM_HIGH": "float32",
        "DM_LOW": "float32",
        "DM_VOL": "int64",
    },
    parse_dates=["DM_DATE"],
)
dailymarket_df["DM_DATE"] = pd.to_datetime(dailymarket_df["DM_DATE"])
dailymarket_df.info()
dailymarket_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2422064 entries, 0 to 2422063
Data columns (total 6 columns):
 #   Column     Dtype         
---  ------     -----         
 0   DM_DATE    datetime64[ns]
 1   DM_S_SYMB  object        
 2   DM_CLOSE   float32       
 3   DM_HIGH    float32       
 4   DM_LOW     float32       
 5   DM_VOL     int64         
dtypes: datetime64[ns](1), float32(3), int64(1), object(1)
memory usage: 83.2+ MB


,DM_DATE,DM_S_SYMB,DM_CLOSE,DM_HIGH,DM_LOW,DM_VOL
0,2015-07-06,AAAAAAAAAAAAERN,242.929993,284.420013,185.080002,111904727
1,2015-07-06,AAAAAAAAAAAAEYJ,445.459991,522.299988,386.480011,78849320
2,2015-07-06,AAAAAAAAAAAAEVC,910.590027,1148.890015,723.369995,807515829
3,2015-07-06,AAAAAAAAAAAACEZ,647.070007,756.679993,473.299988,693226268
4,2015-07-06,AAAAAAAAAAAADOY,385.010010,564.669983,295.630005,34628570


In [127]:
# ClosePrice, DayHigh, DayLow, and Volume are copied from DM_CLOSE, DM_HIGH,
# DM_LOW, and DM_VOL respectively.
dailymarket_df["ClosePrice"] = dailymarket_df["DM_CLOSE"]
dailymarket_df["DayHigh"] = dailymarket_df["DM_HIGH"]
dailymarket_df["DayLow"] = dailymarket_df["DM_LOW"]
dailymarket_df["Volume"] = dailymarket_df["DM_VOL"]

In [128]:
security_info = pd.read_sql("SELECT Symbol AS DM_S_SYMB, SK_SecurityID, SK_CompanyID, EffectiveDate, EndDate FROM dimsecurity", engine)
security_info['EffectiveDate'] = pd.to_datetime(security_info['EffectiveDate'])
security_info.info()
security_info.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   DM_S_SYMB      4000 non-null   object        
 1   SK_SecurityID  4000 non-null   int64         
 2   SK_CompanyID   4000 non-null   int64         
 3   EffectiveDate  4000 non-null   datetime64[ns]
 4   EndDate        4000 non-null   object        
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 156.4+ KB


,DM_S_SYMB,SK_SecurityID,SK_CompanyID,EffectiveDate,EndDate
0,AAAAAAAAAAAAAAA,1,5,1967-04-25,9999-12-31
1,AAAAAAAAAAAAAAB,2,52,1967-04-26,1968-03-30
2,AAAAAAAAAAAAAAC,3,13,1967-04-26,9999-12-31
3,AAAAAAAAAAAAAAD,4,70,1967-04-27,1979-06-27
4,AAAAAAAAAAAAAAE,5,46,1967-04-27,9999-12-31


In [129]:
dailymarket_df = pd.merge(
    dailymarket_df,
    security_info,
    how="left",
    on="DM_S_SYMB",
)
dailymarket_df = dailymarket_df[
    (dailymarket_df["DM_DATE"] >= dailymarket_df["EffectiveDate"])
    & (dailymarket_df["DM_DATE"].dt.date < dailymarket_df["EndDate"])
]
# drop temp columns
dailymarket_df.drop(columns=["EffectiveDate", "EndDate"], inplace=True)
dailymarket_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2422064 entries, 0 to 2567327
Data columns (total 12 columns):
 #   Column         Dtype         
---  ------         -----         
 0   DM_DATE        datetime64[ns]
 1   DM_S_SYMB      object        
 2   DM_CLOSE       float32       
 3   DM_HIGH        float32       
 4   DM_LOW         float32       
 5   DM_VOL         int64         
 6   ClosePrice     float32       
 7   DayHigh        float32       
 8   DayLow         float32       
 9   Volume         int64         
 10  SK_SecurityID  int64         
 11  SK_CompanyID   int64         
dtypes: datetime64[ns](1), float32(6), int64(4), object(1)
memory usage: 184.8+ MB


In [130]:
"""SK_DateID is obtained from DimDate by matching DM_DATE with DateValue to return the
SK_DateID. The match is guaranteed to succeed because DimDate has been populated
with date information for all dates relevant to the benchmark."""
date_info = pd.read_sql("SELECT DateValue, SK_DateID FROM dimdate", engine)
date_info["DateValue"] = pd.to_datetime(date_info["DateValue"])
date_info = date_info.set_index("DateValue")["SK_DateID"].to_dict()
dailymarket_df["SK_DateID"] = dailymarket_df["DM_DATE"].dt.date.map(date_info)

In [131]:
# Step 2: Sort the DataFrame
dailymarket_df.sort_values(by='DM_DATE', inplace=True)

# Step 3 & 4: Group by 'DM_S_SYMB' and apply rolling max
rolling_max = dailymarket_df.groupby('DM_S_SYMB').rolling('365D', on='DM_DATE')['DM_HIGH'].max()

# Reset index to make merging easier
rolling_max = rolling_max.reset_index()

# Step 5: Merge with the original DataFrame
dailymarket_df = dailymarket_df.merge(rolling_max, on=['DM_S_SYMB', 'DM_DATE'], suffixes=('', '_52WeekHigh'))

# Rename the column for clarity
dailymarket_df.rename(columns={'DM_HIGH_52WeekHigh': 'FiftyTwoWeekHigh'}, inplace=True)
dailymarket_df[['DM_DATE', 'DM_S_SYMB', 'DM_HIGH', 'FiftyTwoWeekHigh']].head()

,DM_DATE,DM_S_SYMB,DM_HIGH,FiftyTwoWeekHigh
0,2015-07-06,AAAAAAAAAAAAERN,284.420013,284.420013
1,2015-07-06,AAAAAAAAAAAAEYD,841.200012,841.200012
2,2015-07-06,AAAAAAAAAAAABJB,1055.699951,1055.699951
3,2015-07-06,AAAAAAAAAAAADSX,747.770020,747.770020
4,2015-07-06,AAAAAAAAAAAABYA,708.140015,708.140015


In [132]:
# mostly wrote this myself but dont ask me to explain...........
rolling_rank = (
    dailymarket_df.groupby("DM_S_SYMB")
    .rolling("365D", on="DM_DATE")["DM_HIGH"]
    .rank(method="average", ascending=False)
    .reset_index()
    .rename(columns={"DM_HIGH": "Rank"})
)
rolling_rank["Rank"] = rolling_rank["Rank"].astype("uint32")
# Apply the mask to select DM_DATE only for those rows, then forward fill
mask = rolling_rank['Rank'] == 1
rolling_rank['SK_FiftyTwoWeekHighDate'] = rolling_rank['DM_DATE'].where(mask).ffill()
rolling_rank['SK_FiftyTwoWeekHighDate'] = rolling_rank['SK_FiftyTwoWeekHighDate'].dt.date.map(date_info)
rolling_rank.head()

,DM_S_SYMB,DM_DATE,Rank,SK_FiftyTwoWeekHighDate
0,AAAAAAAAAAAAAAA,2015-07-06,1,20150706
1,AAAAAAAAAAAAAAA,2015-07-07,1,20150707
2,AAAAAAAAAAAAAAA,2015-07-08,2,20150707
3,AAAAAAAAAAAAAAA,2015-07-09,2,20150707
4,AAAAAAAAAAAAAAA,2015-07-10,4,20150707


In [133]:
dailymarket_df = pd.concat([dailymarket_df, rolling_rank['SK_FiftyTwoWeekHighDate']], axis=1)
dailymarket_df.head()

,DM_DATE,DM_S_SYMB,DM_CLOSE,DM_HIGH,DM_LOW,DM_VOL,ClosePrice,DayHigh,DayLow,Volume,SK_SecurityID,SK_CompanyID,SK_DateID,FiftyTwoWeekHigh,SK_FiftyTwoWeekHighDate
0,2015-07-06,AAAAAAAAAAAAERN,242.929993,284.420013,185.080002,111904727,242.929993,284.420013,185.080002,111904727,3523,1275,20150706,284.420013,20150706
1,2015-07-06,AAAAAAAAAAAAEYD,624.479980,841.200012,505.109985,448826960,624.479980,841.200012,505.109985,448826960,3709,1825,20150706,841.200012,20150707
2,2015-07-06,AAAAAAAAAAAABJB,918.270020,1055.699951,808.450012,187580057,918.270020,1055.699951,808.450012,187580057,986,461,20150706,1055.699951,20150707
3,2015-07-06,AAAAAAAAAAAADSX,593.820007,747.770020,593.119995,625016548,593.820007,747.770020,593.119995,625016548,2794,811,20150706,747.770020,20150707
4,2015-07-06,AAAAAAAAAAAABYA,517.380005,708.140015,394.940002,278873866,517.380005,708.140015,394.940002,278873866,1420,229,20150706,708.140015,20150707


In [134]:
dailymarket_df.sort_values(by='DM_DATE', inplace=True)
# Step 3 & 4: Group by 'DM_S_SYMB' and apply rolling min
rolling_min = dailymarket_df.groupby('DM_S_SYMB').rolling('365D', on='DM_DATE')['DM_LOW'].min()
# Reset index to make merging easier
rolling_min = rolling_min.reset_index()
# Step 5: Merge with the original DataFrame
dailymarket_df = dailymarket_df.merge(rolling_min, on=['DM_S_SYMB', 'DM_DATE'], suffixes=('', '_52WeekLow'))
# Rename the column for clarity
dailymarket_df.rename(columns={'DM_LOW_52WeekLow': 'FiftyTwoWeekLow'}, inplace=True)
# dailymarket_df[['DM_DATE', 'DM_S_SYMB', 'DM_LOW', 'FiftyTwoWeekLow']].head()

In [135]:
# same...
rolling_rank = (
    dailymarket_df.groupby("DM_S_SYMB")
    .rolling("365D", on="DM_DATE")["DM_LOW"]
    .rank(method="average", ascending=True)
    .reset_index()
    .rename(columns={"DM_LOW": "Rank"})
)
rolling_rank["Rank"] = rolling_rank["Rank"].astype("uint32")
# Apply the mask to select DM_DATE only for those rows, then forward fill
mask = rolling_rank['Rank'] == 1
rolling_rank['SK_FiftyTwoWeekLowDate'] = rolling_rank['DM_DATE'].where(mask).ffill()
rolling_rank['SK_FiftyTwoWeekLowDate'] = rolling_rank['SK_FiftyTwoWeekLowDate'].dt.date.map(date_info)
# rolling_rank.head()

In [136]:
dailymarket_df = pd.concat([dailymarket_df, rolling_rank['SK_FiftyTwoWeekLowDate']], axis=1)

In [137]:
dailymarket_df['SK_SecurityID'] = dailymarket_df['SK_SecurityID'].astype('uint32')
dailymarket_df['SK_CompanyID'] = dailymarket_df['SK_CompanyID'].astype('uint32')
dailymarket_df['SK_DateID'] = dailymarket_df['SK_DateID'].astype('uint32')
dailymarket_df['FiftyTwoWeekHigh'] = dailymarket_df['FiftyTwoWeekHigh'].astype('float32')
dailymarket_df['SK_FiftyTwoWeekHighDate'] = dailymarket_df['SK_FiftyTwoWeekHighDate'].astype('uint32')
dailymarket_df['FiftyTwoWeekLow'] = dailymarket_df['FiftyTwoWeekLow'].astype('float32')
dailymarket_df['SK_FiftyTwoWeekLowDate'] = dailymarket_df['SK_FiftyTwoWeekLowDate'].astype('uint32')
dailymarket_df['DM_S_SYMB'] = dailymarket_df['DM_S_SYMB'].astype('category')

dailymarket_df.drop(columns=['DM_HIGH', 'DM_LOW', 'DM_VOL', 'DM_CLOSE'], inplace=True)

In [138]:
security_info = pd.read_sql("SELECT Symbol, Dividend, EffectiveDate, EndDate FROM dimsecurity", engine)
security_info['EffectiveDate'] = pd.to_datetime(security_info['EffectiveDate'])

In [139]:
dailymarket_df = dailymarket_df.merge(
    security_info,
    how="left",
    left_on="DM_S_SYMB",
    right_on="Symbol",
)
dailymarket_df = dailymarket_df[
    (dailymarket_df["DM_DATE"] >= dailymarket_df["EffectiveDate"])
    & (dailymarket_df["DM_DATE"].dt.date < dailymarket_df["EndDate"])
]
dailymarket_df.drop(columns=["Symbol", "EffectiveDate", "EndDate"], inplace=True)

In [140]:
dailymarket_df['Yield'] = dailymarket_df['Dividend'] / dailymarket_df['ClosePrice'] * 100
dailymarket_df.drop(columns=["Dividend"], inplace=True)
dailymarket_df['BatchID'] = 1

In [141]:
sql_dtypes = {
    "SK_SecurityID": sqlalchemy.types.Integer,
    "SK_CompanyID": sqlalchemy.types.Integer,
    "SK_DateID": sqlalchemy.types.Integer,
    # PERatio to be done in MySQL
    # "PERatio": sqlalchemy.types.DECIMAL(precision=10, scale=2),
    "Yield": sqlalchemy.types.DECIMAL(precision=5, scale=2),
    "FiftyTwoWeekHigh": sqlalchemy.types.DECIMAL(precision=8, scale=2),
    "SK_FiftyTwoWeekHighDate": sqlalchemy.types.Integer,
    "FiftyTwoWeekLow": sqlalchemy.types.DECIMAL(precision=8, scale=2),
    "SK_FiftyTwoWeekLowDate": sqlalchemy.types.Integer,
    "ClosePrice": sqlalchemy.types.DECIMAL(precision=8, scale=2),
    "DayHigh": sqlalchemy.types.DECIMAL(precision=8, scale=2),
    "DayLow": sqlalchemy.types.DECIMAL(precision=8, scale=2),
    "Volume": sqlalchemy.types.BigInteger,
    "BatchID": sqlalchemy.types.SmallInteger,
    "DM_DATE": sqlalchemy.types.Date,
    "DM_S_SYMB": sqlalchemy.types.CHAR(16),
}

In [142]:
dailymarket_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2422064 entries, 0 to 2567327
Data columns (total 15 columns):
 #   Column                   Dtype         
---  ------                   -----         
 0   DM_DATE                  datetime64[ns]
 1   DM_S_SYMB                object        
 2   ClosePrice               float32       
 3   DayHigh                  float32       
 4   DayLow                   float32       
 5   Volume                   int64         
 6   SK_SecurityID            uint32        
 7   SK_CompanyID             uint32        
 8   SK_DateID                uint32        
 9   FiftyTwoWeekHigh         float32       
 10  SK_FiftyTwoWeekHighDate  uint32        
 11  FiftyTwoWeekLow          float32       
 12  SK_FiftyTwoWeekLowDate   uint32        
 13  Yield                    float64       
 14  BatchID                  int64         
dtypes: datetime64[ns](1), float32(5), float64(1), int64(2), object(1), uint32(5)
memory usage: 203.3+ MB


In [143]:
for i in trange(0, dailymarket_df.shape[0], 100000):
    dailymarket_df.iloc[i:i+100000].to_sql('tempfactmarketprice', engine, if_exists='append', index=False,
                                           dtype=sql_dtypes)

  0%|          | 0/25 [00:00<?, ?it/s]

PendingRollbackError: Can't reconnect until invalid transaction is rolled back.  Please rollback() fully before proceeding (Background on this error at: https://sqlalche.me/e/20/8s2b)

In [ ]:
sql_commands = [
    "CREATE INDEX idx_sk_companyid ON tempfactmarketprice(SK_CompanyID);",
    """CREATE TABLE FactMarketHistory (
        SK_SecurityID INT UNSIGNED NOT NULL,
        SK_CompanyID INT UNSIGNED NOT NULL,
        SK_DateID INT UNSIGNED NOT NULL,
        PERatio DECIMAL(10, 2),
        Yield DECIMAL(5, 2) NOT NULL,
        FiftyTwoWeekHigh DECIMAL(8, 2) NOT NULL,
        SK_FiftyTwoWeekHighDate INT UNSIGNED NOT NULL,
        FiftyTwoWeekLow DECIMAL(8, 2) NOT NULL,
        SK_FiftyTwoWeekLowDate INT UNSIGNED NOT NULL,
        ClosePrice DECIMAL(8, 2) NOT NULL,
        DayHigh DECIMAL(8, 2) NOT NULL,
        DayLow DECIMAL(8, 2) NOT NULL,
        Volume BIGINT UNSIGNED NOT NULL,
        BatchID SMALLINT UNSIGNED NOT NULL
    );""",
    "ALTER TABLE factmarkethistory ADD COLUMN DM_S_SYMB TEXT;",
    """INSERT INTO factmarkethistory
        SELECT SK_SecurityID, fmp.SK_CompanyID, SK_DateID, fmp.ClosePrice / T.Sum_EPS AS PERatio, Yield, FiftyTwoWeekHigh,
        SK_FiftyTwoWeekHighDate, FiftyTwoWeekLow, SK_FiftyTwoWeekLowDate, ClosePrice, DayHigh, DayLow, Volume, BatchID, DM_S_SYMB
        FROM tempfactmarketprice fmp
        LEFT JOIN (SELECT 
            c.CompanyID, 
            c.SK_CompanyID AS SKCID, 
            f.FI_QTR_START_DATE,
            SUM(f.FI_BASIC_EPS) OVER (
                PARTITION BY c.CompanyID 
                ORDER BY f.FI_QTR_START_DATE 
                ROWS BETWEEN 3 PRECEDING AND CURRENT ROW
            ) AS Sum_EPS
        FROM financial f RIGHT JOIN dimCompany c ON f.SK_CompanyID = c.SK_CompanyID
        ORDER BY c.CompanyID, f.FI_QTR_START_DATE) T
        ON T.SKCID = fmp.SK_CompanyID
        AND T.FI_QTR_START_DATE < fmp.DM_DATE 
        AND T.FI_QTR_START_DATE >= DATE_SUB(fmp.DM_DATE, INTERVAL 3 MONTH);""",
    """INSERT INTO dimessages
        SELECT NOW() AS MessageDateAndTime, 1 AS BATCHID, 'FactMarketHistory' AS MessageSource, 'No earnings for company' AS MessageText,
        'Alert' AS MessageType, CONCAT('DM_S_SYMB = ', DM_S_SYMB)
        FROM factmarkethistory 
        WHERE PERatio IS NULL;""",
    "ALTER TABLE factmarkethistory DROP COLUMN DM_S_SYMB;",
    "DROP TABLE tempfactmarketprice;"
]

In [ ]:
# Executing the queries
with engine.connect() as connection:
    for sql in sql_commands:
        connection.execute(text(sql))

In [ ]:
fmh_end_time = datetime.now()
print(f"fmh took {(fmh_end_time - fmh_start_time).total_seconds()}")

### FactWatches

In [ ]:
fw_start_time = datetime.now()

In [ ]:
df = pd.read_csv(
    DATA_DIR + "WatchHistory.txt",
    sep="|",
    header=None,
    names=[
        "W_C_ID",
        "W_S_SYMB",
        "W_DTS",
        "W_ACTION"
    ],
    dtype={
        "W_C_ID": "uint32",
        "W_S_SYMB": "str",
        "W_DTS": "str",
        "W_ACTION": "str"
    },
    parse_dates=["W_DTS"]
)
df.info()
df.head()

In [ ]:
customer_info = pd.read_sql_query("SELECT CustomerID, SK_CustomerID, EffectiveDate, EndDate FROM dimcustomer", engine)
customer_info['EffectiveDate'] = pd.to_datetime(customer_info['EffectiveDate'])
security_info = pd.read_sql_query("SELECT Symbol, SK_SecurityID, EffectiveDate, EndDate FROM dimsecurity", engine)
security_info['EffectiveDate'] = pd.to_datetime(security_info['EffectiveDate'])
date_info = pd.read_sql_query("SELECT DateValue, SK_DateID FROM dimdate", engine)
date_info['DateValue'] = pd.to_datetime(date_info['DateValue'])

In [ ]:
# get SK_CustomerID
df = df.merge(customer_info, how="left", left_on="W_C_ID", right_on="CustomerID")
# filter based on date
df = df[
    (df["W_DTS"] >= df["EffectiveDate"])
    & (df["W_DTS"].dt.date < df["EndDate"])
]
# drop cols
df.drop(columns=["CustomerID", "EffectiveDate", "EndDate"], inplace=True)
# get SK_SecurityID
df = df.merge(security_info, how="left", left_on="W_S_SYMB", right_on="Symbol")
# filter based on date
df = df[
    (df["W_DTS"] >= df["EffectiveDate"])
    & (df["W_DTS"].dt.date < df["EndDate"])
]
# drop cols
df.drop(columns=["Symbol", "EffectiveDate", "EndDate"], inplace=True)
# SK_DateID_DatePlaced - set based on W_DTS.
df['SK_DateID_DatePlaced'] = df['W_DTS'].dt.date.map(date_info.set_index('DateValue')['SK_DateID'])
# BatchID - set to 1.
df['BatchID'] = 1
df.info()
df.head()

In [ ]:
# Mask for rows where W_ACTION is 'CNCL'
mask_cncl = df['W_ACTION'] == 'CNCL'
df.loc[mask_cncl, 'SK_DateID_DateRemoved'] = df.loc[mask_cncl, 'W_DTS'].dt.date.map(date_info.set_index('DateValue')['SK_DateID'])
df.loc[~mask_cncl, 'SK_DateID_DateRemoved'] = None

df.info()
df.head()

In [ ]:
keep_cols = ["SK_CustomerID", "SK_SecurityID", "SK_DateID_DatePlaced", "SK_DateID_DateRemoved", "BatchID"]
df = df.groupby(["W_C_ID", "W_S_SYMB"]).first().reset_index()[keep_cols]
df.info()

In [ ]:
sql_dtypes = {
    "SK_CustomerID": sqlalchemy.types.Integer,
    "SK_SecurityID": sqlalchemy.types.Integer,
    "SK_DateID_DatePlaced": sqlalchemy.types.Integer,
    "SK_DateID_DateRemoved": sqlalchemy.types.Integer,
    "BatchID": sqlalchemy.types.SmallInteger
}

In [ ]:
create_table = """CREATE TABLE FactWatches (
    SK_CustomerID INT UNSIGNED NOT NULL,
    SK_SecurityID INT UNSIGNED NOT NULL,
    SK_DateID_DatePlaced INT UNSIGNED NOT NULL,
    SK_DateID_DateRemoved INT UNSIGNED,
    BatchID SMALLINT UNSIGNED NOT NULL
);"""

with engine.connect() as connection:
    connection.execute(text(create_table))

In [ ]:
for i in trange(0, df.shape[0], 100000):
    df.iloc[i:i+100000].to_sql('factwatches', engine, if_exists='append', index=False, dtype=sql_dtypes)

In [ ]:
fw_end_time = datetime.now()
print(f"Prospect took {(fw_end_time - fw_start_time).total_seconds()}")

### Batch Validation

In [ ]:
fk_start_time = datetime.now()

In [ ]:
filepath = r"fk.sql"
# Read the SQL file
with open(filepath, 'r') as file:
    sql_file = file.read()
queries = sql_file.split(";")

In [ ]:
# Execute the SQL commands
with engine.connect() as connection:
    for query in tqdm(queries):
        query = query.strip() + ";"
        if len(query) < 5:
            continue
        connection.execute(text(query))

In [ ]:
fk_end_time = datetime.now()
print(f"fk took {(fk_end_time - fk_start_time).total_seconds()}")

In [ ]:
filepath = r"..\validation\tpcdi_validation.sql"
# Read the SQL file
with open(filepath, 'r') as file:
    sql_file = file.read()

# Execute the SQL commands
with engine.connect() as connection:
    for query in sql_file.split(";"):
        query = query.strip() + ";"
        if len(query) < 5:
            continue
        connection.execute(text(query))

In [ ]:
end_time = datetime.now()
print(f"Historical load took {(end_time - start_time).total_seconds()}")

## Incremental Load

In [ ]:
# query = """SELECT concat('ALTER TABLE `', TABLE_NAME, '` DROP FOREIGN KEY `', CONSTRAINT_NAME, '`;') 
# FROM information_schema.key_column_usage 
# WHERE CONSTRAINT_SCHEMA = 'tpcdi_sf5' 
# AND referenced_table_name IS NOT NULL;"""
# result = pd.read_sql_query(query, engine).iloc[:, 0]
# for value in result.values:
#     with engine.connect() as cnxn:
#         cnxn.execute(text(value))
#         cnxn.commit()

In [ ]:
BATCH_ID = 2
DATA_DIR = f"..\\data\\sf5\\Batch{BATCH_ID}\\"

In [ ]:
with open(DATA_DIR + "BatchDate.txt") as f:
    BATCH_DATE = f.read().strip()
BATCH_DATE = pd.to_datetime(BATCH_DATE)
BATCH_DATE

### dimCustomer & Prospect

#### dimCustomer

In [ ]:
# Define column names
column_names = [
    "CDC_FLAG", "CDC_DSN", "C_ID", "C_TAX_ID", "C_ST_ID",
    "C_L_NAME", "C_F_NAME", "C_M_NAME", "C_GNDR", "C_TIER",
    "C_DOB", "C_ADLINE1", "C_ADLINE2", "C_ZIPCODE", "C_CITY",
    "C_STATE_PROV", "C_CTRY", "C_CTRY_1", "C_AREA_1", "C_LOCAL_1",
    "C_EXT_1", "C_CTRY_2", "C_AREA_2", "C_LOCAL_2", "C_EXT_2",
    "C_CTRY_3", "C_AREA_3", "C_LOCAL_3", "C_EXT_3", "C_EMAIL_1",
    "C_EMAIL_2", "C_LCL_TX_ID", "C_NAT_TX_ID"
]

In [ ]:
# Define data types
data_types = {
    "CDC_FLAG": "category",
    "CDC_DSN": "int64",
    "C_ID": "int64",
    "C_TAX_ID": "str",
    "C_ST_ID": "category",
    "C_L_NAME": "str",
    "C_F_NAME": "str",
    "C_M_NAME": "str",
    "C_GNDR": "category",
    "C_TIER": "int64",
    "C_DOB": "str",
    "C_ADLINE1": "str",
    "C_ADLINE2": "str",
    "C_ZIPCODE": "str",
    "C_CITY": "str",
    "C_STATE_PROV": "str",
    "C_CTRY": "str",
    "C_CTRY_1": "str",
    "C_AREA_1": "str",
    "C_LOCAL_1": "str",
    "C_EXT_1": "str",
    "C_CTRY_2": "str",
    "C_AREA_2": "str",
    "C_LOCAL_2": "str",
    "C_EXT_2": "str",
    "C_CTRY_3": "str",
    "C_AREA_3": "str",
    "C_LOCAL_3": "str",
    "C_EXT_3": "str",
    "C_EMAIL_1": "str",
    "C_EMAIL_2": "str",
    "C_LCL_TX_ID": "str",
    "C_NAT_TX_ID": "str"
}

In [ ]:
# Read the file
file_path = DATA_DIR + "Customer.txt"
df = pd.read_csv(file_path, sep='|', header=None, names=column_names, dtype=data_types, parse_dates=["C_DOB"])
df.info()

In [ ]:
# Rename columns
df.rename(columns={
    "C_ID": "CustomerID",
    "C_TAX_ID": "TaxID",
    "C_L_NAME": "LastName",
    "C_F_NAME": "FirstName",
    "C_M_NAME": "MiddleInitial",
    "C_TIER": "Tier",
    "C_DOB": "DOB",
    "C_EMAIL_1": "Email1",
    "C_EMAIL_2": "Email2"
}, inplace=True)

In [ ]:
df['Gender'] = df['C_GNDR'].str.upper()
df.loc[~df['Gender'].isin(['M', 'F']), 'Gender'] = 'U'

In [ ]:
df.rename(columns={
    "C_ADLINE1": "AddressLine1",
    "C_ADLINE2": "AddressLine2",
    "C_ZIPCODE": "PostalCode",
    "C_CITY": "City",
    "C_STATE_PROV": "StateProv",
    "C_CTRY": "Country"
}, inplace=True)

In [ ]:
# Status is copied from ST_NAME of the StatusType table by matching C_ST_ID with ST_ID of the StatusType table.
status_mapping = pd.read_sql_query("SELECT ST_ID AS C_ST_ID, ST_NAME from StatusType", engine).set_index("C_ST_ID")["ST_NAME"].to_dict()
df.loc[:, "Status"] = df.loc[:, "C_ST_ID"].map(status_mapping)

In [ ]:
def format_phone_number(row, i):
    # Extract components of the phone number
    ctry_code = row[f"C_CTRY_{i}"]
    area_code = row[f"C_AREA_{i}"]
    local = row[f"C_LOCAL_{i}"]
    ext = row[f"C_EXT_{i}"]        

    if pd.isna(ctry_code):
        ctry_code = None
    if pd.isna(area_code):
        area_code = None
    if pd.isna(local):
        local = None
    if pd.isna(ext):
        ext = None

    # Apply transformation rules
    if ctry_code and area_code and local:
        phone = f"+{ctry_code} ({area_code}) {local}"
    elif area_code and local:
        phone = f"({area_code}) {local}"
    elif local:
        phone = local
    else:
        return None

    # Add extension if present
    if ext:
        phone += f"{ext}"

    return phone

In [ ]:
for i in range(1, 4):
    df[f'Phone{i}'] = df.apply(format_phone_number, axis=1, args=(i, ))

In [ ]:
tax_info = pd.read_sql_query("SELECT TX_ID, TX_NAME, TX_RATE FROM TaxRate", engine).set_index('TX_ID')
tax_name_mapping = tax_info['TX_NAME'].to_dict()
tax_rate_mapping = tax_info['TX_RATE'].to_dict()

# NationalTaxRateDesc and NationalTaxRate are copied from TX_NAME and TX_RATE respectively by matching C_NAT_TX_ID with TX_ID.
df.loc[:, 'NationalTaxRateDesc'] = df.loc[:, 'C_NAT_TX_ID'].map(tax_name_mapping)
df.loc[:, 'NationalTaxRate'] = df.loc[:, 'C_NAT_TX_ID'].map(tax_rate_mapping)

# LocalTaxRateDesc and LocalTaxRate are copied from TX_NAME and TX_RATE respectively by matching C_LCL_TX_ID with TX_ID.
df.loc[:, 'LocalTaxRateDesc'] = df.loc[:, 'C_LCL_TX_ID'].map(tax_name_mapping)
df.loc[:, 'LocalTaxRate'] = df.loc[:, 'C_LCL_TX_ID'].map(tax_rate_mapping)

In [ ]:
df.loc[:, 'IsCurrent'] = 1
df.loc[:, 'EffectiveDate'] = pd.to_datetime(BATCH_DATE)
df.loc[:, 'EndDate'] = pd.Timestamp("9999-12-31")
df.loc[:, 'BatchID'] = BATCH_ID

#### Prospect

In [ ]:
def read_prospect_file(filepath):
    # Define the column names and their data types
    columns = [
        'AgencyID', 'LastName', 'FirstName', 'MiddleInitial', 'Gender', 
        'AddressLine1', 'AddressLine2', 'PostalCode', 'City', 'State', 
        'Country', 'Phone', 'Income', 'NumberCars', 'NumberChildren', 
        'MaritalStatus', 'Age', 'CreditRating', 'OwnOrRentFlag', 
        'Employer', 'NumberCreditCards', 'NetWorth'
    ]

    # Define the data types for reading the file
    dtypes = {
        'AgencyID': 'str', 'LastName': 'str', 'FirstName': 'str', 
        'MiddleInitial': 'str', 'Gender': 'str', 'AddressLine1': 'str', 
        'AddressLine2': 'str', 'PostalCode': 'str', 'City': 'str', 
        'State': 'str', 'Country': 'str', 'Phone': 'str', 
        'Income': 'Int64', 'NumberCars': 'Int8', 'NumberChildren': 'Int8', 
        'MaritalStatus': 'str', 'Age': 'Int8', 'CreditRating': 'Int16', 
        'OwnOrRentFlag': 'str', 'Employer': 'str', 
        'NumberCreditCards': 'Int8', 'NetWorth': 'Int64'
    }

    # Read the CSV file
    raw_prospect_df = pd.read_csv(
        filepath, 
        header=None, 
        names=columns, 
        dtype=dtypes
    )

    return raw_prospect_df

In [ ]:
raw_prospect_df = read_prospect_file(DATA_DIR + "Prospect.csv")
raw_prospect_df.info()
raw_prospect_df.head()

In [ ]:
not_null_cols = ['LastName', 'FirstName', 'City', 'State']
for col in not_null_cols:
    raw_prospect_df.loc[raw_prospect_df[col].isna(), col] = ''
raw_prospect_df.info()

In [ ]:
dtypes = {
    'AgencyID': 'str',
    'SK_RecordDateID': 'uint32',
    'SK_UpdateDateID': 'uint32',
    'BatchID': 'uint16',
    'IsCustomer': 'boolean',
    'LastName': 'str',
    'FirstName': 'str',
    'MiddleInitial': 'str',
    'Gender': 'str',
    'AddressLine1': 'str',
    'AddressLine2': 'str',
    'PostalCode': 'str',
    'City': 'str',
    'State': 'str',
    'Country': 'str',
    'Phone': 'str',
    'Income': 'uint32',
    'NumberCars': 'uint8',
    'NumberChildren': 'uint8',
    'MaritalStatus': 'str',
    'Age': 'uint8',
    'CreditRating': 'uint16',
    'OwnOrRentFlag': 'str',
    'Employer': 'str',
    'NumberCreditCards': 'uint8',
    'NetWorth': 'int64',
    'MarketingNameplate': 'str'
}

# Create an empty DataFrame with the specified schema
prospect_df = pd.DataFrame({col: pd.Series(dtype=typ) for col, typ in dtypes.items()})

In [ ]:
prospect_df["AgencyID"] = raw_prospect_df["AgencyID"]
prospect_df["LastName"] = raw_prospect_df["LastName"]
prospect_df["FirstName"] = raw_prospect_df["FirstName"]
prospect_df["MiddleInitial"] = raw_prospect_df["MiddleInitial"]
prospect_df["Gender"] = raw_prospect_df["Gender"]
# fix data quality issues
prospect_df['Gender'] = prospect_df['Gender'].str.upper()
mask = ~prospect_df['Gender'].isin(["M", "F"])
prospect_df.loc[mask, "Gender"] = "U"
prospect_df["AddressLine1"] = raw_prospect_df["AddressLine1"]
prospect_df["AddressLine2"] = raw_prospect_df["AddressLine2"]
prospect_df["PostalCode"] = raw_prospect_df["PostalCode"]
prospect_df["City"] = raw_prospect_df["City"]
prospect_df["State"] = raw_prospect_df["State"]
prospect_df["Country"] = raw_prospect_df["Country"]
prospect_df["Phone"] = raw_prospect_df["Phone"]
prospect_df["Income"] = raw_prospect_df["Income"]
prospect_df["NumberCars"] = raw_prospect_df["NumberCars"]
prospect_df["NumberChildren"] = raw_prospect_df["NumberChildren"]
prospect_df["MaritalStatus"] = raw_prospect_df["MaritalStatus"]
prospect_df["MaritalStatus"] = prospect_df["MaritalStatus"].str.upper()
mask = ~prospect_df["MaritalStatus"].isin(["S", "M", "D", "W"])
prospect_df.loc[mask, "MaritalStatus"] = "U"
prospect_df["Age"] = raw_prospect_df["Age"]
prospect_df["CreditRating"] = raw_prospect_df["CreditRating"]
prospect_df["OwnOrRentFlag"] = raw_prospect_df["OwnOrRentFlag"]
prospect_df["OwnOrRentFlag"] = prospect_df["OwnOrRentFlag"].str.upper()
mask = ~prospect_df["OwnOrRentFlag"].isin(["O", "R"])
prospect_df.loc[mask, "OwnOrRentFlag"] = "U"
prospect_df["Employer"] = raw_prospect_df["Employer"]
prospect_df["NumberCreditCards"] = raw_prospect_df["NumberCreditCards"]
prospect_df["NetWorth"] = raw_prospect_df["NetWorth"]

In [ ]:
sk_dateid = pd.read_sql_query(f"SELECT SK_DateID FROM DimDate where DateValue = '{BATCH_DATE}'", engine).iloc[0, 0]

In [ ]:
# Load old data from MySQL
old_prospect_df = pd.read_sql('SELECT * FROM Prospect', engine)

# Merge new and old data on AgencyID
merged_df = pd.merge(prospect_df, old_prospect_df, on='AgencyID', suffixes=('_new', '_old'), how='left')

# List of fields to compare
fields_to_compare = ['LastName', 'FirstName', 'MiddleInitial', 'Gender', 'AddressLine1', 'AddressLine2',
                     'PostalCode', 'City', 'State', 'Country', 'Phone', 'Income', 'NumberCars',
                     'NumberChildren', 'MaritalStatus', 'Age', 'CreditRating', 'OwnOrRentFlag',
                     'Employer', 'NumberCreditCards', 'NetWorth']

# Identify changed records
is_changed = merged_df[[f + '_new' for f in fields_to_compare]] != merged_df[[f + '_old' for f in fields_to_compare]].values
has_changed = is_changed.any(axis=1)
# only update the records if the left join succeeded
same_agency_id = ~merged_df['SK_UpdateDateID_old'].isna()

# Update SK_UpdateDateID for changed records
merged_df.loc[(has_changed) & (same_agency_id), 'SK_UpdateDateID_new'] = sk_dateid
# the old ones keep the old SK_UpdateDateID
merged_df.loc[(~has_changed) & (same_agency_id), 'SK_UpdateDateID_new'] = merged_df.loc[(~has_changed) & (same_agency_id), 'SK_UpdateDateID_old']

# For new records (those that are NaN in old data), set SK_UpdateDateID to batch_date_sk
is_new_record = merged_df['SK_UpdateDateID_old'].isna()
merged_df.loc[is_new_record, 'SK_UpdateDateID_new'] = sk_dateid

# Finalize the DataFrame with updated SK_UpdateDateID
prospect_df_updated = merged_df[['AgencyID'] + [f + '_new' for f in fields_to_compare] + ['SK_UpdateDateID_new']]

# rename cols
cols = prospect_df_updated.columns.tolist()
cols = {col: col.replace("_new", "") for col in cols}
prospect_df_updated.rename(columns=cols, inplace=1)

In [ ]:
# SK_RecordDateID is set to the DimDate SK_DateID field that corresponds to the Batch Date.
prospect_df_updated['SK_RecordDateID'] = sk_dateid
prospect_df_updated['SK_RecordDateID'] = prospect_df_updated['SK_RecordDateID'].astype('uint32')

prospect_df_updated.info()

In [ ]:
# Define conditions for each tag with null checks
conditions = {
    "HighValue": (prospect_df_updated["NetWorth"].notnull() & prospect_df_updated["Income"].notnull())
    & ((prospect_df_updated["NetWorth"] > 1_000_000) | (prospect_df_updated["Income"] > 200_000)),
    "Expenses": (
        prospect_df_updated["NumberChildren"].notnull()
        & prospect_df_updated["NumberCreditCards"].notnull()
    )
    & ((prospect_df_updated["NumberChildren"] > 3) | (prospect_df_updated["NumberCreditCards"] > 5)),
    "Boomer": prospect_df_updated["Age"].notnull() & (prospect_df_updated["Age"] > 45),
    "MoneyAlert": (
        prospect_df_updated["Income"].notnull()
        & prospect_df_updated["CreditRating"].notnull()
        & prospect_df_updated["NetWorth"].notnull()
    )
    & (
        (prospect_df_updated["Income"] < 50_000)
        | (prospect_df_updated["CreditRating"] < 600)
        | (prospect_df_updated["NetWorth"] < 100_000)
    ),
    "Spender": (
        prospect_df_updated["NumberCars"].notnull() & prospect_df_updated["NumberCreditCards"].notnull()
    )
    & ((prospect_df_updated["NumberCars"] > 3) | (prospect_df_updated["NumberCreditCards"] > 7)),
    "Inherited": (prospect_df_updated["Age"].notnull() & prospect_df_updated["NetWorth"].notnull())
    & ((prospect_df_updated["Age"] < 25) & (prospect_df_updated["NetWorth"] > 1_000_000)),
}

# Apply conditions to assign tags
prospect_df_updated["MarketingNameplate"] = ""
for tag, condition in conditions.items():
    prospect_df_updated["MarketingNameplate"] += np.where(condition, tag + "+", "")

# Remove trailing '+' and replace empty strings with None
prospect_df_updated["MarketingNameplate"] = (
    prospect_df_updated["MarketingNameplate"].str.rstrip("+").replace("", None)
)

In [ ]:
prospect_df_updated.info()

In [ ]:
prospect_df_updated['BatchID'] = BATCH_ID

#### Update Both

In [ ]:
# temporary prospect_df for matching
prospect_df_temp = prospect_df_updated[
    [
        "AgencyID",
        "CreditRating",
        "NetWorth",
        "MarketingNameplate",
        "LastName",
        "FirstName",
        "AddressLine1",
        "AddressLine2",
        "PostalCode",
    ]
].copy()
prospect_df_temp["LastName"] = prospect_df_temp["LastName"].str.upper()
prospect_df_temp["FirstName"] = prospect_df_temp["FirstName"].str.upper()
prospect_df_temp["AddressLine1"] = prospect_df_temp["AddressLine1"].str.upper()
prospect_df_temp["AddressLine2"] = prospect_df_temp["AddressLine2"].str.upper()
prospect_df_temp["PostalCode"] = prospect_df_temp["PostalCode"].str.upper()

In [ ]:
for index, row in df.iterrows():
    # first check previous batch
    first_name, last_name = row['FirstName'].upper(), row["LastName"].upper()
    address1, address2 = row['AddressLine1'], row["AddressLine2"]
    postcode = row['PostalCode']
    if not pd.isna(address1):
        address1 = address1.upper()
    if not pd.isna(address2):
        address2 = address1.upper()
    if not pd.isna(postcode):
        postcode = postcode.upper()
    
    query = f"""SELECT AgencyID, CreditRating, NetWorth, MarketingNameplate FROM Prospect
    WHERE UPPER(FirstName) = '{first_name}' AND UPPER(LastName) = '{last_name}'"""
    if pd.isna(row['AddressLine1']):
        query += " AND AddressLine1 IS NULL"
    else:
        query += f" AND UPPER(AddressLine1) = '{address1}'"
    if pd.isna(row['AddressLine2']):
        query += " AND AddressLine2 IS NULL"
    else:
        query += f" AND UPPER(AddressLine2) = '{address2}'"
    if pd.isna(row['PostalCode']):
        query += " AND PostalCode IS NULL;"
    else:
        query += f" AND UPPER(PostalCode) = '{postcode}';"

    result = pd.read_sql_query(query, engine)
    if len(result) > 0:
        df.loc[index, "AgencyID"] = result.iloc[0, 0]
        df.loc[index, "CreditRating"] = result.iloc[0, 1]
        df.loc[index, "NetWorth"] = result.iloc[0, 2]
        df.loc[index, "MarketingNameplate"] = result.iloc[0, 3]
    else:
        # check current batch
        match = prospect_df_temp[
            (prospect_df_temp["LastName"] == last_name)
            & (prospect_df_temp["FirstName"] == first_name)
            & (prospect_df_temp["AddressLine1"] == address1)
            & (prospect_df_temp["AddressLine2"] == address2)
            & (prospect_df_temp["PostalCode"] == postcode)
        ]
        if not match.empty:
            df.loc[index, "AgencyID"] = match["AgencyID"].iloc[-1]
            df.loc[index, "CreditRating"] = match["CreditRating"].iloc[-1]
            df.loc[index, "NetWorth"] = match["NetWorth"].iloc[-1]
            df.loc[index, "MarketingNameplate"] = match["MarketingNameplate"].iloc[-1]
        else:
            # set empty
            df.loc[index, "AgencyID"] = None
            df.loc[index, "CreditRating"] = None
            df.loc[index, "NetWorth"] = None
            df.loc[index, "MarketingNameplate"] = None

In [ ]:
max_sk = pd.read_sql_query('SELECT MAX(SK_CustomerID) FROM dimCustomer', engine).iloc[0, 0]
df.loc[:, 'SK_CustomerID'] = range(max_sk + 1, max_sk + 1 + len(df))

A record will be reported in the DImessages table if a customer’s Tier is not one of the valid
values (1,2,3). The MessageSource is “DimCustomer”, the MessageType is “Alert” and the
MessageText is “Invalid customer tier”. The MessageData field is “C_ID = ” followed by the
key value of the record, then “, C_TIER = ” and the C_TIER value.

In [ ]:
invalid_tiers = df[~df['Tier'].isin([1,2,3])][["CustomerID", "Tier"]]
if len(invalid_tiers) > 0:
    invalid_tiers.loc[:, "MessageDateAndTime"] = datetime.now()
    invalid_tiers['BatchID'] = BATCH_ID
    invalid_tiers['MessageSource'] = 'DimCustomer'
    invalid_tiers['MessageText'] = 'Invalid customer tier'
    invalid_tiers['MessageType'] = 'Alert'
    invalid_tiers['MessageData'] = "C_ID = " + invalid_tiers["CustomerID"].astype(str) + ", C_TIER = " + invalid_tiers["Tier"].astype(str)
    
    sql_dtypes = {
        "MessageDateAndTime": sqlalchemy.types.DATETIME,
        "BatchID": sqlalchemy.types.Integer,
        "MessageSource": sqlalchemy.types.CHAR(30),
        "MessageText": sqlalchemy.types.CHAR(50),
        "MessageType": sqlalchemy.types.CHAR(12),
        "MessageData": sqlalchemy.types.CHAR(100)
    }
    invalid_tiers.to_sql('dimessages', con=engine, if_exists='append', index=False, dtype=sql_dtypes)

A record will be reported in the DImessages table if a customer’s DOB is invalid. A customer’s
DOB is invalid if DOB < Batch Date – 100 years or DOB > Batch Date (customer over 100 years
old or born in the future). The MessageSource is “DimCustomer”, the MessageType is “Alert”
and the MessageText is “DOB out of range”. The MessageData field is “C_ID = ” followed by
the key value of the record, then “, C_DOB = ” and the C_DOB value.

In [ ]:
invalid_dobs = df[(df['DOB'] < BATCH_DATE - pd.Timedelta(days=100*365)) | (df['DOB'] > BATCH_DATE + pd.Timedelta(days=100*365))]
if len(invalid_dobs) > 0:
    invalid_dobs.loc[:, "MessageDateAndTime"] = datetime.now()
    invalid_dobs['BatchID'] = BATCH_ID
    invalid_dobs['MessageSource'] = 'DimCustomer'
    invalid_dobs['MessageText'] = 'DOB out of range'
    invalid_dobs['MessageType'] = 'Alert'
    invalid_dobs['MessageData'] = "C_ID = " + invalid_tiers["CustomerID"].astype(str) + ", C_DOB = " + invalid_tiers["DOB"].astype(str)
    
    sql_dtypes = {
        "MessageDateAndTime": sqlalchemy.types.DATETIME,
        "BatchID": sqlalchemy.types.Integer,
        "MessageSource": sqlalchemy.types.CHAR(30),
        "MessageText": sqlalchemy.types.CHAR(50),
        "MessageType": sqlalchemy.types.CHAR(12),
        "MessageData": sqlalchemy.types.CHAR(100)
    }
    invalid_dobs.to_sql('dimessages', con=engine, if_exists='append', index=False, dtype=sql_dtypes)

In [ ]:
insert_customers_mask = df['CDC_FLAG'] == 'I'
keep_cols = [
    "SK_CustomerID", "CustomerID", "TaxID", "Status", "LastName", 
    "FirstName", "MiddleInitial", "Gender", "Tier", "DOB", 
    "AddressLine1", "AddressLine2", "PostalCode", "City", "StateProv", 
    "Country", "Phone1", "Phone2", "Phone3", "Email1", 
    "Email2", "NationalTaxRateDesc", "NationalTaxRate", "LocalTaxRateDesc", 
    "LocalTaxRate", "AgencyID", "CreditRating", "NetWorth", "MarketingNameplate", 
    "IsCurrent", "BatchID", "EffectiveDate", "EndDate"
]
df_insert = df.loc[insert_customers_mask, keep_cols]

In [ ]:
sql_dtypes = {
    'SK_CustomerID': sqlalchemy.types.Integer,
    'CustomerID': sqlalchemy.types.Integer,
    'TaxID': sqlalchemy.types.String(20),
    'Status': sqlalchemy.types.String(10),
    'LastName': sqlalchemy.types.String(30),
    'FirstName': sqlalchemy.types.String(30),
    'MiddleInitial': sqlalchemy.types.String(1),
    'Gender': sqlalchemy.types.String(1),
    'Tier': sqlalchemy.types.SmallInteger,
    'DOB': sqlalchemy.types.Date,
    'AddressLine1': sqlalchemy.types.String(80),
    'AddressLine2': sqlalchemy.types.String(80),
    'PostalCode': sqlalchemy.types.String(12),
    'City': sqlalchemy.types.String(25),
    'StateProv': sqlalchemy.types.String(20),
    'Country': sqlalchemy.types.String(24),
    'Phone1': sqlalchemy.types.String(30),
    'Phone2': sqlalchemy.types.String(30),
    'Phone3': sqlalchemy.types.String(30),
    'Email1': sqlalchemy.types.String(50),
    'Email2': sqlalchemy.types.String(50),
    'NationalTaxRateDesc': sqlalchemy.types.String(50),
    'NationalTaxRate': sqlalchemy.types.Numeric(6, 5),
    'LocalTaxRateDesc': sqlalchemy.types.String(50),
    'LocalTaxRate': sqlalchemy.types.Numeric(6, 5),
    'AgencyID': sqlalchemy.types.String(30),
    'CreditRating': sqlalchemy.types.SmallInteger,
    'NetWorth': sqlalchemy.types.Numeric(10),
    'MarketingNameplate': sqlalchemy.types.String(100),
    'IsCurrent': sqlalchemy.types.Boolean,
    'BatchID': sqlalchemy.types.SmallInteger,
    'EffectiveDate': sqlalchemy.types.Date,
    'EndDate': sqlalchemy.types.Date
}
df_insert.to_sql('dimcustomer', engine, if_exists='append', index=False, dtype=sql_dtypes)

In [ ]:
update_customers_mask = df['CDC_FLAG'] == 'U'
keep_cols = [
    "SK_CustomerID", "CustomerID", "TaxID", "Status", "LastName", 
    "FirstName", "MiddleInitial", "Gender", "Tier", "DOB", 
    "AddressLine1", "AddressLine2", "PostalCode", "City", "StateProv", 
    "Country", "Phone1", "Phone2", "Phone3", "Email1", 
    "Email2", "NationalTaxRateDesc", "NationalTaxRate", "LocalTaxRateDesc", 
    "LocalTaxRate", "AgencyID", "CreditRating", "NetWorth", "MarketingNameplate", 
    "IsCurrent", "BatchID", "EffectiveDate", "EndDate"
]
df_update = df.loc[update_customers_mask, keep_cols]

In [ ]:
df_update

In [ ]:
for index, row in tqdm(df_update.iterrows(), total=df_update.shape[0]):
    customer_id = row['CustomerID']
    required_rows = df_update[df_update['CustomerID'].isin([customer_id])]
    # only one update per day
    if len(required_rows) > 1 and index != required_rows.index[-1]:
        continue
    effective_date = row['EffectiveDate']

    # Obtain the current SK_CustomerID from dimCustomer
    current_customer = pd.read_sql_query(
        f"SELECT SK_CustomerID, Status FROM dimCustomer WHERE IsCurrent = 1 AND CustomerID = {customer_id}",
        engine
    )

    sk_customer_id = current_customer['SK_CustomerID'].iloc[0]
    customer_status = current_customer['Status'].iloc[0]

    # Find all current records in dimAccount with the obtained SK_CustomerID
    current_accounts = pd.read_sql_query(
        f"SELECT * FROM dimAccount WHERE IsCurrent = 1 AND SK_CustomerID = {sk_customer_id}",
        engine
    )

    # Update these records by setting IsCurrent to 0 and EndDate to effective_date
    update_query = f"""
    UPDATE dimAccount
    SET IsCurrent = 0, EndDate = '{effective_date}'
    WHERE IsCurrent = 1 AND SK_CustomerID = {sk_customer_id};
    """
    with engine.connect() as connection:
        connection.execute(text(update_query))
        connection.commit()
    # Prepare new account records
    new_accounts = current_accounts.copy()
    new_accounts['EffectiveDate'] = effective_date
    new_accounts['EndDate'] = pd.Timestamp("9999-12-31")
    new_accounts['Status'] = new_accounts['Status'].apply(lambda x: 'INACTIVE' if customer_status == 'INACTIVE' else x)
    new_accounts['IsCurrent'] = 1
    new_accounts['BatchID'] = BATCH_ID
    max_sk = pd.read_sql("SELECT MAX(SK_AccountID) FROM dimAccount", engine).iloc[0, 0]
    new_accounts["SK_AccountID"] = range(max_sk + 1, max_sk + 1 + new_accounts.shape[0])

    # Bulk insert new account records
    new_accounts.to_sql('dimaccount', con=engine, if_exists='append', index=False, dtype={
        'SK_AccountID': sqlalchemy.types.Integer,
        'AccountID': sqlalchemy.types.Integer,
        'SK_BrokerID': sqlalchemy.types.Integer,
        'SK_CustomerID': sqlalchemy.types.Integer,
        'Status': sqlalchemy.types.String(10),
        'AccountDesc': sqlalchemy.types.String(50),
        'TaxStatus': sqlalchemy.types.SmallInteger,
        'IsCurrent': sqlalchemy.types.Boolean,
        'BatchID': sqlalchemy.types.SmallInteger,
        'EffectiveDate': sqlalchemy.types.Date,
        'EndDate': sqlalchemy.types.Date
    })

    # History-tracking update for the customer
    history_update_query = f"""
    UPDATE dimCustomer
    SET IsCurrent = 0, EndDate = '{effective_date}'
    WHERE IsCurrent = 1 AND CustomerID = {customer_id};
    """
    with engine.connect() as connection:
        connection.execute(text(history_update_query))
        connection.commit()

In [ ]:
# Convert DataFrame columns to the appropriate types
df_update['SK_CustomerID'] = df_update['SK_CustomerID'].astype(np.int32)
df_update['CustomerID'] = df_update['CustomerID'].astype(np.int32)
df_update['Tier'] = df_update['Tier'].astype(pd.Int8Dtype())
df_update['NationalTaxRate'] = df_update['NationalTaxRate'].astype(np.float64)
df_update['LocalTaxRate'] = df_update['LocalTaxRate'].astype(np.float64)
df_update['CreditRating'] = df_update['CreditRating'].astype(pd.Int16Dtype())
df_update['NetWorth'] = df_update['NetWorth'].astype(pd.Float64Dtype())
df_update['BatchID'] = df_update['BatchID'].astype(np.int16)

# Convert date columns to datetime.date
df_update['DOB'] = pd.to_datetime(df_update['DOB']).dt.date
df_update['EffectiveDate'] = pd.to_datetime(df_update['EffectiveDate']).dt.date
df_update['EndDate'] = pd.to_datetime(df_update['EndDate']).dt.date

In [ ]:
df_update.to_sql('dimcustomer', engine, if_exists='append', index=False, dtype=sql_dtypes)

In [ ]:
active_customers = pd.read_sql_query("""SELECT UPPER(FirstName) FirstName , UPPER(LastName) LastName, UPPER(AddressLine1) AddressLine1,
UPPER(AddressLine2) AddressLine2, UPPER(PostalCode) PostalCode FROM dimCustomer WHERE IsCurrent = 1 AND Status = 'ACTIVE';""", engine)
active_customers.info()

In [ ]:
# Create temporary uppercase columns for merging in both DataFrames
merge_fields = ["FirstName", "LastName", "AddressLine1", "AddressLine2", "PostalCode"]
merged_df = prospect_df_temp.merge(active_customers, how='left', on=merge_fields, indicator=True)

In [ ]:
mask = merged_df['_merge'] == 'both'
prospect_df_updated.loc[:, 'IsCustomer'] = mask

In [ ]:
prospect_df_updated['SK_UpdateDateID'] = prospect_df_updated['SK_UpdateDateID'].astype('uint32')

In [ ]:
agency_ids = pd.read_sql_query("SELECT DISTINCT AgencyID FROM Prospect", engine).iloc[:, 0]

In [ ]:
existing_records = prospect_df_updated[prospect_df_updated['AgencyID'].isin(agency_ids)]
new_records = prospect_df_updated[~prospect_df_updated['AgencyID'].isin(agency_ids)]

In [ ]:
# Load data from the MySQL table 'prospect'
prospect_df = pd.read_sql_table('prospect', con=engine)

# Merge the DataFrame with the MySQL table data on 'AgencyID'
merged_df = pd.merge(existing_records, prospect_df, on='AgencyID', suffixes=('_existing', '_prospect'))

# Define a function to compare rows, treating NaNs as equal
def compare_rows(row, columns):
    for col in columns:
        # Both values are NaN
        if pd.isna(row[col + '_existing']) and pd.isna(row[col + '_prospect']):
            continue
        # One value is NaN and the other is not
        elif pd.isna(row[col + '_existing']) or pd.isna(row[col + '_prospect']):
            return False
        # Both values are not NaN, but are different
        elif row[col + '_existing'] != row[col + '_prospect']:
            return False
    return True

# Filter to find records where all columns are the same except 'IsCustomer'
filter_columns = [col.replace('_existing', '') for col in merged_df.columns if 'IsCustomer' not in col and '_existing' in col]
diff_is_customer = merged_df[merged_df.apply(lambda row: compare_rows(row, filter_columns) and (pd.isna(row['IsCustomer_existing']) != pd.isna(row['IsCustomer_prospect']) or row['IsCustomer_existing'] != row['IsCustomer_prospect']), axis=1)]

# Count these records
count_diff_is_customer = diff_is_customer.shape[0]

In [ ]:
sql_dtypes = {
    'AgencyID': sqlalchemy.types.CHAR(30),
    'SK_RecordDateID': sqlalchemy.types.Integer,
    'SK_UpdateDateID': sqlalchemy.types.Integer,
    'BatchID': sqlalchemy.types.SmallInteger,
    'IsCustomer': sqlalchemy.types.Boolean,
    'LastName': sqlalchemy.types.CHAR(30),
    'FirstName': sqlalchemy.types.CHAR(30),
    'MiddleInitial': sqlalchemy.types.CHAR(1),
    'Gender': sqlalchemy.types.CHAR(1),
    'AddressLine1': sqlalchemy.types.CHAR(80),
    'AddressLine2': sqlalchemy.types.CHAR(80),
    'PostalCode': sqlalchemy.types.CHAR(12),
    'City': sqlalchemy.types.CHAR(25),
    'State': sqlalchemy.types.CHAR(20),
    'Country': sqlalchemy.types.CHAR(24),
    'Phone': sqlalchemy.types.CHAR(30),
    'Income': sqlalchemy.types.Integer,
    'NumberCars': sqlalchemy.types.SmallInteger,
    'NumberChildren': sqlalchemy.types.SmallInteger,
    'MaritalStatus': sqlalchemy.types.CHAR(1),
    'Age': sqlalchemy.types.SmallInteger,
    'CreditRating': sqlalchemy.types.SmallInteger,
    'OwnOrRentFlag': sqlalchemy.types.CHAR(1),
    'Employer': sqlalchemy.types.CHAR(30),
    'NumberCreditCards': sqlalchemy.types.SmallInteger,
    'NetWorth': sqlalchemy.types.BigInteger,
    'MarketingNameplate': sqlalchemy.types.CHAR(100)
}

new_records.to_sql('prospect', engine, if_exists='append', index=False, dtype=sql_dtypes)

In [ ]:
existing_records.head()

In [ ]:
with engine.connect() as connection:
    for _, row in tqdm(existing_records.iterrows(), total=existing_records.shape[0]):
        agencyid = row['AgencyID']
        query_parts = []
        for key, value in row.to_dict().items():
            if key != 'AgencyID':
                if pd.isna(value):
                    query_part = f"{key} = NULL"
                elif isinstance(value, str):
                    value = value.replace('"', '\\"')
                    query_part = f"{key} = \"{value}\""
                else:  # for numeric and boolean types
                    query_part = f"{key} = {value}"
                query_parts.append(query_part)
        query = f"UPDATE prospect SET {', '.join(query_parts)} WHERE AgencyID = \"{agencyid}\""
        connection.execute(text(query))
    connection.commit()

In [ ]:
num_rows = prospect_df_updated.shape[0]
message_type = "Status"
message_source = "Prospect"
message_text = "Source rows"
MessageDateAndTime = pd.Timestamp("now")

query = f"""INSERT INTO dimessages (MessageDateAndTime, BatchID, MessageSource, MessageText, MessageType, MessageData)
            VALUES ('{MessageDateAndTime}', {BATCH_ID}, '{message_source}', '{message_text}', '{message_type}', '{num_rows}')"""
with engine.connect() as conn:
    conn.execute(text(query))
    conn.commit()

In [ ]:
num_rows = new_records.shape[0]
message_type = "Status"
message_source = "Prospect"
message_text = "Inserted rows"
MessageDateAndTime = pd.Timestamp("now")

query = f"""INSERT INTO dimessages (MessageDateAndTime, BatchID, MessageSource, MessageText, MessageType, MessageData)
            VALUES ('{MessageDateAndTime}', {BATCH_ID}, '{message_source}', '{message_text}', '{message_type}', '{num_rows}')"""
with engine.connect() as conn:
    conn.execute(text(query))
    conn.commit()

In [ ]:
updated_rows = existing_records.shape[0] - count_diff_is_customer
message_type = "Status"
message_source = "Prospect"
message_text = "Updated rows"
MessageDateAndTime = pd.Timestamp("now")

query = f"""INSERT INTO dimessages (MessageDateAndTime, BatchID, MessageSource, MessageText, MessageType, MessageData)
            VALUES ('{MessageDateAndTime}', {BATCH_ID}, '{message_source}', '{message_text}', '{message_type}', '{updated_rows}')"""
with engine.connect() as conn:
    conn.execute(text(query))
    conn.commit()

### dimAccount

In [ ]:
df = pd.read_csv(
    DATA_DIR + "Account.txt",
    sep="|",
    header=None,
    names=["CDC_FLAG", "CDC_DSN", "CA_ID", "CA_B_ID", "CA_C_ID", "CA_NAME", "CA_TAX_ST", "CA_ST_ID"],
    dtype={
        "CDC_FLAG": "str",
        "CDC_DSN": "int64",
        "CA_ID": "int64",
        "CA_B_ID": "int64",
        "CA_C_ID": "int64",
        "CA_NAME": "str",
        "CA_TAX_ST": "uint8",
        "CA_ST_ID": "str"
    }
)
df.info()
df.head()

In [ ]:
df.rename({
    "CA_ID": "AccountID",
    "CA_NAME": "AccountDesc",
    "CA_TAX_ST": "TaxStatus"
}, axis=1, inplace=True)

In [ ]:
broker_mapping = pd.read_sql_query("SELECT BrokerID AS CA_B_ID, SK_BrokerID FROM dimbroker WHERE isCurrent = 1", engine).set_index("CA_B_ID")["SK_BrokerID"].to_dict()
customer_mapping = pd.read_sql_query("SELECT CustomerID AS CA_C_ID, SK_CustomerID FROM dimcustomer WHERE isCurrent = 1", engine).set_index("CA_C_ID")["SK_CustomerID"].to_dict()
status_mapping = pd.read_sql_query("SELECT ST_ID AS CA_ST_ID, ST_NAME FROM statustype", engine).set_index("CA_ST_ID")["ST_NAME"].to_dict()

In [ ]:
df.loc[:, 'SK_BrokerID'] = df['CA_B_ID'].map(broker_mapping)
df.loc[:, 'SK_CustomerID'] = df['CA_C_ID'].map(customer_mapping)
df.loc[:, 'Status'] = df['CA_ST_ID'].map(status_mapping)
df.loc[:, 'EffectiveDate'] = pd.to_datetime(BATCH_DATE)
df.loc[:, 'EndDate'] = pd.Timestamp("9999-12-31")
df.loc[:, 'BatchID'] = BATCH_ID
df.loc[:, 'IsCurrent'] = 1
df.info()
df.head()

In [ ]:
max_sk = pd.read_sql_query("SELECT MAX(SK_AccountID) FROM dimAccount", engine).iloc[0, 0]
df.loc[:, 'SK_AccountID'] = range(max_sk + 1, max_sk + 1 + df.shape[0])
df.info()

In [ ]:
drop_cols = ["CDC_FLAG", "CDC_DSN", "CA_B_ID", "CA_C_ID", "CA_ST_ID"]
df_insert = df[df["CDC_FLAG"] == "I"].drop(columns=drop_cols)
df_insert.info()
df_insert.head()

In [ ]:
sql_dtypes = {
    'SK_AccountID': sqlalchemy.types.Integer,
    'AccountID': sqlalchemy.types.Integer,
    'SK_BrokerID': sqlalchemy.types.Integer,
    'SK_CustomerID': sqlalchemy.types.Integer,
    'Status': sqlalchemy.types.String(10),
    'AccountDesc': sqlalchemy.types.String(50),
    'TaxStatus': sqlalchemy.types.SmallInteger,
    'IsCurrent': sqlalchemy.types.Boolean,
    'BatchID': sqlalchemy.types.SmallInteger,
    'EffectiveDate': sqlalchemy.types.Date,
    'EndDate': sqlalchemy.types.Date
}

In [ ]:
df_insert.to_sql('dimaccount', engine, if_exists='append', index=False, dtype=sql_dtypes)

In [ ]:
drop_cols = ["CDC_FLAG", "CDC_DSN", "CA_B_ID", "CA_C_ID", "CA_ST_ID"]
df_update = df[df["CDC_FLAG"] == "U"].drop(columns=drop_cols)

In [ ]:
with engine.connect() as connection:
    for index, row in df_update.iterrows():
        account_id = row['AccountID']
        required_rows = df_update[df_update['AccountID'].isin([account_id])]
        # only one update per day
        if len(required_rows) > 1 and index != required_rows.index[-1]:
            continue
        effective_date = row['EffectiveDate']
        query = f"""UPDATE dimAccount
        SET IsCurrent = 0, EndDate = '{effective_date}'
        WHERE IsCurrent = 1 AND AccountID = {account_id};"""
        connection.execute(text(query))
        connection.commit()

In [ ]:
df_update.to_sql('dimaccount', engine, if_exists='append', index=False, dtype=sql_dtypes)

In [ ]:
df_update.head()

### dimTrade

In [ ]:
column_names = [
    "CDC_FLAG", "CDC_DSN", "T_ID", "T_DTS", "T_ST_ID",
    "T_TT_ID", "T_IS_CASH", "T_S_SYMB", "T_QTY", "T_BID_PRICE",
    "T_CA_ID", "T_EXEC_NAME", "T_TRADE_PRICE", "T_CHRG", "T_COMM", "T_TAX"
]

data_types = {
    "CDC_FLAG": "category",
    "CDC_DSN": "int64",
    "T_ID": "int64",
    "T_DTS": "str",
    "T_ST_ID": "str",
    "T_TT_ID": "str",
    "T_IS_CASH": "boolean",
    "T_S_SYMB": "str",
    "T_QTY": "int64",
    "T_BID_PRICE": "float64",
    "T_CA_ID": "int64",
    "T_EXEC_NAME": "str",
    "T_TRADE_PRICE": "float64",
    "T_CHRG": "float64",
    "T_COMM": "float64",
    "T_TAX": "float64"
}

df = pd.read_csv(DATA_DIR + "Trade.txt", sep='|', header=None, names=column_names, dtype=data_types, parse_dates=["T_DTS"])
df.info()
df.head()

In [ ]:
date_mapping = pd.read_sql_query("SELECT SK_DateID, DateValue FROM DimDate", engine).set_index('DateValue')['SK_DateID'].to_dict()
time_mapping = pd.read_sql_query("SELECT SK_TimeID, TimeValue FROM DimTime", engine)

def timedelta_to_time(td):
    return (datetime.min + td).time()

# Applying the function to the series
time_mapping['TimeValue'] = time_mapping['TimeValue'].apply(timedelta_to_time)
time_mapping = time_mapping.set_index('TimeValue')['SK_TimeID'].to_dict()

In [ ]:
# If this is a new Trade record (CDC_FLAG = “I”) then SK_CreateDateID and 
# SK_CreateTimeID must be set based on T_DTS. SK_CloseDateID and SK_CloseTimeID must be set to NULL
insert_mask = df['CDC_FLAG'] == 'I'
df.loc[insert_mask, 'SK_CreateDateID'] = df.loc[insert_mask, 'T_DTS'].dt.date.map(date_mapping)
df.loc[insert_mask, 'SK_CreateTimeID'] = df.loc[insert_mask, 'T_DTS'].dt.time.map(time_mapping)
df.loc[insert_mask, 'SK_CloseDateID'] = None
df.loc[insert_mask, 'SK_CloseTimeID'] = None
df.info()
df.loc[insert_mask, ['SK_CreateDateID', 'SK_CreateTimeID', 'SK_CloseDateID', 'SK_CloseTimeID']].head()

In [ ]:
# If T_ST_ID is “CMPT” or “CNCL”, SK_CloseDateID and SK_CloseTimeID must be set based on T_DTS.
close_mask = df['T_ST_ID'].isin(['CMPT', 'CNCL'])
df.loc[close_mask, 'SK_CloseDateID'] = df.loc[close_mask, 'T_DTS'].dt.date.map(date_mapping)
df.loc[close_mask, 'SK_CloseTimeID'] = df.loc[close_mask, 'T_DTS'].dt.time.map(time_mapping)

In [ ]:
df.rename(columns={
    'T_ID': 'TradeID',
    'T_IS_CASH': 'CashFlag',
    'T_QTY': 'Quantity',
    'T_BID_PRICE': 'BidPrice',
    'T_EXEC_NAME': 'ExecutedBy',
    'T_TRADE_PRICE': 'TradePrice',
    'T_CHRG': 'Fee',
    'T_COMM': 'Commission',
    'T_TAX': 'Tax'
}, inplace=True)

In [ ]:
status_mapping = pd.read_sql("SELECT ST_ID, ST_NAME FROM statustype", engine).set_index("ST_ID")["ST_NAME"].to_dict()
trade_type_mapping = pd.read_sql("SELECT TT_ID, TT_NAME FROM tradetype", engine).set_index("TT_ID")["TT_NAME"].to_dict()
df["Status"] = df["T_ST_ID"].map(status_mapping)
df["Type"] = df["T_TT_ID"].map(trade_type_mapping)

In [ ]:
# Fetching security and account info in one go
security_info = pd.read_sql("SELECT Symbol, SK_SecurityID, SK_CompanyID FROM dimsecurity WHERE IsCurrent = 1", engine).set_index('Symbol')
account_info = pd.read_sql("SELECT AccountID, SK_AccountID, SK_CustomerID, SK_BrokerID FROM dimaccount WHERE IsCurrent = 1",
                           engine).set_index('AccountID')

In [ ]:
df.loc[:, 'SK_SecurityID'] = df.loc[:, 'T_S_SYMB'].map(security_info['SK_SecurityID'].to_dict())
df.loc[:, 'SK_CompanyID'] = df.loc[:, 'T_S_SYMB'].map(security_info['SK_CompanyID'].to_dict())
df.loc[:, 'SK_AccountID'] = df.loc[:, 'T_CA_ID'].map(account_info['SK_AccountID'].to_dict())
df.loc[:, 'SK_CustomerID'] = df.loc[:, 'T_CA_ID'].map(account_info['SK_CustomerID'].to_dict())
df.loc[:, 'SK_BrokerID'] = df.loc[:, 'T_CA_ID'].map(account_info['SK_BrokerID'].to_dict())
df.info()

In [ ]:
df['BatchID'] = BATCH_ID

In [ ]:
# Filter the DataFrame
invalid_trades = df[
    (df["Commission"].notnull())
    & (df["Commission"] > (df["TradePrice"] * df["Quantity"]))
]

if invalid_trades.shape[0] > 0:
    # Create lists without using iterrows
    MessageSource = ["DimTrade"] * len(invalid_trades)
    MessageType = ["Alert"] * len(invalid_trades)
    MessageText = ["Invalid trade commission"] * len(invalid_trades)
    MessageData = [
        "T_ID = "
        + invalid_trades["TradeID"].astype(str)
        + ", T_COMM = "
        + invalid_trades["Commission"].astype(str)
    ]
    # Convert MessageData from a list of Series to a list of strings
    MessageData = MessageData[0].tolist()
    
    query = f"""INSERT INTO Dimessages (MessageDateAndTime, BatchID, MessageSource, MessageText, MessageType, MessageData)
    VALUES """
    for i in range(len(MessageSource)):
        query += f"""('{pd.Timestamp("now")}', {BATCH_ID}, '{MessageSource[i]}', '{MessageText[i]}', '{MessageType[i]}', '{MessageData[i]}'),"""
    with engine.connect() as conn:
        conn.execute(text(query[:-1]))
        conn.commit()

In [ ]:
# Filter the DataFrame for invalid trade fees
invalid_fee_trades = df[
    (df["Fee"].notnull())
    & (df["Fee"] > (df["TradePrice"] / df["Quantity"]))
]

if len(invalid_fee_trades) > 0:
    # Create the required lists
    MessageSource = ["DimTrade"] * len(invalid_fee_trades)
    MessageType = ["Alert"] * len(invalid_fee_trades)
    MessageText = ["Invalid trade fee"] * len(invalid_fee_trades)
    
    # Vectorized operation for MessageData
    MessageData = (
        "T_ID = "
        + invalid_fee_trades["TradeID"].astype(str)
        + ", T_CHRG = "
        + invalid_fee_trades["Fee"].astype(str)
    )
    MessageData = MessageData.tolist()
    
    query = """INSERT INTO Dimessages (MessageDateAndTime, BatchID, MessageSource, MessageText, MessageType, MessageData) VALUES """
    for i in range(len(MessageSource)):
        query += f"""('{pd.Timestamp("now")}', {BATCH_ID}, '{MessageSource[i]}', '{MessageText[i]}', '{MessageType[i]}', '{MessageData[i]}'),"""
    with engine.connect() as conn:
        conn.execute(text(query[:-1]))
        conn.commit()

In [ ]:
sql_dtypes = {
    'TradeID': sqlalchemy.types.Integer,
    'SK_BrokerID': sqlalchemy.types.Integer,
    'SK_CreateDateID': sqlalchemy.types.Integer,
    'SK_CreateTimeID': sqlalchemy.types.Integer,
    'SK_CloseDateID': sqlalchemy.types.Integer,
    'SK_CloseTimeID': sqlalchemy.types.Integer,
    'Status': sqlalchemy.types.CHAR(10),
    'Type': sqlalchemy.types.CHAR(12),
    'CashFlag': sqlalchemy.types.Boolean,
    'SK_SecurityID': sqlalchemy.types.Integer,
    'SK_CompanyID': sqlalchemy.types.Integer,
    'Quantity': sqlalchemy.types.Integer,
    'BidPrice': sqlalchemy.types.Numeric(8, 2),
    'SK_CustomerID': sqlalchemy.types.Integer,
    'SK_AccountID': sqlalchemy.types.Integer,
    'ExecutedBy': sqlalchemy.types.CHAR(64),
    'TradePrice': sqlalchemy.types.Numeric(8, 2),
    'Fee': sqlalchemy.types.Numeric(10, 2),
    'Commission': sqlalchemy.types.Numeric(10, 2),
    'Tax': sqlalchemy.types.Numeric(10, 2),
    'BatchID': sqlalchemy.types.SmallInteger
}
keep_cols = [col for col in df.columns if col in sql_dtypes]

In [ ]:
df.info()

In [ ]:
df_insert = df.loc[df["CDC_FLAG"] == "I", keep_cols]
df_insert.info()

In [ ]:
df_insert.to_sql('dimtrade', engine, if_exists='append', index=False, dtype=sql_dtypes)

In [ ]:
df_update = df.loc[df["CDC_FLAG"] == "U", keep_cols]
df_update.info()

In [ ]:
with engine.connect() as connection:
    for index, row in df_update.iterrows():
        trade_id = row['TradeID']
        required_rows = df_update[df_update['TradeID'] == trade_id]
        # only update using latest value
        if len(required_rows) > 1 and index != required_rows.index[-1]:
            continue
        str_cols = ['Status', 'Type']
        query_parts = []
        for key, value in row.to_dict().items():
            if key in ('TradeID', 'SK_CreateDateID', 'SK_CreateTimeID'):
                continue
            if pd.isna(value):
                query_part = f"{key} = NULL"
            elif key in str_cols:
                value = value.replace("'", "\\'")
                query_part = f"{key} = '{value}'"
            else:  # for numeric and boolean types
                query_part = f"{key} = {value}"
            query_parts.append(query_part)
        query = f"UPDATE dimtrade SET {', '.join(query_parts)} WHERE TradeID = {trade_id};"
        connection.execute(text(query))
    connection.commit()

### FactCashBalances

In [ ]:
df = pd.read_csv(
    DATA_DIR + "CashTransaction.txt",
    sep="|",
    header=None,
    names=[
        "CDC_FLAG",
        "CDC_DSN",
        "CT_CA_ID",
        "CT_DTS",
        "CT_AMT",
        "CT_NAME"
    ],
    dtype={
        "CDC_FLAG": "category",
        "CDC_DSN": "int64",
        "CT_CA_ID": "uint32",
        "CT_DTS": "str",
        "CT_AMT": "float64",
        "CT_NAME": "str"
    },
    parse_dates=["CT_DTS"],
)
df.info()
df.head()

In [ ]:
account_info = pd.read_sql(
    "SELECT AccountID, SK_AccountID, SK_CustomerID FROM dimaccount WHERE IsCurrent = 1",
    engine,
).set_index('AccountID')
df.loc[:, 'SK_AccountID'] = df.loc[:, 'CT_CA_ID'].map(account_info['SK_AccountID'].to_dict())
df.loc[:, 'SK_CustomerID'] = df.loc[:, 'CT_CA_ID'].map(account_info['SK_CustomerID'].to_dict())

date_info = pd.read_sql("SELECT DateValue, SK_DateID FROM dimdate", engine)
date_info["DateValue"] = pd.to_datetime(date_info["DateValue"])
df.loc[:, 'SK_DateID'] = df.loc[:, 'CT_DTS'].dt.date.map(date_info.set_index('DateValue')['SK_DateID'])

In [ ]:
df.loc[:, 'BatchID'] = BATCH_ID

In [ ]:
# Sort the DataFrame by account ID and transaction date
df.sort_values(by=['CT_CA_ID', 'CT_DTS'], inplace=True)

# Create a new column to store the prior cash amount
df['PriorCash'] = df.groupby('CT_CA_ID')['CT_AMT'].cumsum() - df['CT_AMT']
df['PriorCash'].fillna(0, inplace=True)

# Calculate the cash balance
df['Cash'] = df['PriorCash'] + df['CT_AMT']
df = df.groupby(['CT_CA_ID', 'SK_DateID']).last().reset_index()

In [ ]:
sql_dtypes = {
    "SK_CustomerID": sqlalchemy.types.Integer,
    "SK_AccountID": sqlalchemy.types.Integer,
    "SK_DateID": sqlalchemy.types.Integer,
    "Cash": sqlalchemy.types.DECIMAL(precision=15, scale=2),
    "BatchID": sqlalchemy.types.SmallInteger
}
keep_cols = [col for col in df.columns if col in sql_dtypes]

In [ ]:
df_insert = df.loc[df["CDC_FLAG"] == "I", keep_cols]
df_insert.info()

In [ ]:
df_insert.to_sql('factcashbalances', engine, if_exists='append', index=False, dtype=sql_dtypes)

### FactHoldings


In [ ]:
# SQL queries
sql_commands = [
    "DROP TABLE IF EXISTS TempHoldingHistory",
    """
    CREATE TEMPORARY TABLE TempHoldingHistory (
        CDC_FLAG CHAR(1) NOT NULL,
        CDC_DSN INT UNSIGNED NOT NULL,
        HH_H_T_ID INT UNSIGNED NOT NULL,
        HH_T_ID INT UNSIGNED NOT NULL,
        HH_BEFORE_QTY INT NOT NULL,
        HH_AFTER_QTY INT NOT NULL
    )
    """,
    f"""
    LOAD DATA LOCAL INFILE 'E:\\\\Documents\\\\BDMA\\\\ULB\\\\Data Warehouses\\\\tpc-di\\\\TPC-DI\\\\data\\\\sf5\\\\Batch{BATCH_ID}\\\\HoldingHistory.txt'
    INTO TABLE TempHoldingHistory
    FIELDS TERMINATED BY '|'
    LINES TERMINATED BY '\n'
    (CDC_FLAG, CDC_DSN, HH_H_T_ID, HH_T_ID, HH_BEFORE_QTY, HH_AFTER_QTY)
    """,
    f"""
    INSERT INTO FactHoldings (TradeID, CurrentTradeID, SK_CustomerID, SK_AccountID, SK_SecurityID, SK_CompanyID, SK_DateID, SK_TimeID, CurrentPrice, CurrentHolding, BatchID)
    SELECT 
        thh.HH_H_T_ID AS TradeID,
        thh.HH_T_ID AS CurrentTradeID,
        dt.SK_CustomerID,
        dt.SK_AccountID,
        dt.SK_SecurityID,
        dt.SK_CompanyID,
        dt.SK_CloseDateID AS SK_DateID,
        dt.SK_CloseTimeID AS SK_TimeID,
        dt.TradePrice AS CurrentPrice,
        thh.HH_AFTER_QTY AS CurrentHolding,
        {BATCH_ID} AS BatchID
    FROM 
        TempHoldingHistory thh
    JOIN 
        DimTrade dt ON thh.HH_T_ID = dt.TradeID
    """,
    "DROP TABLE TempHoldingHistory"
]

In [ ]:
# Executing the queries
with engine.connect() as connection:
    for sql in sql_commands:
        connection.execute(text(sql))
    connection.commit()

### FactWatches

In [ ]:
df = pd.read_csv(
    DATA_DIR + "WatchHistory.txt",
    sep="|",
    header=None,
    names=[
        "CDC_FLAG",
        "CDC_DSN",
        "W_C_ID",
        "W_S_SYMB",
        "W_DTS",
        "W_ACTION"
    ],
    dtype={
        "CDC_FLAG": "category",
        "CDC_DSN": "int64",
        "W_C_ID": "uint32",
        "W_S_SYMB": "str",
        "W_DTS": "str",
        "W_ACTION": "str"
    },
    parse_dates=["W_DTS"]
)
df.info()
df.head()

In [ ]:
customer_mapping = pd.read_sql_query(
    "SELECT CustomerID, SK_CustomerID FROM dimcustomer WHERE IsCurrent = 1",
    engine).set_index('CustomerID')['SK_CustomerID'].to_dict()

security_mapping = pd.read_sql_query(
    "SELECT Symbol, SK_SecurityID FROM dimsecurity WHERE IsCurrent = 1",
    engine).set_index('Symbol')['SK_SecurityID'].to_dict()

date_info = pd.read_sql_query("SELECT DateValue, SK_DateID FROM dimdate", engine)
date_info['DateValue'] = pd.to_datetime(date_info['DateValue'])
date_mapping = date_info.set_index('DateValue')['SK_DateID'].to_dict()

In [ ]:
df.loc[:, 'SK_CustomerID'] = df.loc[:, 'W_C_ID'].map(customer_mapping)
df.loc[:, 'SK_SecurityID'] = df.loc[:, 'W_S_SYMB'].map(security_mapping)
df.loc[:, 'SK_DateID_DatePlaced'] = df.loc[:, 'W_DTS'].dt.date.map(date_mapping)
df.loc[:, 'BatchID'] = BATCH_ID

In [ ]:
# Mask for rows where W_ACTION is 'CNCL'
mask_cncl = df['W_ACTION'] == 'CNCL'
df.loc[mask_cncl, 'SK_DateID_DateRemoved'] = df.loc[mask_cncl, 'W_DTS'].dt.date.map(date_mapping)
df.head()

In [ ]:
df = df.groupby(["W_C_ID", "W_S_SYMB"]).first().reset_index()
df.info()

In [ ]:
new_records_dict = {'SK_CustomerID': [], 'SK_SecurityID': [], 'SK_DateID_DatePlaced': [], 'SK_DateID_DateRemoved': [], 'BatchID': []}
lst = [0, 0, 0]

def update_or_insert(row, new_records):

    with engine.connect() as connection:
        # Check if the record exists in factwatches
        existing = pd.read_sql_query(
            "SELECT * FROM factwatches WHERE SK_CustomerID = {} AND SK_SecurityID = {}".format(row['SK_CustomerID'], row['SK_SecurityID']),
            connection
        )

        # Handle NaN in SK_DateID_DateRemoved
        date_removed = "NULL" if pd.isna(row['SK_DateID_DateRemoved']) else row['SK_DateID_DateRemoved']

        if not existing.empty:
            # Update the existing record
            query = """
            UPDATE factwatches
            SET SK_DateID_DatePlaced = {}, SK_DateID_DateRemoved = {}, BatchID = {}
            WHERE SK_CustomerID = {} AND SK_SecurityID = {}
            """.format(row['SK_DateID_DatePlaced'], date_removed, row['BatchID'], row['SK_CustomerID'], row['SK_SecurityID'])
            connection.execute(text(query))
            lst[0] += 1
            connection.commit()
        else:
            # Check for matching records in dimcustomer and dimsecurity
            customer_sk = pd.read_sql_query(
                "SELECT SK_CustomerID FROM dimcustomer WHERE CustomerID = {}".format(row['W_C_ID']),
                connection
            )
            security_sk = pd.read_sql_query(
                "SELECT SK_SecurityID FROM dimsecurity WHERE Symbol = '{}'".format(row['W_S_SYMB']),
                connection
            )

            # Check for each combination of SKs in factwatches
            for cust_id in customer_sk['SK_CustomerID']:
                for sec_id in security_sk['SK_SecurityID']:
                    existing = pd.read_sql_query(
                        "SELECT * FROM factwatches WHERE SK_CustomerID = {} AND SK_SecurityID = {}".format(cust_id, sec_id),
                        connection
                    )
                    if not existing.empty:
                        # Update the existing record with new values
                        query = """
                        UPDATE factwatches
                        SET SK_DateID_DatePlaced = {}, SK_DateID_DateRemoved = {}, BatchID = {}
                        WHERE SK_CustomerID = {} AND SK_SecurityID = {}
                        """.format(row['SK_DateID_DatePlaced'], date_removed, row['BatchID'], cust_id, sec_id)
                        connection.execute(text(query))
                        connection.commit()
                        lst[1] += 1
                        return

            # Accumulate new record for bulk insert
            new_records_dict['SK_CustomerID'].append(row['SK_CustomerID'])
            new_records_dict['SK_SecurityID'].append(row['SK_SecurityID'])
            new_records_dict['SK_DateID_DatePlaced'].append(row['SK_DateID_DatePlaced'])
            new_records_dict['SK_DateID_DateRemoved'].append(row['SK_DateID_DateRemoved'] if pd.notna(row['SK_DateID_DateRemoved']) else None)
            new_records_dict['BatchID'].append(row['BatchID'])
            lst[2] += 1

In [ ]:
# Iterate over each row in the DataFrame and accumulate new records
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    update_or_insert(row, new_records)

In [ ]:
sql_dtypes = {
    "SK_CustomerID": sqlalchemy.types.Integer,
    "SK_SecurityID": sqlalchemy.types.Integer,
    "SK_DateID_DatePlaced": sqlalchemy.types.Integer,
    "SK_DateID_DateRemoved": sqlalchemy.types.Integer,
    "BatchID": sqlalchemy.types.SmallInteger
}
# Bulk insert new records
new_records_df = pd.DataFrame(new_records_dict)
new_records_df.to_sql('factwatches', con=engine, if_exists='append', index=False, dtype=sql_dtypes)

### FactMarketHistory

In [ ]:
df = pd.read_csv(
    DATA_DIR + "DailyMarket.txt",
    sep="|",
    header=None,
    names=[
        "CDC_FLAG",
        "CDC_DSN",
        "DM_DATE",
        "DM_S_SYMB",
        "DM_CLOSE",
        "DM_HIGH",
        "DM_LOW",
        "DM_VOL",
    ],
    dtype={
        "CDC_FLAG": "category",
        "CDC_DSN": "int64",
        "DM_DATE": "str",
        "DM_S_SYMB": "str",
        "DM_CLOSE": "float32",
        "DM_HIGH": "float32",
        "DM_LOW": "float32",
        "DM_VOL": "int64",
    },
    parse_dates=["DM_DATE"],
)
df.info()
df.head()

In [ ]:
# ClosePrice, DayHigh, DayLow, and Volume are copied from DM_CLOSE, DM_HIGH,
# DM_LOW, and DM_VOL respectively.
df["ClosePrice"] = df["DM_CLOSE"]
df["DayHigh"] = df["DM_HIGH"]
df["DayLow"] = df["DM_LOW"]
df["Volume"] = df["DM_VOL"]

In [ ]:
security_info = pd.read_sql("SELECT Symbol, SK_SecurityID, SK_CompanyID FROM dimsecurity WHERE IsCurrent = 1",
                            engine).set_index("Symbol")
security_mapping = security_info['SK_SecurityID'].to_dict()
company_mapping = security_info['SK_CompanyID'].to_dict()

df.loc[:, 'SK_SecurityID'] = df.loc[:, 'DM_S_SYMB'].map(security_mapping)
df.loc[:, 'SK_CompanyID'] = df.loc[:, 'DM_S_SYMB'].map(company_mapping)

date_info = pd.read_sql_query("SELECT DateValue, SK_DateID FROM dimdate", engine)
date_info['DateValue'] = pd.to_datetime(date_info['DateValue'])
date_mapping = date_info.set_index('DateValue')['SK_DateID'].to_dict()
df.loc[:, 'SK_DateID'] = df.loc[:, 'DM_DATE'].dt.date.map(date_mapping)

In [ ]:
# Step 2: Sort the DataFrame
df.sort_values(by='DM_DATE', inplace=True)

# Step 3 & 4: Group by 'DM_S_SYMB' and apply rolling max
rolling_max = df.groupby('DM_S_SYMB').rolling('365D', on='DM_DATE')['DM_HIGH'].max()

# Reset index to make merging easier
rolling_max = rolling_max.reset_index()

# Step 5: Merge with the original DataFrame
df = df.merge(rolling_max, on=['DM_S_SYMB', 'DM_DATE'], suffixes=('', '_52WeekHigh'))

# Rename the column for clarity
df.rename(columns={'DM_HIGH_52WeekHigh': 'FiftyTwoWeekHigh'}, inplace=True)

rolling_rank = (
    df.groupby("DM_S_SYMB")
    .rolling("365D", on="DM_DATE")["DM_HIGH"]
    .rank(method="average", ascending=False)
    .reset_index()
    .rename(columns={"DM_HIGH": "Rank"})
)
rolling_rank["Rank"] = rolling_rank["Rank"].astype("uint32")
# Apply the mask to select DM_DATE only for those rows, then forward fill
mask = rolling_rank['Rank'] == 1
rolling_rank['SK_FiftyTwoWeekHighDate'] = rolling_rank['DM_DATE'].where(mask).ffill()
rolling_rank['SK_FiftyTwoWeekHighDate'] = rolling_rank['SK_FiftyTwoWeekHighDate'].dt.date.map(date_mapping)

df = pd.concat([df, rolling_rank['SK_FiftyTwoWeekHighDate']], axis=1)

df.sort_values(by='DM_DATE', inplace=True)
# Step 3 & 4: Group by 'DM_S_SYMB' and apply rolling min
rolling_min = df.groupby('DM_S_SYMB').rolling('365D', on='DM_DATE')['DM_LOW'].min()
# Reset index to make merging easier
rolling_min = rolling_min.reset_index()
# Step 5: Merge with the original DataFrame
df = df.merge(rolling_min, on=['DM_S_SYMB', 'DM_DATE'], suffixes=('', '_52WeekLow'))
# Rename the column for clarity
df.rename(columns={'DM_LOW_52WeekLow': 'FiftyTwoWeekLow'}, inplace=True)

rolling_rank = (
    df.groupby("DM_S_SYMB")
    .rolling("365D", on="DM_DATE")["DM_LOW"]
    .rank(method="average", ascending=True)
    .reset_index()
    .rename(columns={"DM_LOW": "Rank"})
)
rolling_rank["Rank"] = rolling_rank["Rank"].astype("uint32")
# Apply the mask to select DM_DATE only for those rows, then forward fill
mask = rolling_rank['Rank'] == 1
rolling_rank['SK_FiftyTwoWeekLowDate'] = rolling_rank['DM_DATE'].where(mask).ffill()
rolling_rank['SK_FiftyTwoWeekLowDate'] = rolling_rank['SK_FiftyTwoWeekLowDate'].dt.date.map(date_mapping)

df = pd.concat([df, rolling_rank['SK_FiftyTwoWeekLowDate']], axis=1)

In [ ]:
df['SK_SecurityID'] = df['SK_SecurityID'].astype('uint32')
df['SK_CompanyID'] = df['SK_CompanyID'].astype('uint32')
df['SK_DateID'] = df['SK_DateID'].astype('uint32')
df['FiftyTwoWeekHigh'] = df['FiftyTwoWeekHigh'].astype('float32')
df['SK_FiftyTwoWeekHighDate'] = df['SK_FiftyTwoWeekHighDate'].astype('uint32')
df['FiftyTwoWeekLow'] = df['FiftyTwoWeekLow'].astype('float32')
df['SK_FiftyTwoWeekLowDate'] = df['SK_FiftyTwoWeekLowDate'].astype('uint32')
df['DM_S_SYMB'] = df['DM_S_SYMB'].astype('category')

df.drop(columns=['DM_HIGH', 'DM_LOW', 'DM_VOL', 'DM_CLOSE'], inplace=True)

In [ ]:
security_mapping = pd.read_sql("SELECT Symbol, Dividend FROM dimsecurity WHERE IsCurrent = 1", engine).set_index("Symbol")["Dividend"]
df.loc[:, 'Yield'] = df.loc[:, 'DM_S_SYMB'].map(security_mapping) / df.loc[:, 'ClosePrice'] * 100

In [ ]:
df['BatchID'] = BATCH_ID

In [ ]:
sql_dtypes = {
    "SK_SecurityID": sqlalchemy.types.Integer,
    "SK_CompanyID": sqlalchemy.types.Integer,
    "SK_DateID": sqlalchemy.types.Integer,
    "Yield": sqlalchemy.types.DECIMAL(precision=5, scale=2),
    "FiftyTwoWeekHigh": sqlalchemy.types.DECIMAL(precision=8, scale=2),
    "SK_FiftyTwoWeekHighDate": sqlalchemy.types.Integer,
    "FiftyTwoWeekLow": sqlalchemy.types.DECIMAL(precision=8, scale=2),
    "SK_FiftyTwoWeekLowDate": sqlalchemy.types.Integer,
    "ClosePrice": sqlalchemy.types.DECIMAL(precision=8, scale=2),
    "DayHigh": sqlalchemy.types.DECIMAL(precision=8, scale=2),
    "DayLow": sqlalchemy.types.DECIMAL(precision=8, scale=2),
    "Volume": sqlalchemy.types.BigInteger,
    "BatchID": sqlalchemy.types.SmallInteger,
    "DM_DATE": sqlalchemy.types.Date,
    "DM_S_SYMB": sqlalchemy.types.CHAR(16),
}

In [ ]:
df.drop(columns=["CDC_FLAG", "CDC_DSN"], inplace=True)

In [ ]:
df.to_sql('tempfactmarketprice', engine, if_exists='replace', index=False, dtype=sql_dtypes)

In [ ]:
sql_commands = [
    "CREATE INDEX idx_sk_companyid ON tempfactmarketprice(SK_CompanyID);",
    """SELECT fmp.ClosePrice / T.Sum_EPS AS PERatio
        FROM tempfactmarketprice fmp
        LEFT JOIN (SELECT 
            c.CompanyID, 
            c.SK_CompanyID AS SKCID, 
            f.FI_QTR_START_DATE,
            SUM(f.FI_BASIC_EPS) OVER (
                PARTITION BY c.CompanyID 
                ORDER BY f.FI_QTR_START_DATE 
                ROWS BETWEEN 3 PRECEDING AND CURRENT ROW
            ) AS Sum_EPS
        FROM financial f RIGHT JOIN dimCompany c ON f.SK_CompanyID = c.SK_CompanyID
        ORDER BY c.CompanyID, f.FI_QTR_START_DATE) T
        ON T.SKCID = fmp.SK_CompanyID
        AND T.FI_QTR_START_DATE < fmp.DM_DATE 
        AND T.FI_QTR_START_DATE >= DATE_SUB(fmp.DM_DATE, INTERVAL 3 MONTH);""",
    "DROP TABLE tempfactmarketprice;"
]

In [ ]:
with engine.connect() as connection:
    connection.execute(text(sql_commands[0]))
    connection.commit()
peratio = pd.read_sql_query(sql_commands[1], engine)
with engine.connect() as connection:
    connection.execute(text(sql_commands[2]))
    connection.commit()
peratio.info()
peratio

In [ ]:
df = pd.concat([df, peratio], axis=1)

In [ ]:
no_earnings = pd.DataFrame(df.loc[df['PERatio'].isna(), 'DM_S_SYMB'])
no_earnings['MessageDateAndTime'] = datetime.now()
no_earnings['BatchID'] = BATCH_ID
no_earnings['MessageSource'] = 'FactMarketHistory'
no_earnings['MessageText'] = 'No earnings for company'
no_earnings['MessageType'] = 'Alert'
no_earnings.rename(columns={"DM_S_SYMB":"MessageData"}, inplace=True)
no_earnings['MessageData'] = "DM_S_SYMB = " + no_earnings['MessageData'].astype(str)

sql_dtypes = {
    "MessageDateAndTime": sqlalchemy.types.DATETIME,
    "BatchID": sqlalchemy.types.Integer,
    "MessageSource": sqlalchemy.types.CHAR(30),
    "MessageText": sqlalchemy.types.CHAR(50),
    "MessageType": sqlalchemy.types.CHAR(12),
    "MessageData": sqlalchemy.types.CHAR(100)
}
no_earnings.to_sql('dimessages', con=engine, if_exists='append', index=False, dtype=sql_dtypes)

In [ ]:
df.drop(columns=["DM_DATE", "DM_S_SYMB"], inplace=True)
df.info()

In [ ]:
sql_dtypes = {
    "SK_SecurityID": sqlalchemy.types.Integer,
    "SK_CompanyID": sqlalchemy.types.Integer,
    "SK_DateID": sqlalchemy.types.Integer,
    "PERatio": sqlalchemy.types.DECIMAL(precision=10, scale=2),
    "Yield": sqlalchemy.types.DECIMAL(precision=5, scale=2),
    "FiftyTwoWeekHigh": sqlalchemy.types.DECIMAL(precision=8, scale=2),
    "SK_FiftyTwoWeekHighDate": sqlalchemy.types.Integer,
    "FiftyTwoWeekLow": sqlalchemy.types.DECIMAL(precision=8, scale=2),
    "SK_FiftyTwoWeekLowDate": sqlalchemy.types.Integer,
    "ClosePrice": sqlalchemy.types.DECIMAL(precision=8, scale=2),
    "DayHigh": sqlalchemy.types.DECIMAL(precision=8, scale=2),
    "DayLow": sqlalchemy.types.DECIMAL(precision=8, scale=2),
    "Volume": sqlalchemy.types.BigInteger,
    "BatchID": sqlalchemy.types.SmallInteger,
}
len(sql_dtypes)

In [ ]:
df.to_sql('factmarkethistory', con=engine, if_exists='append', index=False, dtype=sql_dtypes)

## Audit

In [ ]:
BATCH_DIRS = ['..\\data\\sf5\\Batch1\\', '..\\data\\sf5\\Batch2\\', '..\\data\\sf5\\Batch3\\']
BATCH_FILES = ['..\\data\\sf5\\Batch1_audit.csv', '..\\data\\sf5\\Batch2_audit.csv', '..\\data\\sf5\\Batch3_audit.csv']
audit_dtypes = {"DataSet": str, " BatchID": int, "Date": 'datetime64', " Attribute": str, " Value": 'Int64', " DValue": 'float64'}

In [ ]:
df = pd.read_csv(BATCH_FILES[0], dtype=audit_dtypes)
df = pd.concat([df, pd.read_csv(BATCH_FILES[1], dtype=audit_dtypes)])
df = pd.concat([df, pd.read_csv(BATCH_FILES[2], dtype=audit_dtypes)])
df.info()

In [ ]:
df = pd.concat([df, pd.read_csv(r"..\data\sf5\Generator_audit.csv", dtype=audit_dtypes)])

In [ ]:
for folder in BATCH_DIRS:
    files = os.listdir(folder)
    files = list(filter(lambda x: "_audit" in x, files))
    for file in files:
        df = pd.concat([df, pd.read_csv(folder + file, dtype=audit_dtypes)])
df.info()

In [ ]:
create_table = """drop table audit"""
with engine.connect() as cxn:
    cxn.execute(text(create_table))
    cxn.commit()

In [ ]:
create_table = """drop table audit;"""
with engine.connect() as cxn:
    cxn.execute(text(create_table))
    cxn.commit()

In [ ]:
create_table = """CREATE TABLE Audit (
    DataSet CHAR(20) NOT NULL,
    BatchID SMALLINT UNSIGNED,
    Date DATE,
    Attribute CHAR(50) NOT NULL,
    Value BIGINT,
    DValue NUMERIC(15, 5)
);"""
with engine.connect() as cxn:
    cxn.execute(text(create_table))
    cxn.commit()

In [ ]:
df.columns = df.columns.str.strip()
df.columns

In [ ]:
df.to_sql('audit', engine, if_exists='replace', index=False)

In [ ]:
audit = pd.read_sql_table('audit', engine)
audit.info()

In [ ]:
with open(r"E:\Documents\BDMA\ULB\Data Warehouses\tpc-di\TPC-DI\validation\tpcdi_audit.sql", encoding='utf-8') as f:
    content = f.read()

In [ ]:
len(content.split("union"))